In [31]:
# ======================================================================
# 0. INSTALACIÓN (si hace falta, solo una vez por entorno)
# ======================================================================
# !pip install openpyxl ipywidgets matplotlib

# ======================================================================
# 1. IMPORTS Y CONFIGURACIÓN GENERAL
# ======================================================================
import openpyxl
from openpyxl.utils import column_index_from_string, get_column_letter
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# --- RUTA DEL EXCEL (MAC) ---
EXCEL_PATH = Path("/Users/macuser/Desktop/DIR COMERCIAL/GD/GD_v1.xlsx")

# --- NOMBRES DE HOJAS ---
SHEET_PROYECTOS = "ProyectosTI"
SHEET_DATOS     = "Datos"
SHEET_SUG       = "Sugerencias"

# --- FILAS / CABECERAS ---
START_ROW_PROYECTOS   = 12   # primera fila de datos
HEADER_ROW_PROYECTOS  = 11   # fila de cabecera por defecto

# --- COLUMNAS BASE ---
COLS = {
    "ID": "A",
    "Q_RADICADO": "B",
    "PRIORIZADO": "C",
    "ESTADO_PROYECTO": "D",
    "NOMBRE_PROYECTO": "E",
    "DESCRIPCION_PROYECTO": "F",
    "RESPONSABLE_PROYECTO": "G",
    "AREA_SOLICITANTE": "H",
    "FECHA_INICIO": "I",
    "FECHA_ESTIMADA_CIERRE": "J",
    "LINEA_BASE": "K",
    "LINEA_BASE_Q_GESTION": "L",
    "AVANCE": "M",
    "ESTIMADO_AVANCE": "N",
    "PORC_CUMPLIMIENTO": "O",
    "CONTRIBUCION": "P",              # valor numérico Mesa de Expertos
    "INICIATIVA_ESTRATEGICA": "Q",    # catálogo desde Datos
    # Agregados de dependencias:
    "TOTAL_DEP": "CN",        # L + P
    "TOTAL_L": "CO",          # solo L
    "TOTAL_P": "CP",          # solo P
    "CUBRIMIENTO_DEP": "CQ",  # P / (L+P)  → % de P sobre el total
    # Rating de Mesa de Alistamiento (PO Sync):
    "RATING_PO_SYNC": "CR",   # 1 a 5 estrellas
}

# --- RANGOS DE DEPENDENCIAS (FLAGS y DESCRIPCIONES) ---
FLAG_START_LETTER = "R"
FLAG_END_LETTER   = "BB"
DESC_START_LETTER = "BC"
DESC_END_LETTER   = "CM"

FLAG_START_COL = column_index_from_string(FLAG_START_LETTER)
FLAG_END_COL   = column_index_from_string(FLAG_END_LETTER)
DESC_START_COL = column_index_from_string(DESC_START_LETTER)
DESC_END_COL   = column_index_from_string(DESC_END_LETTER)

# --- VARIABLES GLOBALES ---
DEP_MAPPING = {}   # {celula/tren/coe -> header columna descripción}
catalogs    = {}   # catálogos para combos

# --- ESTILO CORPORATIVO TELEFÓNICA ---
PRIMARY_COLOR = "#00a9e0"
DARK_COLOR    = "#001b3c"
LIGHT_BG      = "#f5f9fc"
CARD_BORDER   = "#d0d7de"

# --- LOGO TELEFÓNICA ---
TEF_LOGO = None
try:
    with open("/Users/macuser/Desktop/DIR COMERCIAL/GD/Telefonica logo.png", "rb") as f:
        TEF_LOGO = widgets.Image(
            value=f.read(),
            format="png",
            layout=widgets.Layout(width="80px", height="auto", margin="0 12px 0 0"),
        )
except Exception as e:
    print("⚠️ No se pudo cargar el logo de Telefónica:", e)


# ======================================================================
# 2. FUNCIONES BÁSICAS PARA EXCEL + UTILIDADES
# ======================================================================

def load_workbook():
    """Abre el libro de Excel (.xlsx)."""
    if not EXCEL_PATH.exists():
        raise FileNotFoundError(f"No se encontró el archivo: {EXCEL_PATH}")
    wb = openpyxl.load_workbook(EXCEL_PATH, keep_vba=False)
    if SHEET_PROYECTOS not in wb.sheetnames:
        raise KeyError(f"No existe la hoja '{SHEET_PROYECTOS}'.")
    return wb

def get_ws_proyectos(wb=None):
    wb = wb or load_workbook()
    return wb[SHEET_PROYECTOS]

def get_ws_datos(wb=None):
    wb = wb or load_workbook()
    if SHEET_DATOS not in wb.sheetnames:
        raise KeyError(f"No existe la hoja '{SHEET_DATOS}'.")
    return wb[SHEET_DATOS]

def get_ws_sugerencias(wb=None):
    """
    Devuelve la hoja 'Sugerencias'. Si no existe, la crea con cabecera.
    """
    wb = wb or load_workbook()
    if SHEET_SUG not in wb.sheetnames:
        ws = wb.create_sheet(SHEET_SUG)
        ws["A1"] = "Usuario"
        ws["B1"] = "Sugerencia"
    else:
        ws = wb[SHEET_SUG]
        if ws.max_row == 1 and ws["A1"].value is None:
            ws["A1"] = "Usuario"
            ws["B1"] = "Sugerencia"
    return ws

def get_header_row_proyectos(ws):
    """
    Detecta la fila de cabeceras buscando el texto 'ID'
    en la columna COLS['ID'].
    """
    id_col_idx = column_index_from_string(COLS["ID"])
    for r in range(1, START_ROW_PROYECTOS):
        v = ws.cell(row=r, column=id_col_idx).value
        if isinstance(v, str) and v.strip().lower() == "id":
            return r
    return HEADER_ROW_PROYECTOS

def get_unique_list_from_column(ws, col_letter, start_row=2):
    """Devuelve una lista ordenada sin duplicados de una columna."""
    values = set()
    col_idx = column_index_from_string(col_letter)
    for row in range(start_row, ws.max_row + 1):
        value = ws.cell(row=row, column=col_idx).value
        if value not in (None, ""):
            values.add(str(value))
    return sorted(values)

def get_next_row_and_id(ws, id_col_letter="A", start_row=START_ROW_PROYECTOS):
    """
    Busca la siguiente fila libre y el siguiente ID numérico.
    """
    id_col_idx = column_index_from_string(id_col_letter)
    max_row_used = 0
    max_id_found = 0

    for row in range(start_row, ws.max_row + 1):
        val = ws.cell(row=row, column=id_col_idx).value
        if val not in (None, ""):
            max_row_used = row
            if isinstance(val, (int, float)):
                max_id_found = max(max_id_found, int(val))

    next_row = start_row if max_row_used == 0 else max_row_used + 1
    next_id  = max_id_found + 1 if max_id_found > 0 else 1
    return next_row, next_id

def find_column_by_header(ws, header_name, header_row=1):
    """Busca una columna por el texto exacto del header (case-insensitive)."""
    if not header_name:
        return None
    target = str(header_name).strip().lower()
    for col_idx in range(1, ws.max_column + 1):
        val = ws.cell(row=header_row, column=col_idx).value
        if val is None:
            continue
        if str(val).strip().lower() == target:
            return col_idx
    return None

def find_column_by_header_in_range(ws, header_name, start_col_idx, end_col_idx, header_row):
    """Busca una columna por header, restringida a un rango de columnas."""
    if not header_name:
        return None
    target = str(header_name).strip().lower()
    for col_idx in range(start_col_idx, end_col_idx + 1):
        val = ws.cell(row=header_row, column=col_idx).value
        if val is None:
            continue
        if str(val).strip().lower() == target:
            return col_idx
    return None

def find_area_tren_coe_col(ws):
    """
    Intenta localizar la columna de Area/Tren/CoE en la hoja ProyectosTI,
    buscando palabras clave en la fila de cabeceras.
    """
    header_row = get_header_row_proyectos(ws)
    candidate_idx = None
    for col_idx in range(1, ws.max_column + 1):
        val = ws.cell(row=header_row, column=col_idx).value
        if not val:
            continue
        s = str(val).strip().lower()
        if "tren" in s and "coe" in s:
            return col_idx
        if s in ("area tren coe", "area/tren/coe"):
            candidate_idx = col_idx
    return candidate_idx

def to_num_cell(v):
    """Convierte cualquier valor de celda a float, tolerando texto, %, comas, etc."""
    if v is None or v == "":
        return 0.0
    if isinstance(v, (int, float)):
        return float(v)
    s = str(v).strip()
    if s == "":
        return 0.0
    s = s.replace("%", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return 0.0


# ======================================================================
# 3. CARGA DE CATÁLOGOS Y MAPPING DE DEPENDENCIAS (HOJA DATOS)
# ======================================================================

def load_dependency_mapping(wb=None):
    """
    Lee de la hoja 'Datos':
      - 'Celula Dependencia'              → header de flag (ej. 'CÉLULA TASADORES')
      - 'Celula Descripcion Dependencia' → header de descripción (ej. 'DESCRIPCION CÉLULA TASADORES')
    y arma: { 'CÉLULA TASADORES': 'DESCRIPCION CÉLULA TASADORES', ... }
    """
    wb = wb or load_workbook()
    ws_d = get_ws_datos(wb)
    col_cel_dep_idx  = find_column_by_header(ws_d, "Celula Dependencia", header_row=1)
    col_desc_dep_idx = find_column_by_header(ws_d, "Celula Descripcion Dependencia", header_row=1)
    mapping = {}
    if not col_cel_dep_idx or not col_desc_dep_idx:
        return mapping

    for row in range(2, ws_d.max_row + 1):
        cel_name   = ws_d.cell(row=row, column=col_cel_dep_idx).value
        desc_header= ws_d.cell(row=row, column=col_desc_dep_idx).value
        if cel_name and desc_header:
            mapping[str(cel_name).strip()] = str(desc_header).strip()
    return mapping

def load_catalogs():
    """
    Carga catálogos y mapeo de dependencias desde 'Datos'.
    """
    global DEP_MAPPING, catalogs
    wb   = load_workbook()
    ws_d = get_ws_datos(wb)

    estados_list       = get_unique_list_from_column(ws_d, "A")
    priorizacion_list  = get_unique_list_from_column(ws_d, "B")
    responsables_list  = get_unique_list_from_column(ws_d, "C")
    areas_list         = get_unique_list_from_column(ws_d, "D")
    area_tren_coe_list = get_unique_list_from_column(ws_d, "E")

    # Iniciativas Estratégicas
    iniciativas_list = []
    col_ini_idx = find_column_by_header(ws_d, "Iniciativa Estrategica", header_row=1)
    if col_ini_idx:
        col_ini_letter = get_column_letter(col_ini_idx)
        iniciativas_list = get_unique_list_from_column(ws_d, col_ini_letter)
    else:
        iniciativas_list = []

    DEP_MAPPING        = load_dependency_mapping(wb)
    celulas_dep_list   = sorted(DEP_MAPPING.keys()) if DEP_MAPPING else []

    catalogs = {
        "estados": estados_list,
        "q_rad": priorizacion_list,
        "responsables": responsables_list,
        "areas": areas_list,
        "area_tren_coe": area_tren_coe_list,
        "celulas_dep": celulas_dep_list,
        "iniciativas": iniciativas_list,
    }

# Cargamos catálogos al arrancar
try:
    load_catalogs()
except Exception as e:
    print("⚠️ No se pudieron cargar catálogos desde 'Datos'. Motivo:", e)
    catalogs = {
        "estados": ["Nuevo", "En curso", "Detenido", "Cancelado", "Finalizado"],
        "q_rad": ["1Q/25", "2Q/25"],
        "responsables": ["Responsable 1"],
        "areas": ["Área 1"],
        "area_tren_coe": ["Tren X"],
        "celulas_dep": ["Célula A"],
        "iniciativas": ["Inic. 1"],
    }
    DEP_MAPPING = {}


# ======================================================================
# 4. NÚCLEO DEPENDENCIAS: AGREGADOS + SEMÁFORO
# ======================================================================

def compute_dep_aggregates(dep_list):
    """
    A partir de dep_list calcula:
      - total_dep = L+P
      - total_L
      - total_P
      - cubrimiento = P / (L+P)  (0 si no hay dependencias)
    """
    flags = [ (d.get("codigo") or "").strip().upper()
              for d in dep_list
              if (d.get("equipo") or "").strip() ]
    flags = [f for f in flags if f in ("L", "P")]
    total_dep = len(flags)
    total_L   = sum(1 for f in flags if f == "L")
    total_P   = sum(1 for f in flags if f == "P")
    if total_dep > 0:
        cubrimiento = total_P / total_dep  # % de P sobre el total
    else:
        cubrimiento = 0.0
    return total_dep, total_L, total_P, cubrimiento

def write_dep_aggregates(ws, row, dep_list):
    """
    Escribe en CN–CQ:
      - CN: total L+P
      - CO: solo L
      - CP: solo P
      - CQ: P / (L+P)   (para formatear como % en Excel)
    """
    total_dep, total_L, total_P, cub = compute_dep_aggregates(dep_list)
    cn = column_index_from_string(COLS["TOTAL_DEP"])
    co = column_index_from_string(COLS["TOTAL_L"])
    cp = column_index_from_string(COLS["TOTAL_P"])
    cq = column_index_from_string(COLS["CUBRIMIENTO_DEP"])

    ws.cell(row=row, column=cn).value = total_dep
    ws.cell(row=row, column=co).value = total_L
    ws.cell(row=row, column=cp).value = total_P
    ws.cell(row=row, column=cq).value = cub

def apply_dependencies_to_row(ws, row, dep_list):
    """
    Aplica un conjunto de dependencias dinámicas en la fila `row`:
      dep_list = [{equipo, codigo(P/L), descripcion}, ...]
    Usa DEP_MAPPING + cabeceras detectadas dinámicamente.
    """
    header_row = get_header_row_proyectos(ws)

    # 1) Limpiar flags y descripciones existentes
    for equipo, desc_header in DEP_MAPPING.items():
        flag_col_idx = find_column_by_header_in_range(
            ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
        )
        if flag_col_idx:
            ws.cell(row=row, column=flag_col_idx).value = None

        if desc_header:
            desc_col_idx = find_column_by_header_in_range(
                ws, desc_header, DESC_START_COL, DESC_END_COL, header_row
            )
            if desc_col_idx:
                ws.cell(row=row, column=desc_col_idx).value = None

    # 2) Escribir nuevas dependencias
    for dep in dep_list:
        equipo = (dep.get("equipo") or "").strip()
        codigo = (dep.get("codigo") or "").strip().upper()
        texto  = (dep.get("descripcion") or "").strip()

        if not equipo or codigo not in ("P", "L"):
            continue

        desc_header = DEP_MAPPING.get(equipo)

        flag_col_idx = find_column_by_header_in_range(
            ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
        )
        if flag_col_idx:
            ws.cell(row=row, column=flag_col_idx).value = codigo

        if desc_header:
            desc_col_idx = find_column_by_header_in_range(
                ws, desc_header, DESC_START_COL, DESC_END_COL, header_row
            )
            if desc_col_idx and texto:
                ws.cell(row=row, column=desc_col_idx).value = texto

    # 3) Escribir agregados en CN–CQ
    write_dep_aggregates(ws, row, dep_list)

def dep_semaforo(total_dep, total_L, total_P):
    """
    Devuelve (color_hex, texto) para el semáforo de dependencias.
    """
    if total_dep == 0:
        return "#bdc3c7", "Sin dependencias registradas"
    if total_P == 0 and total_L > 0:
        return "#2ecc71", "Todas las dependencias negociadas (L)"
    if total_L == 0 and total_P > 0:
        return "#e74c3c", "Todas las dependencias pendientes (P)"
    return "#f1c40f", "Mix de dependencias negociadas (L) y pendientes (P)"


# ======================================================================
# 5. ALTA DE PROYECTOS (NUEVO REGISTRO + DEPENDENCIAS)
# ======================================================================

def write_project_with_dependencies(project, dep_list):
    """
    Inserta un nuevo proyecto y aplica sus dependencias + métricas CN–CQ.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    next_row, next_id = get_next_row_and_id(
        ws, id_col_letter=COLS["ID"], start_row=START_ROW_PROYECTOS
    )

    project = project.copy()
    project["ID"] = next_id

    for field, col_letter in COLS.items():
        if field in project:
            col_idx = column_index_from_string(col_letter)
            ws.cell(row=next_row, column=col_idx).value = project.get(field)

    # si no se envía rating de PO Sync, lo dejamos vacío por ahora
    apply_dependencies_to_row(ws, next_row, dep_list)

    wb.save(EXCEL_PATH)
    return next_row, next_id


# ======================================================================
# 6. CONSULTAS / RESÚMENES
# ======================================================================

def get_all_project_names():
    """Devuelve la lista de proyectos para el combo de consulta."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    name_col_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    names = set()
    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        val = ws.cell(row=row, column=name_col_idx).value
        if val not in (None, ""):
            names.add(str(val).strip())
    return sorted(names)

def summarize_by_equipo(equipo_name):
    """Resumen de cobertura de dependencias para un equipo."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    col_flag_idx = find_column_by_header_in_range(
        ws, equipo_name, FLAG_START_COL, FLAG_END_COL, header_row
    )
    if not col_flag_idx:
        return {"found": False, "msg": f"No se encontró la columna '{equipo_name}' en R:BB."}

    name_col_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    q_col_idx    = column_index_from_string(COLS["Q_RADICADO"])

    total = pendientes = negociadas = 0
    rows = []

    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        flag = ws.cell(row=row, column=col_flag_idx).value
        if flag is None or str(flag).strip() == "":
            continue
        flag_up = str(flag).strip().upper()
        if flag_up not in ("P", "L"):
            continue

        total += 1
        if flag_up == "P":
            pendientes += 1
        else:
            negociadas += 1

        nombre = ws.cell(row=row, column=name_col_idx).value
        qrad   = ws.cell(row=row, column=q_col_idx).value
        rows.append({"fila": row, "Q_RADICADO": qrad, "PROYECTO": nombre, "FLAG": flag_up})

    pct_pend = (pendientes/total*100) if total > 0 else 0.0
    return {
        "found": True,
        "equipo": equipo_name,
        "total": total,
        "pendientes": pendientes,
        "negociadas": negociadas,
        "pct_pendientes": pct_pend,
        "rows": rows,
    }

def summarize_by_proyecto(nombre_proyecto):
    """Resumen completo de un proyecto (incluye dependencias y agregados)."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    name_col_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    q_col_idx    = column_index_from_string(COLS["Q_RADICADO"])

    target_row = None
    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        val = ws.cell(row=row, column=name_col_idx).value
        if val and str(val).strip() == nombre_proyecto:
            target_row = row
            break

    if not target_row:
        return {"found": False, "msg": f"No se encontró el proyecto '{nombre_proyecto}'."}

    detalles = []

    for equipo, desc_header in DEP_MAPPING.items():
        flag_col_idx = find_column_by_header_in_range(
            ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
        )
        if not flag_col_idx:
            continue

        flag = ws.cell(row=target_row, column=flag_col_idx).value
        if flag is None or str(flag).strip() == "":
            continue

        flag_up = str(flag).strip().upper()
        if flag_up not in ("P", "L"):
            continue

        desc = ""
        if desc_header:
            desc_col_idx = find_column_by_header_in_range(
                ws, desc_header, DESC_START_COL, DESC_END_COL, header_row
            )
            if desc_col_idx:
                desc = ws.cell(row=target_row, column=desc_col_idx).value or ""

        detalles.append({"equipo": equipo, "FLAG": flag_up, "descripcion": desc})

    total      = len(detalles)
    pendientes = sum(1 for d in detalles if d["FLAG"] == "P")
    negociadas = sum(1 for d in detalles if d["FLAG"] == "L")
    pct_pend   = (pendientes/total*100) if total > 0 else 0.0

    lb_col   = column_index_from_string(COLS["LINEA_BASE"])
    av_col   = column_index_from_string(COLS["AVANCE"])
    est_col  = column_index_from_string(COLS["ESTIMADO_AVANCE"])

    lb  = to_num_cell(ws.cell(row=target_row, column=lb_col).value)
    av  = to_num_cell(ws.cell(row=target_row, column=av_col).value)
    est = to_num_cell(ws.cell(row=target_row, column=est_col).value)
    qrad= ws.cell(row=target_row, column=q_col_idx).value

    cn = column_index_from_string(COLS["TOTAL_DEP"])
    co = column_index_from_string(COLS["TOTAL_L"])
    cp = column_index_from_string(COLS["TOTAL_P"])
    cq = column_index_from_string(COLS["CUBRIMIENTO_DEP"])

    total_dep_xl = to_num_cell(ws.cell(row=target_row, column=cn).value)
    total_L_xl   = to_num_cell(ws.cell(row=target_row, column=co).value)
    total_P_xl   = to_num_cell(ws.cell(row=target_row, column=cp).value)
    cub_xl       = to_num_cell(ws.cell(row=target_row, column=cq).value)

    return {
        "found": True,
        "fila": target_row,
        "proyecto": nombre_proyecto,
        "Q_RADICADO": qrad,
        "total_dep": total,
        "pendientes": pendientes,
        "negociadas": negociadas,
        "pct_pendientes": pct_pend,
        "detalles": detalles,
        "linea_base": float(lb),
        "avance": float(av),
        "estimado": float(est),
        "total_dep_xl": total_dep_xl,
        "total_L_xl": total_L_xl,
        "total_P_xl": total_P_xl,
        "cub_xl": float(cub_xl),
    }


# ======================================================================
# 7. ACTUALIZAR PROYECTO EXISTENTE (AVANCE + DEPENDENCIAS)
# ======================================================================

def update_project_row_and_dependencies(row, avance, estimado, dep_list):
    """
    Actualiza:
      - AVANCE, ESTIMADO_AVANCE, PORC_CUMPLIMIENTO
      - Dependencias P/L + descripciones + CN–CQ
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    lb_col  = column_index_from_string(COLS["LINEA_BASE"])
    av_col  = column_index_from_string(COLS["AVANCE"])
    est_col = column_index_from_string(COLS["ESTIMADO_AVANCE"])
    pct_col = column_index_from_string(COLS["PORC_CUMPLIMIENTO"])

    linea_base = to_num_cell(ws.cell(row=row, column=lb_col).value)
    old_av     = to_num_cell(ws.cell(row=row, column=av_col).value)
    old_es     = to_num_cell(ws.cell(row=row, column=est_col).value)

    new_av = float(avance)  if avance  is not None else float(old_av)
    new_es = float(estimado)if estimado is not None else float(old_es)

    ws.cell(row=row, column=av_col).value  = new_av
    ws.cell(row=row, column=est_col).value = new_es

    if new_es > 0:
        pct_cumpl = new_av / new_es
    else:
        pct_cumpl = 0.0
    ws.cell(row=row, column=pct_col).value = pct_cumpl

    apply_dependencies_to_row(ws, row, dep_list)

    wb.save(EXCEL_PATH)

    var_vs_lb = new_av - float(linea_base)
    return {
        "linea_base": float(linea_base),
        "avance": new_av,
        "estimado": new_es,
        "pct_cumpl": pct_cumpl * 100,
        "var_vs_lb_pp": var_vs_lb * 100,
    }


# ======================================================================
# 8. CÁLCULO DE MÉTRICAS AGREGADAS (para la página de Métricas)
# ======================================================================

def compute_metrics(scope="all", filter_value=None):
    """
    scope:
      - "all"    → todos los proyectos
      - "area"   → filtrar por Area/Tren/CoE (columna detectada automáticamente)
      - "celula" → filtrar por una célula (según flag P/L en columna R:BB)
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    name_col_idx   = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    prior_col_idx  = column_index_from_string(COLS["PRIORIZADO"])
    av_col_idx     = column_index_from_string(COLS["AVANCE"])
    cn_idx         = column_index_from_string(COLS["TOTAL_DEP"])
    co_idx         = column_index_from_string(COLS["TOTAL_L"])
    cp_idx         = column_index_from_string(COLS["TOTAL_P"])

    area_col_idx = find_area_tren_coe_col(ws)

    cel_flag_col_idx = None
    if scope == "celula" and filter_value:
        cel_flag_col_idx = find_column_by_header_in_range(
            ws, filter_value, FLAG_START_COL, FLAG_END_COL, header_row
        )

    total_projects       = 0
    total_dep            = 0.0
    total_L              = 0.0
    total_P              = 0.0
    sum_avance           = 0.0
    pri_count            = 0
    pri_avance_sum       = 0.0
    no_pri_count         = 0
    no_pri_avance_sum    = 0.0

    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        name = ws.cell(row=row, column=name_col_idx).value
        if not name:
            continue

        if scope == "area" and filter_value:
            if not area_col_idx:
                continue
            area_val = ws.cell(row=row, column=area_col_idx).value
            if str(area_val).strip() != str(filter_value).strip():
                continue

        if scope == "celula" and filter_value:
            if not cel_flag_col_idx:
                continue
            flag_val = ws.cell(row=row, column=cel_flag_col_idx).value
            if not flag_val or str(flag_val).strip().upper() not in ("P", "L"):
                continue

        total_projects += 1

        dep_val = to_num_cell(ws.cell(row=row, column=cn_idx).value)
        L_val   = to_num_cell(ws.cell(row=row, column=co_idx).value)
        P_val   = to_num_cell(ws.cell(row=row, column=cp_idx).value)

        total_dep += dep_val
        total_L   += L_val
        total_P   += P_val

        av_val = to_num_cell(ws.cell(row=row, column=av_col_idx).value)
        sum_avance += av_val

        pri_val = ws.cell(row=row, column=prior_col_idx).value
        pri_str = str(pri_val).strip().upper() if pri_val not in (None, "") else ""

        if pri_str == "SI":
            pri_count      += 1
            pri_avance_sum += av_val
        else:
            no_pri_count      += 1
            no_pri_avance_sum += av_val

    if total_projects > 0:
        avg_avance = sum_avance / total_projects
    else:
        avg_avance = 0.0

    if pri_count > 0:
        avg_pri = pri_avance_sum / pri_count
    else:
        avg_pri = 0.0

    if no_pri_count > 0:
        avg_no_pri = no_pri_avance_sum / no_pri_count
    else:
        avg_no_pri = 0.0

    if total_dep > 0:
        cobertura_pct = (total_P / total_dep) * 100.0
    else:
        cobertura_pct = 0.0

    return {
        "total_projects": int(total_projects),
        "total_dep": float(total_dep),
        "total_L": float(total_L),
        "total_P": float(total_P),
        "cobertura_pct": float(cobertura_pct),
        "avg_avance": float(avg_avance),
        "avg_pri": float(avg_pri),
        "avg_no_pri": float(avg_no_pri),
        "num_pri": int(pri_count),
    }


# ======================================================================
# 9. UI – COMPONENTES COMUNES (HEADER TELEFÓNICA)
# ======================================================================

def build_header(title, subtitle=""):
    """Header corporativo Telefónica con logo + título, azul oscuro sobre fondo claro."""
    title_html = f"""
    <div style="
        display:flex;
        flex-direction:column;
        justify-content:center;
    ">
      <div style="
          font-family:Segoe UI, Arial, sans-serif;
          font-size:18px;
          font-weight:600;
          color:{DARK_COLOR};
          margin-bottom:2px;
      ">{title}</div>
      <div style="
          font-family:Segoe UI, Arial, sans-serif;
          font-size:12px;
          color:{DARK_COLOR};
          opacity:0.75;
      ">{subtitle}</div>
    </div>
    """

    box = widgets.HBox(
        [
            TEF_LOGO if TEF_LOGO else widgets.HTML(""),
            widgets.HTML(value=title_html),
        ],
        layout=widgets.Layout(
            background_color="white",
            padding="10px 16px",
            border=f"1px solid {PRIMARY_COLOR}",
            border_radius="10px 10px 0 0",
            align_items="center",
        ),
    )
    return box


# ======================================================================
# 10. UI – ALTA DE PROYECTOS (PÁGINA 1)
# ======================================================================

def build_create_form():
    """Formulario de alta de proyectos + dependencias dinámicas."""

    header_box = build_header(
        "Alta de proyectos",
        "Registro de iniciativas y dependencias entre células / trenes / CoE"
    )

    # ---------- Campos base ----------
    w_nombre = widgets.Text(
        description="Proyecto:",
        placeholder="Nombre del proyecto",
        layout=widgets.Layout(width="600px"),
        style={"description_width": "100px"},
    )
    w_estado = widgets.Dropdown(
        options=catalogs.get("estados", []),
        description="Estado:",
        layout=widgets.Layout(width="290px"),
        style={"description_width": "80px"},
    )
    w_prior_q = widgets.Dropdown(
        options=catalogs.get("q_rad", []),
        description="Q Radicado:",
        layout=widgets.Layout(width="290px"),
        style={"description_width": "100px"},
    )
    w_priorizado = widgets.Dropdown(
        options=[("No", "NO"), ("Sí", "SI")],
        description="Priorizado:",
        layout=widgets.Layout(width="200px"),
        style={"description_width": "90px"},
    )
    w_resp = widgets.Combobox(
        options=catalogs.get("responsables", []),
        description="Responsable:",
        placeholder="Selecciona o escribe…",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "100px"},
    )
    w_area = widgets.Combobox(
        options=catalogs.get("areas", []),
        description="Área solicitante:",
        placeholder="Selecciona o escribe…",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "110px"},
    )
    w_area_tren = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Area/Tren/CoE:",
        placeholder="Se rellena al elegir célula (o puedes sobrescribir)",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "110px"},
    )
    w_iniciativa = widgets.Combobox(
        options=catalogs.get("iniciativas", []),
        description="Inic. Estrat.:",
        placeholder="Selecciona iniciativa…",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "110px"},
    )
    w_desc = widgets.Textarea(
        description="Descripción:",
        placeholder="Breve descripción del proyecto…",
        layout=widgets.Layout(width="600px", height="110px"),
        style={"description_width": "100px"},
    )
    w_f_ini = widgets.DatePicker(
        description="Fecha inicio:",
        layout=widgets.Layout(width="260px"),
        style={"description_width": "100px"},
    )
    w_f_fin = widgets.DatePicker(
        description="Fecha cierre:",
        layout=widgets.Layout(width="260px"),
        style={"description_width": "100px"},
    )
    w_linea = widgets.FloatSlider(
        value=0.25, min=0.0, max=1.0, step=0.05,
        description="Línea Base:",
        readout_format=".2f",
        layout=widgets.Layout(width="450px"),
        style={"description_width": "100px"},
    )

    # ---------- Dependencias dinámicas (alta) ----------
    dep_rows      = []
    dep_container = widgets.VBox(
        [],
        layout=widgets.Layout(width="100%")
    )

    def make_dep_row(equipo="", flag="P", descripcion=""):
        cb_equipo = widgets.Combobox(
            options=catalogs.get("celulas_dep", []),
            description="Célula/Tren:",
            placeholder="Célula / Tren / CoE",
            ensure_option=False,
            layout=widgets.Layout(width="360px"),
            style={"description_width": "90px"},
        )
        if equipo:
            cb_equipo.value = equipo

        def on_equipo_change(change):
            if change["name"] == "value":
                new_val = change["new"]
                if new_val:
                    w_area_tren.value = new_val

        cb_equipo.observe(on_equipo_change, names="value")

        dd_flag = widgets.Dropdown(
            options=[("Pendiente (P)", "P"), ("Negociada (L)", "L")],
            description="Estado:",
            layout=widgets.Layout(width="200px"),
            style={"description_width": "70px"},
        )
        dd_flag.value = flag or "P"

        ta_desc = widgets.Textarea(
            placeholder="Descripción de la dependencia…",
            layout=widgets.Layout(width="600px", height="60px"),
        )
        ta_desc.value = descripcion or ""

        btn_del = widgets.Button(
            icon="trash",
            tooltip="Eliminar dependencia",
            layout=widgets.Layout(width="40px"),
            button_style="danger",
        )

        top = widgets.HBox(
            [cb_equipo, dd_flag, btn_del],
            layout=widgets.Layout(width="100%", justify_content="flex-start")
        )
        box = widgets.VBox(
            [top, ta_desc],
            layout=widgets.Layout(
                border=f"1px solid {CARD_BORDER}",
                padding="6px",
                margin="4px 0",
                width="100%",
                background_color="white",
            )
        )

        row_dict = {
            "equipo": cb_equipo,
            "flag": dd_flag,
            "desc": ta_desc,
            "box": box,
            "btn": btn_del,
        }

        def on_del_clicked(b):
            if row_dict in dep_rows:
                dep_rows.remove(row_dict)
                dep_container.children = [r["box"] for r in dep_rows]

        btn_del.on_click(on_del_clicked)
        return row_dict

    def add_dep_row(equipo="", flag="P", descripcion=""):
        row = make_dep_row(equipo, flag, descripcion)
        dep_rows.append(row)
        dep_container.children = [r["box"] for r in dep_rows]

    btn_add_dep = widgets.Button(
        description="+ Agregar dependencia",
        button_style="info",
        icon="plus",
        layout=widgets.Layout(width="220px"),
    )
    btn_add_dep.on_click(lambda b: add_dep_row())

    add_dep_row()

    out = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="90px",
            background_color="white",
        )
    )
    btn_save = widgets.Button(
        description="Guardar en Excel",
        button_style="success",
        icon="save",
        layout=widgets.Layout(width="250px", height="40px", margin="10px 0 0 0"),
    )

    def on_click_guardar(b):
        out.clear_output()
        with out:
            if not w_nombre.value.strip():
                print("⚠️ El nombre del proyecto es obligatorio.")
                return
            if not w_f_ini.value:
                print("⚠️ Debes seleccionar la fecha de inicio.")
                return
            if not w_f_fin.value:
                print("⚠️ Debes seleccionar la fecha de cierre.")
                return
            if w_f_fin.value < w_f_ini.value:
                print("⚠️ La fecha de cierre no puede ser anterior a la de inicio.")
                return

            project = {
                "Q_RADICADO": w_prior_q.value,
                "PRIORIZADO": w_priorizado.value,
                "ESTADO_PROYECTO": w_estado.value,
                "NOMBRE_PROYECTO": w_nombre.value.strip(),
                "DESCRIPCION_PROYECTO": w_desc.value.strip(),
                "RESPONSABLE_PROYECTO": (w_resp.value or "").strip(),
                "AREA_SOLICITANTE": (w_area.value or "").strip(),
                "FECHA_INICIO": w_f_ini.value,
                "FECHA_ESTIMADA_CIERRE": w_f_fin.value,
                "LINEA_BASE": float(w_linea.value),
                "AVANCE": 0.0,
                "ESTIMADO_AVANCE": 1.0,
                "PORC_CUMPLIMIENTO": 0.0,
                "CONTRIBUCION": 0.0,
                "INICIATIVA_ESTRATEGICA": (w_iniciativa.value or "").strip(),
            }

            dep_list = []
            for r in dep_rows:
                eq = r["equipo"].value
                fl = r["flag"].value
                ds = r["desc"].value
                if eq:
                    dep_list.append({
                        "equipo": eq,
                        "codigo": fl,
                        "descripcion": ds,
                    })

            try:
                row, pid = write_project_with_dependencies(project, dep_list)
                print(f"✅ Proyecto guardado en fila {row} con ID {pid}.")
                print(f"   Dependencias registradas: {len(dep_list)}")
            except PermissionError as e:
                print("❌ No se pudo guardar (archivo bloqueado).")
                print("   Cierra el Excel si está abierto.")
                print("   Detalle:", e)
            except Exception as e:
                print("❌ Error inesperado al guardar:", repr(e))

    btn_save.on_click(on_click_guardar)

    box_sup = widgets.VBox(
        [
            widgets.HBox([w_nombre]),
            widgets.HBox([w_estado, w_prior_q, w_priorizado]),
            widgets.HBox([w_resp]),
            widgets.HBox([w_area]),
            widgets.HBox([w_area_tren]),
            widgets.HBox([w_iniciativa]),
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="8px 0",
            background_color="white",
        ),
    )
    box_fecha = widgets.VBox(
        [
            widgets.HBox([w_f_ini, w_f_fin]),
            widgets.HBox([w_linea]),
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="0 0 8px 0",
            background_color="white",
        ),
    )
    box_desc = widgets.VBox(
        [w_desc],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="0 0 8px 0",
            background_color="white",
        ),
    )
    box_dep = widgets.VBox(
        [
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Dependencias (P/L + descripción, por equipo)</b>"
            ),
            btn_add_dep,
            dep_container,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="0 0 8px 0",
            background_color="white",
        ),
    )

    body = widgets.VBox(
        [
            box_sup,
            box_fecha,
            box_desc,
            box_dep,
            widgets.HBox([btn_save]),
            widgets.HTML(
                f"<hr style='margin:10px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            out,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="10px 0"),
    )
    return panel


# ======================================================================
# 11. UI – CONSULTA + EDICIÓN (PÁGINA 2)
# ======================================================================

def build_consult_panel():
    """Panel de consulta, cobertura y edición de proyectos."""

    header_box = build_header(
        "Consulta y edición",
        "Cobertura de dependencias y actualización de avance"
    )

    w_tipo = widgets.ToggleButtons(
        options=[
            ("Por Equipo (Tren/Célula/CoE)", "equipo"),
            ("Por Proyecto (edición)", "proyecto"),
        ],
        description="Ver por:",
        style={"description_width": "80px"},
        layout=widgets.Layout(width="840px"),
    )

    w_equipo = widgets.Combobox(
        options=sorted(catalogs.get("celulas_dep", [])),
        description="Equipo:",
        placeholder="Selecciona Tren / Célula / CoE",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "70px"},
    )
    btn_clear_equipo = widgets.Button(
        icon="times",
        tooltip="Limpiar equipo",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    w_proj = widgets.Combobox(
        options=get_all_project_names(),
        description="Proyecto:",
        placeholder="Escribe / selecciona",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "80px"},
    )
    btn_clear_proj = widgets.Button(
        icon="times",
        tooltip="Limpiar proyecto",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    def on_clear_equipo(b):
        w_equipo.value = ""

    def on_clear_proj(b):
        w_proj.value = ""

    btn_clear_equipo.on_click(on_clear_equipo)
    btn_clear_proj.on_click(on_clear_proj)

    box_equipo = widgets.HBox([w_equipo, btn_clear_equipo])
    box_proy   = widgets.HBox([w_proj, btn_clear_proj])

    def toggle_inputs(change):
        if w_tipo.value == "equipo":
            box_equipo.layout.display = "flex"
            box_proy.layout.display   = "none"
        else:
            box_equipo.layout.display = "none"
            box_proy.layout.display   = "flex"

    w_tipo.observe(toggle_inputs, names="value")
    toggle_inputs(None)

    btn_consultar = widgets.Button(
        description="Consultar",
        button_style="info",
        icon="search",
        layout=widgets.Layout(width="200px", height="35px"),
    )
    btn_limpiar = widgets.Button(
        description="Limpiar filtros",
        button_style="",
        icon="eraser",
        layout=widgets.Layout(width="160px", height="35px", margin="0 0 0 10px"),
    )
    btn_refrescar = widgets.Button(
        description="Refrescar catálogos",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="180px", height="35px", margin="0 0 0 10px"),
    )

    out_resumen = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="230px",
            background_color="white",
        )
    )

    edit_state = {"row": None}

    w_edit_avance = widgets.FloatSlider(
        value=0.0, min=0.0, max=1.0, step=0.05,
        description="Avance:",
        readout_format=".2f",
        layout=widgets.Layout(width="360px"),
        style={"description_width": "80px"},
    )
    w_edit_estimado = widgets.FloatSlider(
        value=1.0, min=0.0, max=1.0, step=0.05,
        description="Estimado:",
        readout_format=".2f",
        layout=widgets.Layout(width="360px"),
        style={"description_width": "80px"},
    )

    dep_rows_edit      = []
    dep_container_edit = widgets.VBox(
        [],
        layout=widgets.Layout(width="100%")
    )

    def make_dep_row_edit(equipo="", flag="P", descripcion=""):
        cb_equipo = widgets.Combobox(
            options=catalogs.get("celulas_dep", []),
            description="Célula/Tren:",
            placeholder="Célula / Tren / CoE",
            ensure_option=False,
            layout=widgets.Layout(width="360px"),
            style={"description_width": "90px"},
        )
        if equipo:
            cb_equipo.value = equipo

        dd_flag = widgets.Dropdown(
            options=[("Pendiente (P)", "P"), ("Negociada (L)", "L")],
            description="Estado:",
            layout=widgets.Layout(width="200px"),
            style={"description_width": "70px"},
        )
        dd_flag.value = flag or "P"

        ta_desc = widgets.Textarea(
            placeholder="Descripción de la dependencia…",
            layout=widgets.Layout(width="600px", height="60px"),
        )
        ta_desc.value = descripcion or ""

        btn_del = widgets.Button(
            icon="trash",
            tooltip="Eliminar dependencia",
            layout=widgets.Layout(width="40px"),
            button_style="danger",
        )

        top = widgets.HBox(
            [cb_equipo, dd_flag, btn_del],
            layout=widgets.Layout(width="100%", justify_content="flex-start")
        )
        box = widgets.VBox(
            [top, ta_desc],
            layout=widgets.Layout(
                border=f"1px solid {CARD_BORDER}",
                padding="6px",
                margin="4px 0",
                width="100%",
                background_color="white",
            )
        )

        row_dict = {
            "equipo": cb_equipo,
            "flag": dd_flag,
            "desc": ta_desc,
            "box": box,
            "btn": btn_del,
        }

        def on_del_clicked(b):
            if row_dict in dep_rows_edit:
                dep_rows_edit.remove(row_dict)
                dep_container_edit.children = [r["box"] for r in dep_rows_edit]

        btn_del.on_click(on_del_clicked)
        return row_dict

    def add_dep_row_edit(equipo="", flag="P", descripcion=""):
        row = make_dep_row_edit(equipo, flag, descripcion)
        dep_rows_edit.append(row)
        dep_container_edit.children = [r["box"] for r in dep_rows_edit]

    btn_add_dep_edit = widgets.Button(
        description="+ Agregar dependencia",
        button_style="info",
        icon="plus",
        layout=widgets.Layout(width="220px"),
    )
    btn_add_dep_edit.on_click(lambda b: add_dep_row_edit())

    out_edit_calc = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="90px",
            background_color="white",
        )
    )
    btn_edit = widgets.Button(
        description="Guardar cambios de proyecto",
        button_style="warning",
        icon="edit",
        layout=widgets.Layout(width="260px", height="35px"),
    )

    def refresh_edit_metrics(linea_base, avance, estimado):
        out_edit_calc.clear_output()
        with out_edit_calc:
            if estimado > 0:
                pct_cumpl = avance/estimado*100
            else:
                pct_cumpl = 0.0
            var_pp = (avance - linea_base)*100
            print(f"Línea base: {linea_base:.2f} ({linea_base*100:.1f}%)")
            print(f"Avance actual: {avance:.2f} ({avance*100:.1f}%)")
            print(f"Variación vs línea base: {var_pp:+.1f} pp")
            print(f"% avance vs estimado: {pct_cumpl:.1f}%")

    def on_consultar_clicked(b):
        out_resumen.clear_output()
        with out_resumen:
            if w_tipo.value == "equipo":
                equipo = w_equipo.value
                edit_state["row"] = None
                dep_rows_edit.clear()
                dep_container_edit.children = []
                if not equipo:
                    print("⚠️ Selecciona un equipo.")
                    return
                res = summarize_by_equipo(equipo)
                if not res["found"]:
                    print("⚠️", res["msg"])
                    return
                print(f"📊 Resumen por equipo: {res['equipo']}")
                print(f"- Total dependencias: {res['total']}")
                print(f"- Pendientes (P):     {res['pendientes']}")
                print(f"- Negociadas (L):     {res['negociadas']}")
                print(f"- % sin negociar:     {res['pct_pendientes']:.1f}%")
                print("\nProyectos:")
                for r in res["rows"]:
                    print(f"  · Fila {r['fila']}: [{r['Q_RADICADO']}] {r['PROYECTO']} → {r['FLAG']}")
            else:
                proyecto = w_proj.value
                if not proyecto:
                    print("⚠️ Selecciona un proyecto.")
                    return
                res = summarize_by_proyecto(proyecto)
                if not res["found"]:
                    print("⚠️", res["msg"])
                    return
                edit_state["row"] = res["fila"]
                print(f"📊 Proyecto: [{res['Q_RADICADO']}] {res['proyecto']}")
                print(f"- Fila hoja:          {res['fila']}")
                print(f"- Total dependencias: {res['total_dep_xl']} (L+P)")
                print(f"- Pendientes (P):     {res['total_P_xl']}")
                print(f"- Negociadas (L):     {res['total_L_xl']}")
                print(f"- Cubrimiento (CQ):   {res['cub_xl']:.2f}  (formato % en Excel)")
                print("\nDependencias actuales:")
                if not res["detalles"]:
                    print("  (Sin dependencias registradas)")
                else:
                    for d in res["detalles"]:
                        print(f"  · {d['equipo']} → {d['FLAG']}")

                w_edit_avance.value   = res["avance"]
                w_edit_estimado.value = res["estimado"]
                refresh_edit_metrics(res["linea_base"], res["avance"], res["estimado"])

                dep_rows_edit.clear()
                dep_container_edit.children = []
                for d in res["detalles"]:
                    add_dep_row_edit(d["equipo"], d["FLAG"], d.get("descripcion", ""))
                add_dep_row_edit()

    btn_consultar.on_click(on_consultar_clicked)

    def on_limpiar_clicked(b):
        w_equipo.value = ""
        w_proj.value   = ""
        out_resumen.clear_output()
        out_edit_calc.clear_output()
        edit_state["row"] = None
        dep_rows_edit.clear()
        dep_container_edit.children = []

    btn_limpiar.on_click(on_limpiar_clicked)

    def on_refrescar_clicked(b):
        try:
            load_catalogs()
        except Exception as e:
            out_resumen.clear_output()
            with out_resumen:
                print("⚠️ Error recargando catálogos:", e)
            return

        w_equipo.options = sorted(catalogs.get("celulas_dep", []))
        w_proj.options   = get_all_project_names()
        w_equipo.value   = ""
        w_proj.value     = ""

        out_resumen.clear_output()
        out_edit_calc.clear_output()
        edit_state["row"] = None
        dep_rows_edit.clear()
        dep_container_edit.children = []

        with out_resumen:
            print("🔄 Catálogos y listas recargados desde Excel.")

    btn_refrescar.on_click(on_refrescar_clicked)

    def on_click_edit(b):
        out_edit_calc.clear_output()
        with out_edit_calc:
            row = edit_state.get("row")
            if not row:
                print("⚠️ Primero consulta un proyecto en modo 'Por Proyecto'.")
                return

            dep_list = []
            for r in dep_rows_edit:
                eq = r["equipo"].value
                fl = r["flag"].value
                ds = r["desc"].value
                if eq:
                    dep_list.append({
                        "equipo": eq,
                        "codigo": fl,
                        "descripcion": ds,
                    })

            metrics = update_project_row_and_dependencies(
                row=row,
                avance=w_edit_avance.value,
                estimado=w_edit_estimado.value,
                dep_list=dep_list,
            )
            print("✅ Proyecto actualizado.")
            print(f"Línea base: {metrics['linea_base']:.2f} ({metrics['linea_base']*100:.1f}%)")
            print(f"Avance: {metrics['avance']:.2f} ({metrics['avance']*100:.1f}%)")
            print(f"Variación vs línea base: {metrics['var_vs_lb_pp']:+.1f} pp")
            print(f"% avance vs estimado: {metrics['pct_cumpl']:.1f}%")
            print(f"Dependencias registradas: {len(dep_list)}")

    btn_edit.on_click(on_click_edit)

    edit_box = widgets.VBox(
        [
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Edición de proyecto seleccionado</b>"
            ),
            widgets.HBox([w_edit_avance, w_edit_estimado]),
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Dependencias (P/L + descripción)</b>"
            ),
            btn_add_dep_edit,
            dep_container_edit,
            widgets.HBox([btn_edit]),
            out_edit_calc,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="10px 0 0 0",
            background_color="white",
        ),
    )

    body = widgets.VBox(
        [
            w_tipo,
            box_equipo,
            box_proy,
            widgets.HBox([btn_consultar, btn_limpiar, btn_refrescar]),
            widgets.HTML(
                f"<hr style='margin:8px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            out_resumen,
            edit_box,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_consulta = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_consulta


# ======================================================================
# 12. UI – MÉTRICAS (PÁGINA 3)
# ======================================================================

def build_metrics_panel():
    """Panel de métricas agregadas con gráficos y filtros."""

    header_box = build_header(
        "Métricas",
        "Visión agregada de proyectos y dependencias por Area/Tren/CoE y Célula"
    )

    w_scope = widgets.ToggleButtons(
        options=[
            ("Todos", "all"),
            ("Por Area/Tren/CoE", "area"),
            ("Por Célula", "celula"),
        ],
        description="Ámbito:",
        style={"description_width": "80px"},
        layout=widgets.Layout(width="840px"),
    )

    w_area = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Area/Tren/CoE:",
        placeholder="Selecciona un Area/Tren/CoE",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "110px"},
    )
    btn_clear_area = widgets.Button(
        icon="times",
        tooltip="Limpiar Area/Tren/CoE",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    w_celula = widgets.Combobox(
        options=catalogs.get("celulas_dep", []),
        description="Célula:",
        placeholder="Selecciona una célula / tren / CoE",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "70px"},
    )
    btn_clear_celula = widgets.Button(
        icon="times",
        tooltip="Limpiar célula",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    def on_clear_area(b):
        w_area.value = ""

    def on_clear_celula(b):
        w_celula.value = ""

    btn_clear_area.on_click(on_clear_area)
    btn_clear_celula.on_click(on_clear_celula)

    box_area   = widgets.HBox([w_area, btn_clear_area])
    box_celula = widgets.HBox([w_celula, btn_clear_celula])

    def toggle_filter_boxes(change):
        if w_scope.value == "all":
            box_area.layout.display   = "none"
            box_celula.layout.display = "none"
        elif w_scope.value == "area":
            box_area.layout.display   = "flex"
            box_celula.layout.display = "none"
        else:
            box_area.layout.display   = "none"
            box_celula.layout.display = "flex"

    w_scope.observe(toggle_filter_boxes, names="value")
    toggle_filter_boxes(None)

    btn_metrics = widgets.Button(
        description="Ver métricas",
        button_style="info",
        icon="bar-chart",
        layout=widgets.Layout(width="200px", height="35px"),
    )
    btn_refresh = widgets.Button(
        description="Refrescar catálogos",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="180px", height="35px", margin="0 0 0 10px"),
    )

    out_text = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="160px",
            background_color="white",
        )
    )

    out_charts = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="420px",
            background_color="white",
            overflow="auto",
        )
    )

    def on_refresh_metrics_catalogs(b):
        try:
            load_catalogs()
        except Exception as e:
            out_text.clear_output()
            with out_text:
                print("⚠️ Error recargando catálogos:", e)
            return

        w_area.options   = catalogs.get("area_tren_coe", [])
        w_celula.options = catalogs.get("celulas_dep", [])
        w_area.value     = ""
        w_celula.value   = ""

        out_text.clear_output()
        out_charts.clear_output()
        with out_text:
            print("🔄 Catálogos de Area/Tren/CoE y Células recargados desde Excel.")

    btn_refresh.on_click(on_refresh_metrics_catalogs)

    def on_click_metrics(b):
        out_text.clear_output()
        out_charts.clear_output()

        scope = w_scope.value
        fval  = None
        label_scope = "Todos los proyectos"

        if scope == "area":
            fval = w_area.value
            if not fval:
                with out_text:
                    print("⚠️ Selecciona un Area/Tren/CoE.")
                return
            label_scope = f"Area/Tren/CoE: {fval}"
        elif scope == "celula":
            fval = w_celula.value
            if not fval:
                with out_text:
                    print("⚠️ Selecciona una Célula / Tren / CoE.")
                return
            label_scope = f"Célula/Tren/CoE (dep): {fval}"

        metrics = compute_metrics(scope=scope, filter_value=fval)

        with out_text:
            print(f"🎯 Ámbito: {label_scope}")
            print(f"- Total de proyectos:            {metrics['total_projects']}")
            print(f"- Nº total de dependencias:      {metrics['total_dep']} (L+P)")
            print(f"- Nº dependencias pendientes P:  {metrics['total_P']}")
            print(f"- Nº dependencias negociadas L:  {metrics['total_L']}")
            print(f"- % cubrimiento (P / total):     {metrics['cobertura_pct']:.1f}%")
            print(f"- Promedio avance proyectos:     {metrics['avg_avance']*100:.1f}%")
            print(f"- Promedio avance priorizados:   {metrics['avg_pri']*100:.1f}%")
            print(f"- Promedio avance no prioriz.:   {metrics['avg_no_pri']*100:.1f}%")
            print(f"- Nº proyectos priorizados:      {metrics['num_pri']}")

        with out_charts:
            if metrics["total_projects"] == 0:
                print("Sin proyectos en el ámbito seleccionado.")
                return

            fig1, ax1 = plt.subplots(figsize=(5, 3))
            labels1 = ["Proyectos", "Dep L", "Dep P"]
            values1 = [
                metrics["total_projects"],
                metrics["total_L"],
                metrics["total_P"],
            ]
            ax1.bar(labels1, values1)
            ax1.set_title("Conteos básicos", fontsize=11)
            ax1.set_ylabel("Cantidad")
            ax1.grid(axis="y", alpha=0.3)
            plt.tight_layout()
            display(fig1)
            plt.close(fig1)

            fig2, ax2 = plt.subplots(figsize=(6, 3))
            labels2 = [
                "Avance global",
                "Avance priorizados",
                "Avance no priorizados",
                "% cubrimiento P",
            ]
            values2 = [
                metrics["avg_avance"] * 100,
                metrics["avg_pri"] * 100,
                metrics["avg_no_pri"] * 100,
                metrics["cobertura_pct"],
            ]
            ax2.bar(labels2, values2)
            ax2.set_title("Promedios y cubrimiento (%)", fontsize=11)
            ax2.set_ylabel("%")
            ax2.set_ylim(0, 110)
            ax2.grid(axis="y", alpha=0.3)
            plt.xticks(rotation=15, ha="right")
            plt.tight_layout()
            display(fig2)
            plt.close(fig2)

    btn_metrics.on_click(on_click_metrics)

    body = widgets.VBox(
        [
            w_scope,
            box_area,
            box_celula,
            widgets.HBox([btn_metrics, btn_refresh]),
            widgets.HTML(
                f"<hr style='margin:8px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Resumen numérico</b>"
            ),
            out_text,
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};margin-top:8px;'>Gráficos</b>"
            ),
            out_charts,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_metrics = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_metrics


# ======================================================================
# 13. MESA DE EXPERTOS – LÓGICA (PENDIENTES, SEMÁFORO, FILTRO TREN)
# ======================================================================

def get_expert_project_list(tren_filter=None):
    """
    Devuelve una lista de proyectos en estado Nuevo / En curso
    con datos para la Mesa de Expertos y Mesa de Alistamiento.
    Incluye lista de dependencias pendientes para semáforo y rating PO Sync.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    name_idx   = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    desc_idx   = column_index_from_string(COLS["DESCRIPCION_PROYECTO"])
    est_idx    = column_index_from_string(COLS["ESTADO_PROYECTO"])
    pri_idx    = column_index_from_string(COLS["PRIORIZADO"])
    cn_idx     = column_index_from_string(COLS["TOTAL_DEP"])
    co_idx     = column_index_from_string(COLS["TOTAL_L"])
    cp_idx     = column_index_from_string(COLS["TOTAL_P"])
    cq_idx     = column_index_from_string(COLS["CUBRIMIENTO_DEP"])
    contrib_idx= column_index_from_string(COLS["CONTRIBUCION"])
    inic_idx   = column_index_from_string(COLS["INICIATIVA_ESTRATEGICA"])
    rating_idx = column_index_from_string(COLS["RATING_PO_SYNC"])

    area_col_idx = find_area_tren_coe_col(ws)

    projects = []

    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        estado = ws.cell(row=row, column=est_idx).value
        if not estado:
            continue
        s = str(estado).strip().lower()
        if not (("nuevo" in s) or ("curso" in s)):
            continue

        if tren_filter and area_col_idx:
            area_val = ws.cell(row=row, column=area_col_idx).value
            if str(area_val).strip() != str(tren_filter).strip():
                continue

        nombre = ws.cell(row=row, column=name_idx).value
        if not nombre:
            continue

        desc = ws.cell(row=row, column=desc_idx).value or ""
        pri  = ws.cell(row=row, column=pri_idx).value or "NO"

        total_dep = to_num_cell(ws.cell(row=row, column=cn_idx).value)
        total_L   = to_num_cell(ws.cell(row=row, column=co_idx).value)
        total_P   = to_num_cell(ws.cell(row=row, column=cp_idx).value)
        cub_raw   = to_num_cell(ws.cell(row=row, column=cq_idx).value)
        cub       = cub_raw*100 if cub_raw <= 1 else cub_raw

        contrib   = to_num_cell(ws.cell(row=row, column=contrib_idx).value)
        inic      = ws.cell(row=row, column=inic_idx).value or ""

        if total_dep > 0:
            cobertura_pct = (total_P / total_dep) * 100.0
        else:
            cobertura_pct = 0.0

        pending_equips = []
        for equipo, desc_header in DEP_MAPPING.items():
            flag_col_idx = find_column_by_header_in_range(
                ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
            )
            if not flag_col_idx:
                continue
            flag = ws.cell(row=row, column=flag_col_idx).value
            if not flag:
                continue
            flag_up = str(flag).strip().upper()
            if flag_up == "P":
                pending_equips.append(equipo)

        rating_raw = ws.cell(row=row, column=rating_idx).value
        rating_po  = int(to_num_cell(rating_raw)) if rating_raw not in (None, "") else 0
        if rating_po < 0:
            rating_po = 0
        if rating_po > 5:
            rating_po = 5

        projects.append({
            "row": row,
            "nombre": str(nombre),
            "descripcion": str(desc),
            "estado": str(estado),
            "priorizado": str(pri),
            "total_dep": total_dep,
            "total_L": total_L,
            "total_P": total_P,
            "cobertura_pct": cobertura_pct,
            "contribucion": contrib,
            "iniciativa": str(inic),
            "pending_equips": pending_equips,
            "rating_po": rating_po,
        })

    return projects

def get_cancelled_detained_summary():
    """
    Texto simple con proyectos Cancelados / Detenidos para mostrar en Accordion.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    name_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    est_idx  = column_index_from_string(COLS["ESTADO_PROYECTO"])

    lines = []
    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        estado = ws.cell(row=row, column=est_idx).value
        if not estado:
            continue
        s = str(estado).strip().lower()
        if ("cancel" in s) or ("detenid" in s):
            nombre = ws.cell(row=row, column=name_idx).value or "(Sin nombre)"
            lines.append(f"- Fila {row}: [{estado}] {nombre}")

    if not lines:
        return "No hay proyectos cancelados o detenidos registrados."
    return "\n".join(lines)

def update_expert_fields(row, priorizado, contribucion, iniciativa):
    """
    Actualiza en la fila:
      - PRIORIZADO (C)
      - CONTRIBUCION (P)
      - INICIATIVA_ESTRATEGICA (Q)
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    pri_idx  = column_index_from_string(COLS["PRIORIZADO"])
    cont_idx = column_index_from_string(COLS["CONTRIBUCION"])
    inic_idx = column_index_from_string(COLS["INICIATIVA_ESTRATEGICA"])

    pri_val = "SI" if str(priorizado).upper().startswith("SI") else "NO"
    ws.cell(row=row, column=pri_idx).value  = pri_val
    ws.cell(row=row, column=cont_idx).value = float(contribucion) if contribucion is not None else 0.0
    ws.cell(row=row, column=inic_idx).value = iniciativa

    wb.save(EXCEL_PATH)

def update_alistamiento_rating(row, rating):
    """
    Actualiza rating de alistamiento (1 a 5) en columna RATING_PO_SYNC.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    rating_idx = column_index_from_string(COLS["RATING_PO_SYNC"])

    r = int(rating) if rating is not None else 0
    if r < 0:
        r = 0
    if r > 5:
        r = 5
    ws.cell(row=row, column=rating_idx).value = r

    wb.save(EXCEL_PATH)


# ======================================================================
# 14. UI – MESA DE EXPERTOS (PÁGINA 4, sin rating)
# ======================================================================

def build_expert_panel():
    """Panel de Mesa de Expertos para priorizar y asignar contribución / iniciativa."""

    header_box = build_header(
        "Mesa de Expertos",
        "Priorización de iniciativas y análisis de dependencias para líderes y Head of TI"
    )

    expert_state = {
        "projects": [],
        "index": 0,
    }

    w_tren = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Tren:",
        placeholder="Filtrar por Area/Tren/CoE (opcional)",
        ensure_option=False,
        layout=widgets.Layout(width="420px"),
        style={"description_width": "50px"},
    )
    btn_clear_tren = widgets.Button(
        icon="times",
        tooltip="Limpiar Tren",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    def on_clear_tren(b):
        w_tren.value = ""

    btn_clear_tren.on_click(on_clear_tren)
    box_tren = widgets.HBox([w_tren, btn_clear_tren])

    w_counter = widgets.Label(
        value="Proyectos evaluados: 0 / 0",
        layout=widgets.Layout(width="300px"),
    )

    html_proj = widgets.HTML(
        layout=widgets.Layout(width="100%", height="150px")
    )
    html_stats = widgets.HTML(
        layout=widgets.Layout(width="100%", height="120px")
    )

    w_priorizado = widgets.ToggleButtons(
        options=[("No priorizado", "NO"), ("Priorizar (Sí)", "SI")],
        description="Priorizado:",
        style={"description_width": "90px"},
        layout=widgets.Layout(width="360px"),
    )

    w_contrib = widgets.FloatText(
        value=0.0,
        description="Contribución:",
        layout=widgets.Layout(width="260px"),
        style={"description_width": "100px"},
    )

    w_inic_exp = widgets.Combobox(
        options=catalogs.get("iniciativas", []),
        description="Inic. Estrat.:",
        placeholder="Selecciona iniciativa…",
        ensure_option=False,
        layout=widgets.Layout(width="360px"),
        style={"description_width": "100px"},
    )

    btn_save_next = widgets.Button(
        description="Guardar y siguiente",
        button_style="success",
        icon="check",
        layout=widgets.Layout(width="220px", height="35px"),
    )
    btn_skip = widgets.Button(
        description="Saltar",
        button_style="warning",
        icon="forward",
        layout=widgets.Layout(width="120px", height="35px", margin="0 0 0 10px"),
    )
    btn_prev = widgets.Button(
        description="Volver",
        button_style="",
        icon="arrow-left",
        layout=widgets.Layout(width="120px", height="35px", margin="0 0 0 10px"),
    )
    btn_reload = widgets.Button(
        description="Recargar lista",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="160px", height="35px", margin="0 0 0 10px"),
    )

    out_expert = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="80px",
            background_color="white",
        )
    )

    acc_output = widgets.Output(
        layout=widgets.Layout(
            padding="6px",
            background_color="white",
        )
    )
    with acc_output:
        print(get_cancelled_detained_summary())
    accordion = widgets.Accordion(children=[acc_output])
    accordion.set_title(0, "Proyectos cancelados / detenidos (informativo)")

    def render_current():
        out_expert.clear_output()
        projects = expert_state["projects"]
        idx      = expert_state["index"]
        total    = len(projects)

        if total == 0:
            w_counter.value = "Proyectos evaluados: 0 / 0"
            html_proj.value = (
                "<div style='font-family:Segoe UI, Arial; font-size:14px;'>"
                "No hay proyectos en estado Nuevo / En curso para priorizar."
                "</div>"
            )
            html_stats.value = ""
            btn_save_next.disabled = True
            btn_skip.disabled      = True
            btn_prev.disabled      = True
            return

        if idx >= total:
            w_counter.value = f"Proyectos evaluados: {total} / {total}"
            html_proj.value = (
                "<div style='font-family:Segoe UI, Arial; font-size:14px;'>"
                "Has llegado al final de la lista de proyectos.</div>"
            )
            html_stats.value = ""
            btn_save_next.disabled = True
            btn_skip.disabled      = True
            btn_prev.disabled      = False if total > 0 else True
            return

        btn_save_next.disabled = False
        btn_skip.disabled      = False
        btn_prev.disabled      = (idx == 0)

        p = projects[idx]
        w_counter.value = f"Proyectos evaluados: {idx} / {total}"

        sem_color, sem_text = dep_semaforo(p["total_dep"], p["total_L"], p["total_P"])

        pending_list_html = ""
        if p["pending_equips"]:
            items = "".join(f"<li>{eq}</li>" for eq in p["pending_equips"])
            pending_list_html = f"""
            <div style="margin-top:4px;">
              <b>Dependencias pendientes (P):</b>
              <ul style="margin:4px 0 0 18px; padding:0; font-size:12px;">
                {items}
              </ul>
            </div>
            """

        html_proj.value = f"""
        <div style="
            font-family:Segoe UI, Arial;
            font-size:14px;
            color:{DARK_COLOR};
        ">
          <div style="font-size:20px; font-weight:600; margin-bottom:6px;">
            {p['nombre']}
          </div>
          <div style="font-size:14px; margin-bottom:8px; line-height:1.4;">
            {p['descripcion']}
          </div>
          <div style="font-size:13px; color:#333; margin-bottom:4px;">
            Estado actual: <b>{p['estado']}</b> · Priorizado actual: <b>{p['priorizado']}</b>
          </div>
          <div style="font-size:13px; display:flex; align-items:center; margin-top:4px;">
            <span style="
                display:inline-block;
                width:14px;
                height:14px;
                border-radius:50%;
                background-color:{sem_color};
                margin-right:6px;
                border:1px solid #999;
            "></span>
            <span>{sem_text}</span>
          </div>
          {pending_list_html}
        </div>
        """

        html_stats.value = f"""
        <div style="
            font-family:Segoe UI, Arial;
            font-size:13px;
            color:{DARK_COLOR};
        ">
          <b>Dependencias totales:</b> {p['total_dep']}<br>
          <b>Negociadas (L):</b> {p['total_L']} · <b>Pendientes (P):</b> {p['total_P']}<br>
          <b>% cubrimiento (P / total):</b> {p['cobertura_pct']:.1f}%<br>
          <b>Contribución actual:</b> {p['contribucion']:.2f} ·
          <b>Inic. Estratégica:</b> {p['iniciativa']}
        </div>
        """

        pri_val = str(p["priorizado"]).upper()
        w_priorizado.value = "SI" if pri_val == "SI" else "NO"
        w_contrib.value    = float(p["contribucion"])
        w_inic_exp.value   = p["iniciativa"]

    def reload_projects(b=None):
        expert_state["projects"] = get_expert_project_list(tren_filter=w_tren.value or None)
        expert_state["index"]    = 0
        render_current()
        with out_expert:
            out_expert.clear_output()
            total = len(expert_state["projects"])
            filtro_txt = f" (Tren = {w_tren.value})" if w_tren.value else ""
            print(f"🔄 Lista de proyectos para Mesa de Expertos recargada ({total} proyecto(s)){filtro_txt}.")

    def on_save_next(b):
        out_expert.clear_output()
        projects = expert_state["projects"]
        idx      = expert_state["index"]

        if idx >= len(projects):
            with out_expert:
                print("No hay más proyectos que guardar.")
            return

        p = projects[idx]
        row = p["row"]

        priorizado  = w_priorizado.value
        contrib     = w_contrib.value
        iniciativa  = w_inic_exp.value or ""

        try:
            update_expert_fields(row, priorizado, contrib, iniciativa)
            with out_expert:
                print(
                    f"✅ Proyecto actualizado en fila {row}. "
                    f"Priorizado={priorizado}, Contribución={contrib}, Inic='{iniciativa}'."
                )
        except PermissionError as e:
            with out_expert:
                print("❌ No se pudo guardar (archivo bloqueado). Cierra el Excel si está abierto.")
                print("   Detalle:", e)
            return
        except Exception as e:
            with out_expert:
                print("❌ Error inesperado al guardar:", repr(e))
            return

        expert_state["index"] += 1
        render_current()

    def on_skip(b):
        expert_state["index"] += 1
        render_current()

    def on_prev(b):
        if expert_state["index"] > 0:
            expert_state["index"] -= 1
            render_current()

    btn_save_next.on_click(on_save_next)
    btn_skip.on_click(on_skip)
    btn_prev.on_click(on_prev)
    btn_reload.on_click(reload_projects)

    reload_projects()

    body = widgets.VBox(
        [
            widgets.HBox([box_tren, w_counter]),
            widgets.HTML(
                f"<hr style='margin:4px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            html_proj,
            html_stats,
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Decisión de la Mesa de Expertos</b>"
            ),
            widgets.HBox([w_priorizado, w_contrib]),
            widgets.HBox([w_inic_exp]),
            widgets.HBox([btn_save_next, btn_skip, btn_prev, btn_reload]),
            widgets.HTML(
                f"<hr style='margin:8px 0;border:0;border-top:1px solid {CARD_BORDER};'>"
            ),
            widgets.HTML(
                "<span style='font-size:11px;font-family:Segoe UI, Arial;color:#555;'>"
                "Solo se muestran proyectos en estado <b>Nuevo</b> o <b>En curso</b>. "
                "Los proyectos cancelados/detenidos se consultan abajo como referencia."
                "</span>"
            ),
            accordion,
            out_expert,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_expert = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_expert


# ======================================================================
# 15. UI – MESA DE ALISTAMIENTO (PO Sync) (PÁGINA 5)
# ======================================================================

def build_alistamiento_panel():
    """
    Mesa de Alistamiento (PO Sync):
    - Rating 1–5 estrellas
    - Filtro por Tren
    - Nombre, descripción, dependencias, semáforo, contribución, iniciativa
    """
    header_box = build_header(
        "Mesa de Alistamiento (PO Sync)",
        "Evaluación de alistamiento en estrellas (1–5) por tren, dependencias y contribución"
    )

    state = {
        "projects": [],
        "index": 0,
    }

    w_tren = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Tren:",
        placeholder="Filtrar por Area/Tren/CoE (opcional)",
        ensure_option=False,
        layout=widgets.Layout(width="420px"),
        style={"description_width": "50px"},
    )
    btn_clear_tren = widgets.Button(
        icon="times",
        tooltip="Limpiar Tren",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    def on_clear_tren(b):
        w_tren.value = ""

    btn_clear_tren.on_click(on_clear_tren)
    box_tren = widgets.HBox([w_tren, btn_clear_tren])

    w_counter = widgets.Label(
        value="Proyectos calificados: 0 / 0",
        layout=widgets.Layout(width="300px"),
    )

    html_proj = widgets.HTML(
        layout=widgets.Layout(width="100%", height="160px")
    )
    html_stats = widgets.HTML(
        layout=widgets.Layout(width="100%", height="110px")
    )

    w_stars = widgets.ToggleButtons(
        options=[
            ("★☆☆☆☆", 1),
            ("★★☆☆☆", 2),
            ("★★★☆☆", 3),
            ("★★★★☆", 4),
            ("★★★★★", 5),
        ],
        description="Alistamiento:",
        style={"description_width": "100px"},
        layout=widgets.Layout(width="420px"),
    )

    btn_save_next = widgets.Button(
        description="Guardar rating y siguiente",
        button_style="success",
        icon="star",
        layout=widgets.Layout(width="260px", height="35px"),
    )
    btn_skip = widgets.Button(
        description="Saltar",
        button_style="warning",
        icon="forward",
        layout=widgets.Layout(width="120px", height="35px", margin="0 0 0 10px"),
    )
    btn_prev = widgets.Button(
        description="Volver",
        button_style="",
        icon="arrow-left",
        layout=widgets.Layout(width="120px", height="35px", margin="0 0 0 10px"),
    )
    btn_reload = widgets.Button(
        description="Recargar lista",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="160px", height="35px", margin="0 0 0 10px"),
    )

    out_alist = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="80px",
            background_color="white",
        )
    )

    def render_current():
        out_alist.clear_output()
        projects = state["projects"]
        idx      = state["index"]
        total    = len(projects)

        if total == 0:
            w_counter.value = "Proyectos calificados: 0 / 0"
            html_proj.value = (
                "<div style='font-family:Segoe UI, Arial; font-size:14px;'>"
                "No hay proyectos en estado Nuevo / En curso para alistamiento."
                "</div>"
            )
            html_stats.value = ""
            btn_save_next.disabled = True
            btn_skip.disabled      = True
            btn_prev.disabled      = True
            return

        if idx >= total:
            w_counter.value = f"Proyectos calificados: {total} / {total}"
            html_proj.value = (
                "<div style='font-family:Segoe UI, Arial; font-size:14px;'>"
                "Has llegado al final de la lista de proyectos.</div>"
            )
            html_stats.value = ""
            btn_save_next.disabled = True
            btn_skip.disabled      = True
            btn_prev.disabled      = False if total > 0 else True
            return

        btn_save_next.disabled = False
        btn_skip.disabled      = False
        btn_prev.disabled      = (idx == 0)

        p = projects[idx]
        w_counter.value = f"Proyectos calificados: {idx} / {total}"

        sem_color, sem_text = dep_semaforo(p["total_dep"], p["total_L"], p["total_P"])

        pending_list_html = ""
        if p["pending_equips"]:
            items = "".join(f"<li>{eq}</li>" for eq in p["pending_equips"])
            pending_list_html = f"""
            <div style="margin-top:4px;">
              <b>Dependencias pendientes (P):</b>
              <ul style="margin:4px 0 0 18px; padding:0; font-size:12px;">
                {items}
              </ul>
            </div>
            """

        html_proj.value = f"""
        <div style="
            font-family:Segoe UI, Arial;
            font-size:14px;
            color:{DARK_COLOR};
        ">
          <div style="font-size:20px; font-weight:600; margin-bottom:6px;">
            {p['nombre']}
          </div>
          <div style="font-size:14px; margin-bottom:8px; line-height:1.4;">
            {p['descripcion']}
          </div>
          <div style="font-size:13px; color:#333; margin-bottom:4px;">
            Estado actual: <b>{p['estado']}</b> · Priorizado: <b>{p['priorizado']}</b>
          </div>
          <div style="font-size:13px; display:flex; align-items:center; margin-top:4px;">
            <span style="
                display:inline-block;
                width:14px;
                height:14px;
                border-radius:50%;
                background-color:{sem_color};
                margin-right:6px;
                border:1px solid #999;
            "></span>
            <span>{sem_text}</span>
          </div>
          {pending_list_html}
        </div>
        """

        html_stats.value = f"""
        <div style="
            font-family:Segoe UI, Arial;
            font-size:13px;
            color:{DARK_COLOR};
        ">
          <b>Dependencias totales:</b> {p['total_dep']}<br>
          <b>Negociadas (L):</b> {p['total_L']} · <b>Pendientes (P):</b> {p['total_P']}<br>
          <b>% cubrimiento (P / total):</b> {p['cobertura_pct']:.1f}%<br>
          <b>Contribución:</b> {p['contribucion']:.2f} ·
          <b>Inic. Estratégica:</b> {p['iniciativa']}
        </div>
        """

        # rating actual
        rating_po = p.get("rating_po", 0)
        if rating_po < 1 or rating_po > 5:
            rating_po = 3
        w_stars.value = rating_po

    def reload_projects(b=None):
        state["projects"] = get_expert_project_list(tren_filter=w_tren.value or None)
        state["index"]    = 0
        render_current()
        with out_alist:
            out_alist.clear_output()
            total = len(state["projects"])
            filtro_txt = f" (Tren = {w_tren.value})" if w_tren.value else ""
            print(f"🔄 Lista de proyectos para Mesa de Alistamiento recargada ({total} proyecto(s)){filtro_txt}.")

    def on_save_next(b):
        out_alist.clear_output()
        projects = state["projects"]
        idx      = state["index"]

        if idx >= len(projects):
            with out_alist:
                print("No hay más proyectos que guardar.")
            return

        p = projects[idx]
        row = p["row"]
        rating = w_stars.value

        try:
            update_alistamiento_rating(row, rating)
            with out_alist:
                print(
                    f"✅ Rating de alistamiento actualizado en fila {row}. "
                    f"Estrellas = {rating}."
                )
        except PermissionError as e:
            with out_alist:
                print("❌ No se pudo guardar (archivo bloqueado). Cierra el Excel si está abierto.")
                print("   Detalle:", e)
            return
        except Exception as e:
            with out_alist:
                print("❌ Error inesperado al guardar rating:", repr(e))
            return

        state["index"] += 1
        render_current()

    def on_skip(b):
        state["index"] += 1
        render_current()

    def on_prev(b):
        if state["index"] > 0:
            state["index"] -= 1
            render_current()

    btn_save_next.on_click(on_save_next)
    btn_skip.on_click(on_skip)
    btn_prev.on_click(on_prev)
    btn_reload.on_click(reload_projects)

    reload_projects()

    body = widgets.VBox(
        [
            widgets.HBox([box_tren, w_counter]),
            widgets.HTML(
                f"<hr style='margin:4px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            html_proj,
            html_stats,
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Calificación de alistamiento (PO Sync)</b>"
            ),
            widgets.HBox([w_stars]),
            widgets.HBox([btn_save_next, btn_skip, btn_prev, btn_reload]),
            out_alist,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_alist = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_alist


# ======================================================================
# 16. FEEDBACK – HOJA SUGERENCIAS (PÁGINA 6)
# ======================================================================

def append_suggestion(usuario, sugerencia):
    """
    Añade una sugerencia a la hoja 'Sugerencias':
    Columnas: Usuario (A), Sugerencia (B)
    """
    wb = load_workbook()
    ws = get_ws_sugerencias(wb)

    next_row = ws.max_row + 1
    ws.cell(row=next_row, column=1).value = usuario
    ws.cell(row=next_row, column=2).value = sugerencia

    wb.save(EXCEL_PATH)

def build_feedback_panel():
    """Página de feedback: login sencillo (nombre) + sugerencia."""

    header_box = build_header(
        "Feedback y sugerencias",
        "Comparte aprendizajes, fricciones y mejoras sobre la herramienta de Gestión de Dependencias TI"
    )

    w_user = widgets.Text(
        description="Nombre:",
        placeholder="Tu nombre / rol",
        layout=widgets.Layout(width="400px"),
        style={"description_width": "70px"},
    )
    w_sugg = widgets.Textarea(
        description="Sugerencia:",
        placeholder="Escribe aquí tu feedback, ideas de mejora, bugs, etc.",
        layout=widgets.Layout(width="650px", height="140px"),
        style={"description_width": "80px"},
    )

    btn_send = widgets.Button(
        description="Enviar feedback",
        button_style="success",
        icon="paper-plane",
        layout=widgets.Layout(width="200px", height="35px", margin="6px 0 0 0"),
    )

    out_feedback = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="90px",
            background_color="white",
        )
    )

    def on_send_clicked(b):
        out_feedback.clear_output()
        with out_feedback:
            usuario = w_user.value.strip()
            suger   = w_sugg.value.strip()
            if not usuario:
                print("⚠️ Por favor indica tu nombre.")
                return
            if not suger:
                print("⚠️ La sugerencia está vacía.")
                return
            try:
                append_suggestion(usuario, suger)
                print(f"✅ Gracias, {usuario}. Tu feedback fue registrado en la hoja 'Sugerencias'.")
                w_sugg.value = ""
            except PermissionError as e:
                print("❌ No se pudo guardar la sugerencia (archivo bloqueado).")
                print("   Cierra el Excel si está abierto.")
                print("   Detalle:", e)
            except Exception as e:
                print("❌ Error inesperado al guardar la sugerencia:", repr(e))

    btn_send.on_click(on_send_clicked)

    body = widgets.VBox(
        [
            widgets.HTML(
                "<div style='font-family:Segoe UI, Arial; font-size:13px; color:#333;'>"
                "Usaremos estas sugerencias para ajustar tanto la UI del notebook "
                "como la estructura de la base GD_v1. "
                "</div>"
            ),
            widgets.HTML("<br>"),
            widgets.HBox([w_user]),
            widgets.HBox([w_sugg]),
            widgets.HBox([btn_send]),
            out_feedback,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_feedback = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_feedback


# ======================================================================
# 17. APP SHELL – TABS (ALTA / CONSULTA / MÉTRICAS / MESA EXP / ALIST / FEEDBACK)
# ======================================================================

form_panel      = build_create_form()
consulta_panel  = build_consult_panel()
metrics_panel   = build_metrics_panel()
expert_panel    = build_expert_panel()
alist_panel     = build_alistamiento_panel()
feedback_panel  = build_feedback_panel()

tabs = widgets.Tab(
    children=[
        form_panel,
        consulta_panel,
        metrics_panel,
        expert_panel,
        alist_panel,
        feedback_panel,
    ],
    layout=widgets.Layout(width="930px"),
)
tabs.set_title(0, "Alta de proyectos")
tabs.set_title(1, "Consulta y edición")
tabs.set_title(2, "Métricas")
tabs.set_title(3, "Mesa de Expertos")
tabs.set_title(4, "Mesa de Alistamiento")
tabs.set_title(5, "Feedback")

app_shell = widgets.VBox(
    [
        widgets.HTML(
            f"""
            <div style="
                background: linear-gradient(90deg, {DARK_COLOR}, {PRIMARY_COLOR});
                color:white;
                padding:8px 16px;
                border-radius:10px 10px 0 0;
                font-family:Segoe UI, Arial, sans-serif;
                font-size:13px;
                margin-bottom:4px;
            ">
              Gestión de Dependencias TI · Cofre GD_v1 (Notebook Python + Excel)
            </div>
            """
        ),
        tabs,
    ],
    layout=widgets.Layout(
        width="950px",
        margin="10px 0 30px 0",
    ),
)

display(app_shell)


In [1]:
# ======================================================================
# 0. INSTALACIÓN (si hace falta, solo una vez por entorno)
# ======================================================================
# !pip install openpyxl ipywidgets matplotlib

# ======================================================================
# 1. IMPORTS Y CONFIGURACIÓN GENERAL
# ======================================================================
import openpyxl
from openpyxl.utils import column_index_from_string, get_column_letter
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# ----------------------------------------------------------------------
# ÍNDICE GENERAL DE MÓDULOS / CAPÍTULOS
# ----------------------------------------------------------------------
# CAP 0. Instalación
#         Comentario inicial con el pip install recomendado.
#
# CAP 1. Imports y configuración general
#         Librerías, rutas, nombres de hojas, columnas y estilos globales.
#
# CAP 2. Funciones básicas para Excel + utilidades
#         Abrir libro, obtener hojas, localizar cabeceras, columnas,
#         conversión de valores numéricos, etc.
#
# CAP 3. Carga de catálogos y mapping de dependencias (hoja Datos)
#         Lee los catálogos (estados, responsables, áreas, trenes, etc.)
#         y el mapeo Célula → columna de descripción de dependencia.
#
# CAP 4. Núcleo de dependencias: agregados + semáforo
#         Calcula totales L/P, cubrimiento y actualiza columnas CN–CQ.
#         Define el semáforo de dependencias (color + texto).
#
# CAP 5. Alta de proyectos
#         Inserta un nuevo proyecto en ProyectosTI, escribe sus campos,
#         aplica dependencias dinámicas y agrega métricas de dependencias.
#
# CAP 6. Consultas / resúmenes
#         Funciones para resumir por equipo (célula/tren) o por proyecto,
#         incluyendo avance, línea base y detalle de dependencias.
#
# CAP 7. Actualizar proyecto existente
#         Actualiza avance / estimado / % cumplimiento y dependencias
#         para una fila existente de ProyectosTI.
#
# CAP 8. Cálculo de métricas agregadas
#         Métricas globales o filtradas por área/célula para la página
#         de Métricas (totales de dependencias, coberturas, promedios).
#
# CAP 9. UI – Header Telefónica corporativo
#         Construye el encabezado visual con logo y título/subtítulo.
#
# CAP 10. UI – Alta de proyectos (Página 1)
#         Formulario de registro de proyectos + dependencias dinámicas,
#         con validaciones y escritura en Excel.
#
# CAP 11. UI – Consulta y edición (Página 2)
#         Búsqueda por equipo o proyecto, muestra resumen y permite
#         actualizar avance y dependencias.
#
# CAP 12. UI – Métricas (Página 3)
#         Panel para ver métricas agregadas y gráficos con filtros por
#         ámbito (todos, área/tren o célula).
#
# CAP 13. Mesa de Expertos – lógica (datos + actualización)
#         Construye la lista de proyectos a evaluar, trae dependencias
#         pendientes y permite actualizar priorización y contribución.
#
# CAP 14. UI – Mesa de Expertos (Página 4)
#         Interfaz para recorrer proyectos, ver semáforo grande con
#         dependencias pendientes debajo, y registrar decisión.
#
# CAP 15. UI – Mesa de Alistamiento (PO Sync) (Página 5)
#         Interfaz para calificar alistamiento con estrellas por tren.
#         El semáforo grande y las dependencias pendientes se muestran
#         ahora en una columna a la derecha; las estrellas son el foco.
#
# CAP 16. Feedback – hoja Sugerencias (Página 6)
#         Formulario simple para registrar feedback en la hoja
#         'Sugerencias' del Excel.
#
# CAP 17. App Shell – Tabs y display
#         Construye el contenedor principal con pestañas y muestra
#         toda la aplicación en el notebook.
# ----------------------------------------------------------------------


# --- RUTA DEL EXCEL (MAC) ---
EXCEL_PATH = Path("/Users/macuser/Desktop/DIR COMERCIAL/GD/GD_v1.xlsx")

# --- NOMBRES DE HOJAS ---
SHEET_PROYECTOS = "ProyectosTI"
SHEET_DATOS     = "Datos"
SHEET_SUG       = "Sugerencias"

# --- FILAS / CABECERAS ---
START_ROW_PROYECTOS   = 12   # primera fila de datos
HEADER_ROW_PROYECTOS  = 11   # fila de cabecera por defecto

# --- COLUMNAS BASE ---
COLS = {
    "ID": "A",
    "Q_RADICADO": "B",
    "PRIORIZADO": "C",
    "ESTADO_PROYECTO": "D",
    "NOMBRE_PROYECTO": "E",
    "DESCRIPCION_PROYECTO": "F",
    "RESPONSABLE_PROYECTO": "G",
    "AREA_SOLICITANTE": "H",
    "FECHA_INICIO": "I",
    "FECHA_ESTIMADA_CIERRE": "J",
    "LINEA_BASE": "K",
    "LINEA_BASE_Q_GESTION": "L",
    "AVANCE": "M",
    "ESTIMADO_AVANCE": "N",
    "PORC_CUMPLIMIENTO": "O",
    "CONTRIBUCION": "P",              # valor numérico Mesa de Expertos
    "INICIATIVA_ESTRATEGICA": "Q",    # catálogo desde Datos
    # Agregados de dependencias:
    "TOTAL_DEP": "CN",        # L + P
    "TOTAL_L": "CO",          # solo L
    "TOTAL_P": "CP",          # solo P
    "CUBRIMIENTO_DEP": "CQ",  # P / (L+P)  → % de P sobre el total
    # Rating de Mesa de Alistamiento (PO Sync):
    "RATING_PO_SYNC": "CR",   # 1 a 5 estrellas
}

# --- RANGOS DE DEPENDENCIAS (FLAGS y DESCRIPCIONES) ---
FLAG_START_LETTER = "R"
FLAG_END_LETTER   = "BB"
DESC_START_LETTER = "BC"
DESC_END_LETTER   = "CM"

FLAG_START_COL = column_index_from_string(FLAG_START_LETTER)
FLAG_END_COL   = column_index_from_string(FLAG_END_LETTER)
DESC_START_COL = column_index_from_string(DESC_START_LETTER)
DESC_END_COL   = column_index_from_string(DESC_END_LETTER)

# --- VARIABLES GLOBALES ---
DEP_MAPPING = {}   # {celula/tren/coe -> header columna descripción}
catalogs    = {}   # catálogos para combos

# --- ESTILO CORPORATIVO TELEFÓNICA ---
PRIMARY_COLOR = "#00a9e0"
DARK_COLOR    = "#001b3c"
LIGHT_BG      = "#f5f9fc"
CARD_BORDER   = "#d0d7de"

# --- LOGO TELEFÓNICA ---
TEF_LOGO = None
try:
    with open("/Users/macuser/Desktop/DIR COMERCIAL/GD/Telefonica logo.png", "rb") as f:
        TEF_LOGO = widgets.Image(
            value=f.read(),
            format="png",
            layout=widgets.Layout(width="80px", height="auto", margin="0 12px 0 0"),
        )
except Exception as e:
    print("⚠️ No se pudo cargar el logo de Telefónica:", e)


# ======================================================================
# 2. FUNCIONES BÁSICAS PARA EXCEL + UTILIDADES
# ======================================================================

def load_workbook():
    """Abre el libro de Excel (.xlsx) y valida la existencia de la hoja ProyectosTI."""
    if not EXCEL_PATH.exists():
        raise FileNotFoundError(f"No se encontró el archivo: {EXCEL_PATH}")
    wb = openpyxl.load_workbook(EXCEL_PATH, keep_vba=False)
    if SHEET_PROYECTOS not in wb.sheetnames:
        raise KeyError(f"No existe la hoja '{SHEET_PROYECTOS}'.")
    return wb

def get_ws_proyectos(wb=None):
    """Devuelve la hoja ProyectosTI del workbook."""
    wb = wb or load_workbook()
    return wb[SHEET_PROYECTOS]

def get_ws_datos(wb=None):
    """Devuelve la hoja Datos del workbook."""
    wb = wb or load_workbook()
    if SHEET_DATOS not in wb.sheetnames:
        raise KeyError(f"No existe la hoja '{SHEET_DATOS}'.")
    return wb[SHEET_DATOS]

def get_ws_sugerencias(wb=None):
    """
    Devuelve la hoja 'Sugerencias'. Si no existe, la crea con cabecera.
    """
    wb = wb or load_workbook()
    if SHEET_SUG not in wb.sheetnames:
        ws = wb.create_sheet(SHEET_SUG)
        ws["A1"] = "Usuario"
        ws["B1"] = "Sugerencia"
    else:
        ws = wb[SHEET_SUG]
        if ws.max_row == 1 and ws["A1"].value is None:
            ws["A1"] = "Usuario"
            ws["B1"] = "Sugerencia"
    return ws

def get_header_row_proyectos(ws):
    """
    Detecta la fila de cabeceras buscando el texto 'ID'
    en la columna COLS['ID'].
    """
    id_col_idx = column_index_from_string(COLS["ID"])
    for r in range(1, START_ROW_PROYECTOS):
        v = ws.cell(row=r, column=id_col_idx).value
        if isinstance(v, str) and v.strip().lower() == "id":
            return r
    return HEADER_ROW_PROYECTOS

def get_unique_list_from_column(ws, col_letter, start_row=2):
    """Devuelve una lista ordenada sin duplicados de una columna."""
    values = set()
    col_idx = column_index_from_string(col_letter)
    for row in range(start_row, ws.max_row + 1):
        value = ws.cell(row=row, column=col_idx).value
        if value not in (None, ""):
            values.add(str(value))
    return sorted(values)

def get_next_row_and_id(ws, id_col_letter="A", start_row=START_ROW_PROYECTOS):
    """
    Busca la siguiente fila libre y el siguiente ID numérico.
    """
    id_col_idx = column_index_from_string(id_col_letter)
    max_row_used = 0
    max_id_found = 0

    for row in range(start_row, ws.max_row + 1):
        val = ws.cell(row=row, column=id_col_idx).value
        if val not in (None, ""):
            max_row_used = row
            if isinstance(val, (int, float)):
                max_id_found = max(max_id_found, int(val))

    next_row = start_row if max_row_used == 0 else max_row_used + 1
    next_id  = max_id_found + 1 if max_id_found > 0 else 1
    return next_row, next_id

def find_column_by_header(ws, header_name, header_row=1):
    """Busca una columna por el texto exacto del header (case-insensitive)."""
    if not header_name:
        return None
    target = str(header_name).strip().lower()
    for col_idx in range(1, ws.max_column + 1):
        val = ws.cell(row=header_row, column=col_idx).value
        if val is None:
            continue
        if str(val).strip().lower() == target:
            return col_idx
    return None

def find_column_by_header_in_range(ws, header_name, start_col_idx, end_col_idx, header_row):
    """Busca una columna por header, restringida a un rango de columnas."""
    if not header_name:
        return None
    target = str(header_name).strip().lower()
    for col_idx in range(start_col_idx, end_col_idx + 1):
        val = ws.cell(row=header_row, column=col_idx).value
        if val is None:
            continue
        if str(val).strip().lower() == target:
            return col_idx
    return None

def find_area_tren_coe_col(ws):
    """
    Intenta localizar la columna de Area/Tren/CoE en la hoja ProyectosTI,
    buscando palabras clave en la fila de cabeceras.
    """
    header_row = get_header_row_proyectos(ws)
    candidate_idx = None
    for col_idx in range(1, ws.max_column + 1):
        val = ws.cell(row=header_row, column=col_idx).value
        if not val:
            continue
        s = str(val).strip().lower()
        if "tren" in s and "coe" in s:
            return col_idx
        if s in ("area tren coe", "area/tren/coe"):
            candidate_idx = col_idx
    return candidate_idx

def to_num_cell(v):
    """Convierte cualquier valor de celda a float, tolerando texto, %, comas, etc."""
    if v is None or v == "":
        return 0.0
    if isinstance(v, (int, float)):
        return float(v)
    s = str(v).strip()
    if s == "":
        return 0.0
    s = s.replace("%", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return 0.0


# ======================================================================
# 3. CARGA DE CATÁLOGOS Y MAPPING DE DEPENDENCIAS (HOJA DATOS)
# ======================================================================

def load_dependency_mapping(wb=None):
    """
    Lee de la hoja 'Datos':
      - 'Celula Dependencia'              → header de flag (ej. 'CÉLULA TASADORES')
      - 'Celula Descripcion Dependencia' → header de descripción (ej. 'DESCRIPCION CÉLULA TASADORES')
    y arma: { 'CÉLULA TASADORES': 'DESCRIPCION CÉLULA TASADORES', ... }
    """
    wb = wb or load_workbook()
    ws_d = get_ws_datos(wb)
    col_cel_dep_idx  = find_column_by_header(ws_d, "Celula Dependencia", header_row=1)
    col_desc_dep_idx = find_column_by_header(ws_d, "Celula Descripcion Dependencia", header_row=1)
    mapping = {}
    if not col_cel_dep_idx or not col_desc_dep_idx:
        return mapping

    for row in range(2, ws_d.max_row + 1):
        cel_name   = ws_d.cell(row=row, column=col_cel_dep_idx).value
        desc_header= ws_d.cell(row=row, column=col_desc_dep_idx).value
        if cel_name and desc_header:
            mapping[str(cel_name).strip()] = str(desc_header).strip()
    return mapping

def load_catalogs():
    """
    Carga catálogos y mapeo de dependencias desde 'Datos'.
    """
    global DEP_MAPPING, catalogs
    wb   = load_workbook()
    ws_d = get_ws_datos(wb)

    estados_list       = get_unique_list_from_column(ws_d, "A")
    priorizacion_list  = get_unique_list_from_column(ws_d, "B")
    responsables_list  = get_unique_list_from_column(ws_d, "C")
    areas_list         = get_unique_list_from_column(ws_d, "D")
    area_tren_coe_list = get_unique_list_from_column(ws_d, "E")

    # Iniciativas Estratégicas
    iniciativas_list = []
    col_ini_idx = find_column_by_header(ws_d, "Iniciativa Estrategica", header_row=1)
    if col_ini_idx:
        col_ini_letter = get_column_letter(col_ini_idx)
        iniciativas_list = get_unique_list_from_column(ws_d, col_ini_letter)
    else:
        iniciativas_list = []

    DEP_MAPPING        = load_dependency_mapping(wb)
    celulas_dep_list   = sorted(DEP_MAPPING.keys()) if DEP_MAPPING else []

    catalogs = {
        "estados": estados_list,
        "q_rad": priorizacion_list,
        "responsables": responsables_list,
        "areas": areas_list,
        "area_tren_coe": area_tren_coe_list,
        "celulas_dep": celulas_dep_list,
        "iniciativas": iniciativas_list,
    }

# Cargamos catálogos al arrancar
try:
    load_catalogs()
except Exception as e:
    print("⚠️ No se pudieron cargar catálogos desde 'Datos'. Motivo:", e)
    catalogs = {
        "estados": ["Nuevo", "En curso", "Detenido", "Cancelado", "Finalizado"],
        "q_rad": ["1Q/25", "2Q/25"],
        "responsables": ["Responsable 1"],
        "areas": ["Área 1"],
        "area_tren_coe": ["Tren X"],
        "celulas_dep": ["Célula A"],
        "iniciativas": ["Inic. 1"],
    }
    DEP_MAPPING = {}


# ======================================================================
# 4. NÚCLEO DEPENDENCIAS: AGREGADOS + SEMÁFORO
# ======================================================================

def compute_dep_aggregates(dep_list):
    """
    A partir de dep_list calcula:
      - total_dep = L+P
      - total_L
      - total_P
      - cubrimiento = P / (L+P)  (0 si no hay dependencias)
    """
    flags = [ (d.get("codigo") or "").strip().upper()
              for d in dep_list
              if (d.get("equipo") or "").strip() ]
    flags = [f for f in flags if f in ("L", "P")]
    total_dep = len(flags)
    total_L   = sum(1 for f in flags if f == "L")
    total_P   = sum(1 for f in flags if f == "P")
    if total_dep > 0:
        cubrimiento = total_P / total_dep  # % de P sobre el total
    else:
        cubrimiento = 0.0
    return total_dep, total_L, total_P, cubrimiento

def write_dep_aggregates(ws, row, dep_list):
    """
    Escribe en CN–CQ:
      - CN: total L+P
      - CO: solo L
      - CP: solo P
      - CQ: P / (L+P)   (para formatear como % en Excel)
    """
    total_dep, total_L, total_P, cub = compute_dep_aggregates(dep_list)
    cn = column_index_from_string(COLS["TOTAL_DEP"])
    co = column_index_from_string(COLS["TOTAL_L"])
    cp = column_index_from_string(COLS["TOTAL_P"])
    cq = column_index_from_string(COLS["CUBRIMIENTO_DEP"])

    ws.cell(row=row, column=cn).value = total_dep
    ws.cell(row=row, column=co).value = total_L
    ws.cell(row=row, column=cp).value = total_P
    ws.cell(row=row, column=cq).value = cub

def apply_dependencies_to_row(ws, row, dep_list):
    """
    Aplica un conjunto de dependencias dinámicas en la fila `row`:
      dep_list = [{equipo, codigo(P/L), descripcion}, ...]
    Usa DEP_MAPPING + cabeceras detectadas dinámicamente.
    """
    header_row = get_header_row_proyectos(ws)

    # 1) Limpiar flags y descripciones existentes
    for equipo, desc_header in DEP_MAPPING.items():
        flag_col_idx = find_column_by_header_in_range(
            ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
        )
        if flag_col_idx:
            ws.cell(row=row, column=flag_col_idx).value = None

        if desc_header:
            desc_col_idx = find_column_by_header_in_range(
                ws, desc_header, DESC_START_COL, DESC_END_COL, header_row
            )
            if desc_col_idx:
                ws.cell(row=row, column=desc_col_idx).value = None

    # 2) Escribir nuevas dependencias
    for dep in dep_list:
        equipo = (dep.get("equipo") or "").strip()
        codigo = (dep.get("codigo") or "").strip().upper()
        texto  = (dep.get("descripcion") or "").strip()

        if not equipo or codigo not in ("P", "L"):
            continue

        desc_header = DEP_MAPPING.get(equipo)

        flag_col_idx = find_column_by_header_in_range(
            ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
        )
        if flag_col_idx:
            ws.cell(row=row, column=flag_col_idx).value = codigo

        if desc_header:
            desc_col_idx = find_column_by_header_in_range(
                ws, desc_header, DESC_START_COL, DESC_END_COL, header_row
            )
            if desc_col_idx and texto:
                ws.cell(row=row, column=desc_col_idx).value = texto

    # 3) Escribir agregados en CN–CQ
    write_dep_aggregates(ws, row, dep_list)

def dep_semaforo(total_dep, total_L, total_P):
    """
    Semáforo de dependencias:
      - Gris   → sin dependencias
      - Verde  → todas negociadas (L)
      - Rojo   → todas pendientes (P)
      - Amarillo → mix L/P
    Devuelve (color_hex, texto_descriptivo).
    """
    if total_dep == 0:
        return "#bdc3c7", "Sin dependencias registradas"
    if total_P == 0 and total_L > 0:
        return "#2ecc71", "Todas las dependencias negociadas (L)"
    if total_L == 0 and total_P > 0:
        return "#e74c3c", "Todas las dependencias pendientes (P)"
    return "#f1c40f", "Mix de dependencias negociadas (L) y pendientes (P)"


# ======================================================================
# 5. ALTA DE PROYECTOS (NUEVO REGISTRO + DEPENDENCIAS)
# ======================================================================

def write_project_with_dependencies(project, dep_list):
    """
    Inserta un nuevo proyecto y aplica sus dependencias + métricas CN–CQ.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    next_row, next_id = get_next_row_and_id(
        ws, id_col_letter=COLS["ID"], start_row=START_ROW_PROYECTOS
    )

    project = project.copy()
    project["ID"] = next_id

    for field, col_letter in COLS.items():
        if field in project:
            col_idx = column_index_from_string(col_letter)
            ws.cell(row=next_row, column=col_idx).value = project.get(field)

    # Dependencias y agregados
    apply_dependencies_to_row(ws, next_row, dep_list)

    wb.save(EXCEL_PATH)
    return next_row, next_id


# ======================================================================
# 6. CONSULTAS / RESÚMENES
# ======================================================================

def get_all_project_names():
    """Devuelve la lista de nombres de proyecto para el combo de consulta."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    name_col_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    names = set()
    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        val = ws.cell(row=row, column=name_col_idx).value
        if val not in (None, ""):
            names.add(str(val).strip())
    return sorted(names)

def summarize_by_equipo(equipo_name):
    """Resumen de cobertura de dependencias para un equipo."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    col_flag_idx = find_column_by_header_in_range(
        ws, equipo_name, FLAG_START_COL, FLAG_END_COL, header_row
    )
    if not col_flag_idx:
        return {"found": False, "msg": f"No se encontró la columna '{equipo_name}' en R:BB."}

    name_col_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    q_col_idx    = column_index_from_string(COLS["Q_RADICADO"])

    total = pendientes = negociadas = 0
    rows = []

    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        flag = ws.cell(row=row, column=col_flag_idx).value
        if flag is None or str(flag).strip() == "":
            continue
        flag_up = str(flag).strip().upper()
        if flag_up not in ("P", "L"):
            continue

        total += 1
        if flag_up == "P":
            pendientes += 1
        else:
            negociadas += 1

        nombre = ws.cell(row=row, column=name_col_idx).value
        qrad   = ws.cell(row=row, column=q_col_idx).value
        rows.append({"fila": row, "Q_RADICADO": qrad, "PROYECTO": nombre, "FLAG": flag_up})

    pct_pend = (pendientes/total*100) if total > 0 else 0.0
    return {
        "found": True,
        "equipo": equipo_name,
        "total": total,
        "pendientes": pendientes,
        "negociadas": negociadas,
        "pct_pendientes": pct_pend,
        "rows": rows,
    }

def summarize_by_proyecto(nombre_proyecto):
    """Resumen completo de un proyecto (incluye dependencias y agregados)."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    name_col_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    q_col_idx    = column_index_from_string(COLS["Q_RADICADO"])

    target_row = None
    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        val = ws.cell(row=row, column=name_col_idx).value
        if val and str(val).strip() == nombre_proyecto:
            target_row = row
            break

    if not target_row:
        return {"found": False, "msg": f"No se encontró el proyecto '{nombre_proyecto}'."}

    detalles = []

    for equipo, desc_header in DEP_MAPPING.items():
        flag_col_idx = find_column_by_header_in_range(
            ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
        )
        if not flag_col_idx:
            continue

        flag = ws.cell(row=target_row, column=flag_col_idx).value
        if flag is None or str(flag).strip() == "":
            continue

        flag_up = str(flag).strip().upper()
        if flag_up not in ("P", "L"):
            continue

        desc = ""
        if desc_header:
            desc_col_idx = find_column_by_header_in_range(
                ws, desc_header, DESC_START_COL, DESC_END_COL, header_row
            )
            if desc_col_idx:
                desc = ws.cell(row=target_row, column=desc_col_idx).value or ""

        detalles.append({"equipo": equipo, "FLAG": flag_up, "descripcion": desc})

    total      = len(detalles)
    pendientes = sum(1 for d in detalles if d["FLAG"] == "P")
    negociadas = sum(1 for d in detalles if d["FLAG"] == "L")
    pct_pend   = (pendientes/total*100) if total > 0 else 0.0

    lb_col   = column_index_from_string(COLS["LINEA_BASE"])
    av_col   = column_index_from_string(COLS["AVANCE"])
    est_col  = column_index_from_string(COLS["ESTIMADO_AVANCE"])

    lb  = to_num_cell(ws.cell(row=target_row, column=lb_col).value)
    av  = to_num_cell(ws.cell(row=target_row, column=av_col).value)
    est = to_num_cell(ws.cell(row=target_row, column=est_col).value)
    qrad= ws.cell(row=target_row, column=q_col_idx).value

    cn = column_index_from_string(COLS["TOTAL_DEP"])
    co = column_index_from_string(COLS["TOTAL_L"])
    cp = column_index_from_string(COLS["TOTAL_P"])
    cq = column_index_from_string(COLS["CUBRIMIENTO_DEP"])

    total_dep_xl = to_num_cell(ws.cell(row=target_row, column=cn).value)
    total_L_xl   = to_num_cell(ws.cell(row=target_row, column=co).value)
    total_P_xl   = to_num_cell(ws.cell(row=target_row, column=cp).value)
    cub_xl       = to_num_cell(ws.cell(row=target_row, column=cq).value)

    return {
        "found": True,
        "fila": target_row,
        "proyecto": nombre_proyecto,
        "Q_RADICADO": qrad,
        "total_dep": total,
        "pendientes": pendientes,
        "negociadas": negociadas,
        "pct_pendientes": pct_pend,
        "detalles": detalles,
        "linea_base": float(lb),
        "avance": float(av),
        "estimado": float(est),
        "total_dep_xl": total_dep_xl,
        "total_L_xl": total_L_xl,
        "total_P_xl": total_P_xl,
        "cub_xl": float(cub_xl),
    }


# ======================================================================
# 7. ACTUALIZAR PROYECTO EXISTENTE (AVANCE + DEPENDENCIAS)
# ======================================================================

def update_project_row_and_dependencies(row, avance, estimado, dep_list):
    """
    Actualiza:
      - AVANCE, ESTIMADO_AVANCE, PORC_CUMPLIMIENTO
      - Dependencias P/L + descripciones + CN–CQ
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    lb_col  = column_index_from_string(COLS["LINEA_BASE"])
    av_col  = column_index_from_string(COLS["AVANCE"])
    est_col = column_index_from_string(COLS["ESTIMADO_AVANCE"])
    pct_col = column_index_from_string(COLS["PORC_CUMPLIMIENTO"])

    linea_base = to_num_cell(ws.cell(row=row, column=lb_col).value)
    old_av     = to_num_cell(ws.cell(row=row, column=av_col).value)
    old_es     = to_num_cell(ws.cell(row=row, column=est_col).value)

    new_av = float(avance)  if avance  is not None else float(old_av)
    new_es = float(estimado)if estimado is not None else float(old_es)

    ws.cell(row=row, column=av_col).value  = new_av
    ws.cell(row=row, column=est_col).value = new_es

    if new_es > 0:
        pct_cumpl = new_av / new_es
    else:
        pct_cumpl = 0.0
    ws.cell(row=row, column=pct_col).value = pct_cumpl

    apply_dependencies_to_row(ws, row, dep_list)

    wb.save(EXCEL_PATH)

    var_vs_lb = new_av - float(linea_base)
    return {
        "linea_base": float(linea_base),
        "avance": new_av,
        "estimado": new_es,
        "pct_cumpl": pct_cumpl * 100,
        "var_vs_lb_pp": var_vs_lb * 100,
    }


# ======================================================================
# 8. CÁLCULO DE MÉTRICAS AGREGADAS (para la página de Métricas)
# ======================================================================

def compute_metrics(scope="all", filter_value=None):
    """
    scope:
      - "all"    → todos los proyectos
      - "area"   → filtrar por Area/Tren/CoE (columna detectada automáticamente)
      - "celula" → filtrar por una célula (según flag P/L en columna R:BB)
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    name_col_idx   = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    prior_col_idx  = column_index_from_string(COLS["PRIORIZADO"])
    av_col_idx     = column_index_from_string(COLS["AVANCE"])
    cn_idx         = column_index_from_string(COLS["TOTAL_DEP"])
    co_idx         = column_index_from_string(COLS["TOTAL_L"])
    cp_idx         = column_index_from_string(COLS["TOTAL_P"])

    area_col_idx = find_area_tren_coe_col(ws)

    cel_flag_col_idx = None
    if scope == "celula" and filter_value:
        cel_flag_col_idx = find_column_by_header_in_range(
            ws, filter_value, FLAG_START_COL, FLAG_END_COL, header_row
        )

    total_projects       = 0
    total_dep            = 0.0
    total_L              = 0.0
    total_P              = 0.0
    sum_avance           = 0.0
    pri_count            = 0
    pri_avance_sum       = 0.0
    no_pri_count         = 0
    no_pri_avance_sum    = 0.0

    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        name = ws.cell(row=row, column=name_col_idx).value
        if not name:
            continue

        if scope == "area" and filter_value:
            if not area_col_idx:
                continue
            area_val = ws.cell(row=row, column=area_col_idx).value
            if str(area_val).strip() != str(filter_value).strip():
                continue

        if scope == "celula" and filter_value:
            if not cel_flag_col_idx:
                continue
            flag_val = ws.cell(row=row, column=cel_flag_col_idx).value
            if not flag_val or str(flag_val).strip().upper() not in ("P", "L"):
                continue

        total_projects += 1

        dep_val = to_num_cell(ws.cell(row=row, column=cn_idx).value)
        L_val   = to_num_cell(ws.cell(row=row, column=co_idx).value)
        P_val   = to_num_cell(ws.cell(row=row, column=cp_idx).value)

        total_dep += dep_val
        total_L   += L_val
        total_P   += P_val

        av_val = to_num_cell(ws.cell(row=row, column=av_col_idx).value)
        sum_avance += av_val

        pri_val = ws.cell(row=row, column=prior_col_idx).value
        pri_str = str(pri_val).strip().upper() if pri_val not in (None, "") else ""

        if pri_str == "SI":
            pri_count      += 1
            pri_avance_sum += av_val
        else:
            no_pri_count      += 1
            no_pri_avance_sum += av_val

    if total_projects > 0:
        avg_avance = sum_avance / total_projects
    else:
        avg_avance = 0.0

    if pri_count > 0:
        avg_pri = pri_avance_sum / pri_count
    else:
        avg_pri = 0.0

    if no_pri_count > 0:
        avg_no_pri = no_pri_avance_sum / no_pri_count
    else:
        avg_no_pri = 0.0

    if total_dep > 0:
        cobertura_pct = (total_P / total_dep) * 100.0
    else:
        cobertura_pct = 0.0

    return {
        "total_projects": int(total_projects),
        "total_dep": float(total_dep),
        "total_L": float(total_L),
        "total_P": float(total_P),
        "cobertura_pct": float(cobertura_pct),
        "avg_avance": float(avg_avance),
        "avg_pri": float(avg_pri),
        "avg_no_pri": float(avg_no_pri),
        "num_pri": int(pri_count),
    }


# ======================================================================
# 9. UI – COMPONENTES COMUNES (HEADER TELEFÓNICA)
# ======================================================================

def build_header(title, subtitle=""):
    """Header corporativo Telefónica con logo + título y subtítulo."""
    title_html = f"""
    <div style="
        display:flex;
        flex-direction:column;
        justify-content:center;
    ">
      <div style="
          font-family:Segoe UI, Arial, sans-serif;
          font-size:18px;
          font-weight:600;
          color:{DARK_COLOR};
          margin-bottom:2px;
      ">{title}</div>
      <div style="
          font-family:Segoe UI, Arial, sans-serif;
          font-size:12px;
          color:{DARK_COLOR};
          opacity:0.75;
      ">{subtitle}</div>
    </div>
    """

    box = widgets.HBox(
        [
            TEF_LOGO if TEF_LOGO else widgets.HTML(""),
            widgets.HTML(value=title_html),
        ],
        layout=widgets.Layout(
            background_color="white",
            padding="10px 16px",
            border=f"1px solid {PRIMARY_COLOR}",
            border_radius="10px 10px 0 0",
            align_items="center",
        ),
    )
    return box


# ======================================================================
# 10. UI – ALTA DE PROYECTOS (PÁGINA 1)
# ======================================================================

def build_create_form():
    """Formulario de alta de proyectos + dependencias dinámicas."""

    header_box = build_header(
        "Alta de proyectos",
        "Registro de iniciativas y dependencias entre células / trenes / CoE"
    )

    # ---------- Campos base ----------
    w_nombre = widgets.Text(
        description="Proyecto:",
        placeholder="Nombre del proyecto",
        layout=widgets.Layout(width="600px"),
        style={"description_width": "100px"},
    )
    w_estado = widgets.Dropdown(
        options=catalogs.get("estados", []),
        description="Estado:",
        layout=widgets.Layout(width="290px"),
        style={"description_width": "80px"},
    )
    w_prior_q = widgets.Dropdown(
        options=catalogs.get("q_rad", []),
        description="Q Radicado:",
        layout=widgets.Layout(width="290px"),
        style={"description_width": "100px"},
    )
    w_priorizado = widgets.Dropdown(
        options=[("No", "NO"), ("Sí", "SI")],
        description="Priorizado:",
        layout=widgets.Layout(width="200px"),
        style={"description_width": "90px"},
    )
    w_resp = widgets.Combobox(
        options=catalogs.get("responsables", []),
        description="Responsable:",
        placeholder="Selecciona o escribe…",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "100px"},
    )
    w_area = widgets.Combobox(
        options=catalogs.get("areas", []),
        description="Área solicitante:",
        placeholder="Selecciona o escribe…",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "110px"},
    )
    w_area_tren = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Area/Tren/CoE:",
        placeholder="Se rellena al elegir célula (o puedes sobrescribir)",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "110px"},
    )
    w_iniciativa = widgets.Combobox(
        options=catalogs.get("iniciativas", []),
        description="Inic. Estrat.:",
        placeholder="Selecciona iniciativa…",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "110px"},
    )
    w_desc = widgets.Textarea(
        description="Descripción:",
        placeholder="Breve descripción del proyecto…",
        layout=widgets.Layout(width="600px", height="110px"),
        style={"description_width": "100px"},
    )
    w_f_ini = widgets.DatePicker(
        description="Fecha inicio:",
        layout=widgets.Layout(width="260px"),
        style={"description_width": "100px"},
    )
    w_f_fin = widgets.DatePicker(
        description="Fecha cierre:",
        layout=widgets.Layout(width="260px"),
        style={"description_width": "100px"},
    )
    w_linea = widgets.FloatSlider(
        value=0.25, min=0.0, max=1.0, step=0.05,
        description="Línea Base:",
        readout_format=".2f",
        layout=widgets.Layout(width="450px"),
        style={"description_width": "100px"},
    )

    # ---------- Dependencias dinámicas (alta) ----------
    dep_rows      = []
    dep_container = widgets.VBox(
        [],
        layout=widgets.Layout(width="100%")
    )

    def make_dep_row(equipo="", flag="P", descripcion=""):
        cb_equipo = widgets.Combobox(
            options=catalogs.get("celulas_dep", []),
            description="Célula/Tren:",
            placeholder="Célula / Tren / CoE",
            ensure_option=False,
            layout=widgets.Layout(width="360px"),
            style={"description_width": "90px"},
        )
        if equipo:
            cb_equipo.value = equipo

        def on_equipo_change(change):
            if change["name"] == "value":
                new_val = change["new"]
                if new_val:
                    w_area_tren.value = new_val

        cb_equipo.observe(on_equipo_change, names="value")

        dd_flag = widgets.Dropdown(
            options=[("Pendiente (P)", "P"), ("Negociada (L)", "L")],
            description="Estado:",
            layout=widgets.Layout(width="200px"),
            style={"description_width": "70px"},
        )
        dd_flag.value = flag or "P"

        ta_desc = widgets.Textarea(
            placeholder="Descripción de la dependencia…",
            layout=widgets.Layout(width="600px", height="60px"),
        )
        ta_desc.value = descripcion or ""

        btn_del = widgets.Button(
            icon="trash",
            tooltip="Eliminar dependencia",
            layout=widgets.Layout(width="40px"),
            button_style="danger",
        )

        top = widgets.HBox(
            [cb_equipo, dd_flag, btn_del],
            layout=widgets.Layout(width="100%", justify_content="flex-start")
        )
        box = widgets.VBox(
            [top, ta_desc],
            layout=widgets.Layout(
                border=f"1px solid {CARD_BORDER}",
                padding="6px",
                margin="4px 0",
                width="100%",
                background_color="white",
            )
        )

        row_dict = {
            "equipo": cb_equipo,
            "flag": dd_flag,
            "desc": ta_desc,
            "box": box,
            "btn": btn_del,
        }

        def on_del_clicked(b):
            if row_dict in dep_rows:
                dep_rows.remove(row_dict)
                dep_container.children = [r["box"] for r in dep_rows]

        btn_del.on_click(on_del_clicked)
        return row_dict

    def add_dep_row(equipo="", flag="P", descripcion=""):
        row = make_dep_row(equipo, flag, descripcion)
        dep_rows.append(row)
        dep_container.children = [r["box"] for r in dep_rows]

    btn_add_dep = widgets.Button(
        description="+ Agregar dependencia",
        button_style="info",
        icon="plus",
        layout=widgets.Layout(width="220px"),
    )
    btn_add_dep.on_click(lambda b: add_dep_row())

    add_dep_row()

    out = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="90px",
            background_color="white",
        )
    )
    btn_save = widgets.Button(
        description="Guardar en Excel",
        button_style="success",
        icon="save",
        layout=widgets.Layout(width="250px", height="40px", margin="10px 0 0 0"),
    )

    def on_click_guardar(b):
        out.clear_output()
        with out:
            if not w_nombre.value.strip():
                print("⚠️ El nombre del proyecto es obligatorio.")
                return
            if not w_f_ini.value:
                print("⚠️ Debes seleccionar la fecha de inicio.")
                return
            if not w_f_fin.value:
                print("⚠️ Debes seleccionar la fecha de cierre.")
                return
            if w_f_fin.value < w_f_ini.value:
                print("⚠️ La fecha de cierre no puede ser anterior a la de inicio.")
                return

            project = {
                "Q_RADICADO": w_prior_q.value,
                "PRIORIZADO": w_priorizado.value,
                "ESTADO_PROYECTO": w_estado.value,
                "NOMBRE_PROYECTO": w_nombre.value.strip(),
                "DESCRIPCION_PROYECTO": w_desc.value.strip(),
                "RESPONSABLE_PROYECTO": (w_resp.value or "").strip(),
                "AREA_SOLICITANTE": (w_area.value or "").strip(),
                "FECHA_INICIO": w_f_ini.value,
                "FECHA_ESTIMADA_CIERRE": w_f_fin.value,
                "LINEA_BASE": float(w_linea.value),
                "AVANCE": 0.0,
                "ESTIMADO_AVANCE": 1.0,
                "PORC_CUMPLIMIENTO": 0.0,
                "CONTRIBUCION": 0.0,
                "INICIATIVA_ESTRATEGICA": (w_iniciativa.value or "").strip(),
            }

            dep_list = []
            for r in dep_rows:
                eq = r["equipo"].value
                fl = r["flag"].value
                ds = r["desc"].value
                if eq:
                    dep_list.append({
                        "equipo": eq,
                        "codigo": fl,
                        "descripcion": ds,
                    })

            try:
                row, pid = write_project_with_dependencies(project, dep_list)
                print(f"✅ Proyecto guardado en fila {row} con ID {pid}.")
                print(f"   Dependencias registradas: {len(dep_list)}")
            except PermissionError as e:
                print("❌ No se pudo guardar (archivo bloqueado).")
                print("   Cierra el Excel si está abierto.")
                print("   Detalle:", e)
            except Exception as e:
                print("❌ Error inesperado al guardar:", repr(e))

    btn_save.on_click(on_click_guardar)

    box_sup = widgets.VBox(
        [
            widgets.HBox([w_nombre]),
            widgets.HBox([w_estado, w_prior_q, w_priorizado]),
            widgets.HBox([w_resp]),
            widgets.HBox([w_area]),
            widgets.HBox([w_area_tren]),
            widgets.HBox([w_iniciativa]),
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="8px 0",
            background_color="white",
        ),
    )
    box_fecha = widgets.VBox(
        [
            widgets.HBox([w_f_ini, w_f_fin]),
            widgets.HBox([w_linea]),
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="0 0 8px 0",
            background_color="white",
        ),
    )
    box_desc = widgets.VBox(
        [w_desc],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="0 0 8px 0",
            background_color="white",
        ),
    )
    box_dep = widgets.VBox(
        [
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Dependencias (P/L + descripción, por equipo)</b>"
            ),
            btn_add_dep,
            dep_container,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="0 0 8px 0",
            background_color="white",
        ),
    )

    body = widgets.VBox(
        [
            box_sup,
            box_fecha,
            box_desc,
            box_dep,
            widgets.HBox([btn_save]),
            widgets.HTML(
                f"<hr style='margin:10px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            out,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="10px 0"),
    )
    return panel


# ======================================================================
# 11. UI – CONSULTA + EDICIÓN (PÁGINA 2)
# ======================================================================

def build_consult_panel():
    """Panel de consulta, cobertura y edición de proyectos."""

    header_box = build_header(
        "Consulta y edición",
        "Cobertura de dependencias y actualización de avance"
    )

    w_tipo = widgets.ToggleButtons(
        options=[
            ("Por Equipo (Tren/Célula/CoE)", "equipo"),
            ("Por Proyecto (edición)", "proyecto"),
        ],
        description="Ver por:",
        style={"description_width": "80px"},
        layout=widgets.Layout(width="840px"),
    )

    w_equipo = widgets.Combobox(
        options=sorted(catalogs.get("celulas_dep", [])),
        description="Equipo:",
        placeholder="Selecciona Tren / Célula / CoE",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "70px"},
    )
    btn_clear_equipo = widgets.Button(
        icon="times",
        tooltip="Limpiar equipo",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    w_proj = widgets.Combobox(
        options=get_all_project_names(),
        description="Proyecto:",
        placeholder="Escribe / selecciona",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "80px"},
    )
    btn_clear_proj = widgets.Button(
        icon="times",
        tooltip="Limpiar proyecto",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    def on_clear_equipo(b):
        w_equipo.value = ""

    def on_clear_proj(b):
        w_proj.value = ""

    btn_clear_equipo.on_click(on_clear_equipo)
    btn_clear_proj.on_click(on_clear_proj)

    box_equipo = widgets.HBox([w_equipo, btn_clear_equipo])
    box_proy   = widgets.HBox([w_proj, btn_clear_proj])

    def toggle_inputs(change):
        if w_tipo.value == "equipo":
            box_equipo.layout.display = "flex"
            box_proy.layout.display   = "none"
        else:
            box_equipo.layout.display = "none"
            box_proy.layout.display   = "flex"

    w_tipo.observe(toggle_inputs, names="value")
    toggle_inputs(None)

    btn_consultar = widgets.Button(
        description="Consultar",
        button_style="info",
        icon="search",
        layout=widgets.Layout(width="200px", height="35px"),
    )
    btn_limpiar = widgets.Button(
        description="Limpiar filtros",
        button_style="",
        icon="eraser",
        layout=widgets.Layout(width="160px", height="35px", margin="0 0 0 10px"),
    )
    btn_refrescar = widgets.Button(
        description="Refrescar catálogos",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="180px", height="35px", margin="0 0 0 10px"),
    )

    out_resumen = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="230px",
            background_color="white",
        )
    )

    edit_state = {"row": None}

    w_edit_avance = widgets.FloatSlider(
        value=0.0, min=0.0, max=1.0, step=0.05,
        description="Avance:",
        readout_format=".2f",
        layout=widgets.Layout(width="360px"),
        style={"description_width": "80px"},
    )
    w_edit_estimado = widgets.FloatSlider(
        value=1.0, min=0.0, max=1.0, step=0.05,
        description="Estimado:",
        readout_format=".2f",
        layout=widgets.Layout(width="360px"),
        style={"description_width": "80px"},
    )

    dep_rows_edit      = []
    dep_container_edit = widgets.VBox(
        [],
        layout=widgets.Layout(width="100%")
    )

    def make_dep_row_edit(equipo="", flag="P", descripcion=""):
        cb_equipo = widgets.Combobox(
            options=catalogs.get("celulas_dep", []),
            description="Célula/Tren:",
            placeholder="Célula / Tren / CoE",
            ensure_option=False,
            layout=widgets.Layout(width="360px"),
            style={"description_width": "90px"},
        )
        if equipo:
            cb_equipo.value = equipo

        dd_flag = widgets.Dropdown(
            options=[("Pendiente (P)", "P"), ("Negociada (L)", "L")],
            description="Estado:",
            layout=widgets.Layout(width="200px"),
            style={"description_width": "70px"},
        )
        dd_flag.value = flag or "P"

        ta_desc = widgets.Textarea(
            placeholder="Descripción de la dependencia…",
            layout=widgets.Layout(width="600px", height="60px"),
        )
        ta_desc.value = descripcion or ""

        btn_del = widgets.Button(
            icon="trash",
            tooltip="Eliminar dependencia",
            layout=widgets.Layout(width="40px"),
            button_style="danger",
        )

        top = widgets.HBox(
            [cb_equipo, dd_flag, btn_del],
            layout=widgets.Layout(width="100%", justify_content="flex-start")
        )
        box = widgets.VBox(
            [top, ta_desc],
            layout=widgets.Layout(
                border=f"1px solid {CARD_BORDER}",
                padding="6px",
                margin="4px 0",
                width="100%",
                background_color="white",
            )
        )

        row_dict = {
            "equipo": cb_equipo,
            "flag": dd_flag,
            "desc": ta_desc,
            "box": box,
            "btn": btn_del,
        }

        def on_del_clicked(b):
            if row_dict in dep_rows_edit:
                dep_rows_edit.remove(row_dict)
                dep_container_edit.children = [r["box"] for r in dep_rows_edit]

        btn_del.on_click(on_del_clicked)
        return row_dict

    def add_dep_row_edit(equipo="", flag="P", descripcion=""):
        row = make_dep_row_edit(equipo, flag, descripcion)
        dep_rows_edit.append(row)
        dep_container_edit.children = [r["box"] for r in dep_rows_edit]

    btn_add_dep_edit = widgets.Button(
        description="+ Agregar dependencia",
        button_style="info",
        icon="plus",
        layout=widgets.Layout(width="220px"),
    )
    btn_add_dep_edit.on_click(lambda b: add_dep_row_edit())

    out_edit_calc = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="90px",
            background_color="white",
        )
    )
    btn_edit = widgets.Button(
        description="Guardar cambios de proyecto",
        button_style="warning",
        icon="edit",
        layout=widgets.Layout(width="260px", height="35px"),
    )

    def refresh_edit_metrics(linea_base, avance, estimado):
        out_edit_calc.clear_output()
        with out_edit_calc:
            if estimado > 0:
                pct_cumpl = avance/estimado*100
            else:
                pct_cumpl = 0.0
            var_pp = (avance - linea_base)*100
            print(f"Línea base: {linea_base:.2f} ({linea_base*100:.1f}%)")
            print(f"Avance actual: {avance:.2f} ({avance*100:.1f}%)")
            print(f"Variación vs línea base: {var_pp:+.1f} pp")
            print(f"% avance vs estimado: {pct_cumpl:.1f}%")

    def on_consultar_clicked(b):
        out_resumen.clear_output()
        with out_resumen:
            if w_tipo.value == "equipo":
                equipo = w_equipo.value
                edit_state["row"] = None
                dep_rows_edit.clear()
                dep_container_edit.children = []
                if not equipo:
                    print("⚠️ Selecciona un equipo.")
                    return
                res = summarize_by_equipo(equipo)
                if not res["found"]:
                    print("⚠️", res["msg"])
                    return
                print(f"📊 Resumen por equipo: {res['equipo']}")
                print(f"- Total dependencias: {res['total']}")
                print(f"- Pendientes (P):     {res['pendientes']}")
                print(f"- Negociadas (L):     {res['negociadas']}")
                print(f"- % sin negociar:     {res['pct_pendientes']:.1f}%")
                print("\nProyectos:")
                for r in res["rows"]:
                    print(f"  · Fila {r['fila']}: [{r['Q_RADICADO']}] {r['PROYECTO']} → {r['FLAG']}")
            else:
                proyecto = w_proj.value
                if not proyecto:
                    print("⚠️ Selecciona un proyecto.")
                    return
                res = summarize_by_proyecto(proyecto)
                if not res["found"]:
                    print("⚠️", res["msg"])
                    return
                edit_state["row"] = res["fila"]
                print(f"📊 Proyecto: [{res['Q_RADICADO']}] {res['proyecto']}")
                print(f"- Fila hoja:          {res['fila']}")
                print(f"- Total dependencias: {res['total_dep_xl']} (L+P)")
                print(f"- Pendientes (P):     {res['total_P_xl']}")
                print(f"- Negociadas (L):     {res['total_L_xl']}")
                print(f"- Cubrimiento (CQ):   {res['cub_xl']:.2f}  (formato % en Excel)")
                print("\nDependencias actuales:")
                if not res["detalles"]:
                    print("  (Sin dependencias registradas)")
                else:
                    for d in res["detalles"]:
                        print(f"  · {d['equipo']} → {d['FLAG']}")

                w_edit_avance.value   = res["avance"]
                w_edit_estimado.value = res["estimado"]
                refresh_edit_metrics(res["linea_base"], res["avance"], res["estimado"])

                dep_rows_edit.clear()
                dep_container_edit.children = []
                for d in res["detalles"]:
                    add_dep_row_edit(d["equipo"], d["FLAG"], d.get("descripcion", ""))
                add_dep_row_edit()

    btn_consultar.on_click(on_consultar_clicked)

    def on_limpiar_clicked(b):
        w_equipo.value = ""
        w_proj.value   = ""
        out_resumen.clear_output()
        out_edit_calc.clear_output()
        edit_state["row"] = None
        dep_rows_edit.clear()
        dep_container_edit.children = []

    btn_limpiar.on_click(on_limpiar_clicked)

    def on_refrescar_clicked(b):
        try:
            load_catalogs()
        except Exception as e:
            out_resumen.clear_output()
            with out_resumen:
                print("⚠️ Error recargando catálogos:", e)
            return

        w_equipo.options = sorted(catalogs.get("celulas_dep", []))
        w_proj.options   = get_all_project_names()
        w_equipo.value   = ""
        w_proj.value     = ""

        out_resumen.clear_output()
        out_edit_calc.clear_output()
        edit_state["row"] = None
        dep_rows_edit.clear()
        dep_container_edit.children = []

        with out_resumen:
            print("🔄 Catálogos y listas recargados desde Excel.")

    btn_refrescar.on_click(on_refrescar_clicked)

    def on_click_edit(b):
        out_edit_calc.clear_output()
        with out_edit_calc:
            row = edit_state.get("row")
            if not row:
                print("⚠️ Primero consulta un proyecto en modo 'Por Proyecto'.")
                return

            dep_list = []
            for r in dep_rows_edit:
                eq = r["equipo"].value
                fl = r["flag"].value
                ds = r["desc"].value
                if eq:
                    dep_list.append({
                        "equipo": eq,
                        "codigo": fl,
                        "descripcion": ds,
                    })

            metrics = update_project_row_and_dependencies(
                row=row,
                avance=w_edit_avance.value,
                estimado=w_edit_estimado.value,
                dep_list=dep_list,
            )
            print("✅ Proyecto actualizado.")
            print(f"Línea base: {metrics['linea_base']:.2f} ({metrics['linea_base']*100:.1f}%)")
            print(f"Avance: {metrics['avance']:.2f} ({metrics['avance']*100:.1f}%)")
            print(f"Variación vs línea base: {metrics['var_vs_lb_pp']:+.1f} pp")
            print(f"% avance vs estimado: {metrics['pct_cumpl']:.1f}%")
            print(f"Dependencias registradas: {len(dep_list)}")

    btn_edit.on_click(on_click_edit)

    edit_box = widgets.VBox(
        [
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Edición de proyecto seleccionado</b>"
            ),
            widgets.HBox([w_edit_avance, w_edit_estimado]),
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Dependencias (P/L + descripción)</b>"
            ),
            btn_add_dep_edit,
            dep_container_edit,
            widgets.HBox([btn_edit]),
            out_edit_calc,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="10px 0 0 0",
            background_color="white",
        ),
    )

    body = widgets.VBox(
        [
            w_tipo,
            box_equipo,
            box_proy,
            widgets.HBox([btn_consultar, btn_limpiar, btn_refrescar]),
            widgets.HTML(
                f"<hr style='margin:8px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            out_resumen,
            edit_box,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_consulta = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_consulta


# ======================================================================
# 12. UI – MÉTRICAS (PÁGINA 3)
# ======================================================================

def build_metrics_panel():
    """Panel de métricas agregadas con gráficos y filtros."""

    header_box = build_header(
        "Métricas",
        "Visión agregada de proyectos y dependencias por Area/Tren/CoE y Célula"
    )

    w_scope = widgets.ToggleButtons(
        options=[
            ("Todos", "all"),
            ("Por Area/Tren/CoE", "area"),
            ("Por Célula", "celula"),
        ],
        description="Ámbito:",
        style={"description_width": "80px"},
        layout=widgets.Layout(width="840px"),
    )

    w_area = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Area/Tren/CoE:",
        placeholder="Selecciona un Area/Tren/CoE",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "110px"},
    )
    btn_clear_area = widgets.Button(
        icon="times",
        tooltip="Limpiar Area/Tren/CoE",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    w_celula = widgets.Combobox(
        options=catalogs.get("celulas_dep", []),
        description="Célula:",
        placeholder="Selecciona una célula / tren / CoE",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "70px"},
    )
    btn_clear_celula = widgets.Button(
        icon="times",
        tooltip="Limpiar célula",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    def on_clear_area(b):
        w_area.value = ""

    def on_clear_celula(b):
        w_celula.value = ""

    btn_clear_area.on_click(on_clear_area)
    btn_clear_celula.on_click(on_clear_celula)

    box_area   = widgets.HBox([w_area, btn_clear_area])
    box_celula = widgets.HBox([w_celula, btn_clear_celula])

    def toggle_filter_boxes(change):
        if w_scope.value == "all":
            box_area.layout.display   = "none"
            box_celula.layout.display = "none"
        elif w_scope.value == "area":
            box_area.layout.display   = "flex"
            box_celula.layout.display = "none"
        else:
            box_area.layout.display   = "none"
            box_celula.layout.display = "flex"

    w_scope.observe(toggle_filter_boxes, names="value")
    toggle_filter_boxes(None)

    btn_metrics = widgets.Button(
        description="Ver métricas",
        button_style="info",
        icon="bar-chart",
        layout=widgets.Layout(width="200px", height="35px"),
    )
    btn_refresh = widgets.Button(
        description="Refrescar catálogos",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="180px", height="35px", margin="0 0 0 10px"),
    )

    out_text = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="160px",
            background_color="white",
        )
    )

    out_charts = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="420px",
            background_color="white",
            overflow="auto",
        )
    )

    def on_refresh_metrics_catalogs(b):
        try:
            load_catalogs()
        except Exception as e:
            out_text.clear_output()
            with out_text:
                print("⚠️ Error recargando catálogos:", e)
            return

        w_area.options   = catalogs.get("area_tren_coe", [])
        w_celula.options = catalogs.get("celulas_dep", [])
        w_area.value     = ""
        w_celula.value   = ""

        out_text.clear_output()
        out_charts.clear_output()
        with out_text:
            print("🔄 Catálogos de Area/Tren/CoE y Células recargados desde Excel.")

    btn_refresh.on_click(on_refresh_metrics_catalogs)

    def on_click_metrics(b):
        out_text.clear_output()
        out_charts.clear_output()

        scope = w_scope.value
        fval  = None
        label_scope = "Todos los proyectos"

        if scope == "area":
            fval = w_area.value
            if not fval:
                with out_text:
                    print("⚠️ Selecciona un Area/Tren/CoE.")
                return
            label_scope = f"Area/Tren/CoE: {fval}"
        elif scope == "celula":
            fval = w_celula.value
            if not fval:
                with out_text:
                    print("⚠️ Selecciona una Célula / Tren / CoE.")
                return
            label_scope = f"Célula/Tren/CoE (dep): {fval}"

        metrics = compute_metrics(scope=scope, filter_value=fval)

        with out_text:
            print(f"🎯 Ámbito: {label_scope}")
            print(f"- Total de proyectos:            {metrics['total_projects']}")
            print(f"- Nº total de dependencias:      {metrics['total_dep']} (L+P)")
            print(f"- Nº dependencias pendientes P:  {metrics['total_P']}")
            print(f"- Nº dependencias negociadas L:  {metrics['total_L']}")
            print(f"- % cubrimiento (P / total):     {metrics['cobertura_pct']:.1f}%")
            print(f"- Promedio avance proyectos:     {metrics['avg_avance']*100:.1f}%")
            print(f"- Promedio avance priorizados:   {metrics['avg_pri']*100:.1f}%")
            print(f"- Promedio avance no prioriz.:   {metrics['avg_no_pri']*100:.1f}%")
            print(f"- Nº proyectos priorizados:      {metrics['num_pri']}")

        with out_charts:
            if metrics["total_projects"] == 0:
                print("Sin proyectos en el ámbito seleccionado.")
                return

            fig1, ax1 = plt.subplots(figsize=(5, 3))
            labels1 = ["Proyectos", "Dep L", "Dep P"]
            values1 = [
                metrics["total_projects"],
                metrics["total_L"],
                metrics["total_P"],
            ]
            ax1.bar(labels1, values1)
            ax1.set_title("Conteos básicos", fontsize=11)
            ax1.set_ylabel("Cantidad")
            ax1.grid(axis="y", alpha=0.3)
            plt.tight_layout()
            display(fig1)
            plt.close(fig1)

            fig2, ax2 = plt.subplots(figsize=(6, 3))
            labels2 = [
                "Avance global",
                "Avance priorizados",
                "Avance no priorizados",
                "% cubrimiento P",
            ]
            values2 = [
                metrics["avg_avance"] * 100,
                metrics["avg_pri"] * 100,
                metrics["avg_no_pri"] * 100,
                metrics["cobertura_pct"],
            ]
            ax2.bar(labels2, values2)
            ax2.set_title("Promedios y cubrimiento (%)", fontsize=11)
            ax2.set_ylabel("%")
            ax2.set_ylim(0, 110)
            ax2.grid(axis="y", alpha=0.3)
            plt.xticks(rotation=15, ha="right")
            plt.tight_layout()
            display(fig2)
            plt.close(fig2)

    btn_metrics.on_click(on_click_metrics)

    body = widgets.VBox(
        [
            w_scope,
            box_area,
            box_celula,
            widgets.HBox([btn_metrics, btn_refresh]),
            widgets.HTML(
                f"<hr style='margin:8px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Resumen numérico</b>"
            ),
            out_text,
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};margin-top:8px;'>Gráficos</b>"
            ),
            out_charts,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_metrics = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_metrics


# ======================================================================
# 13. MESA DE EXPERTOS – LÓGICA (PENDIENTES, SEMÁFORO, FILTRO TREN)
# ======================================================================

def get_expert_project_list(tren_filter=None):
    """
    Devuelve una lista de proyectos en estado Nuevo / En curso
    con datos para la Mesa de Expertos y Mesa de Alistamiento.
    Incluye lista de dependencias pendientes para semáforo y rating PO Sync.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    name_idx   = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    desc_idx   = column_index_from_string(COLS["DESCRIPCION_PROYECTO"])
    est_idx    = column_index_from_string(COLS["ESTADO_PROYECTO"])
    pri_idx    = column_index_from_string(COLS["PRIORIZADO"])
    cn_idx     = column_index_from_string(COLS["TOTAL_DEP"])
    co_idx     = column_index_from_string(COLS["TOTAL_L"])
    cp_idx     = column_index_from_string(COLS["TOTAL_P"])
    cq_idx     = column_index_from_string(COLS["CUBRIMIENTO_DEP"])
    contrib_idx= column_index_from_string(COLS["CONTRIBUCION"])
    inic_idx   = column_index_from_string(COLS["INICIATIVA_ESTRATEGICA"])
    rating_idx = column_index_from_string(COLS["RATING_PO_SYNC"])

    area_col_idx = find_area_tren_coe_col(ws)

    projects = []

    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        estado = ws.cell(row=row, column=est_idx).value
        if not estado:
            continue
        s = str(estado).strip().lower()
        if not (("nuevo" in s) or ("curso" in s)):
            continue

        if tren_filter and area_col_idx:
            area_val = ws.cell(row=row, column=area_col_idx).value
            if str(area_val).strip() != str(tren_filter).strip():
                continue

        nombre = ws.cell(row=row, column=name_idx).value
        if not nombre:
            continue

        desc = ws.cell(row=row, column=desc_idx).value or ""
        pri  = ws.cell(row=row, column=pri_idx).value or "NO"

        total_dep = to_num_cell(ws.cell(row=row, column=cn_idx).value)
        total_L   = to_num_cell(ws.cell(row=row, column=co_idx).value)
        total_P   = to_num_cell(ws.cell(row=row, column=cp_idx).value)
        cub_raw   = to_num_cell(ws.cell(row=row, column=cq_idx).value)
        cub       = cub_raw*100 if cub_raw <= 1 else cub_raw

        contrib   = to_num_cell(ws.cell(row=row, column=contrib_idx).value)
        inic      = ws.cell(row=row, column=inic_idx).value or ""

        if total_dep > 0:
            cobertura_pct = (total_P / total_dep) * 100.0
        else:
            cobertura_pct = 0.0

        pending_equips = []
        for equipo, desc_header in DEP_MAPPING.items():
            flag_col_idx = find_column_by_header_in_range(
                ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
            )
            if not flag_col_idx:
                continue
            flag = ws.cell(row=row, column=flag_col_idx).value
            if not flag:
                continue
            flag_up = str(flag).strip().upper()
            if flag_up == "P":
                pending_equips.append(equipo)

        rating_raw = ws.cell(row=row, column=rating_idx).value
        rating_po  = int(to_num_cell(rating_raw)) if rating_raw not in (None, "") else 0
        if rating_po < 0:
            rating_po = 0
        if rating_po > 5:
            rating_po = 5

        projects.append({
            "row": row,
            "nombre": str(nombre),
            "descripcion": str(desc),
            "estado": str(estado),
            "priorizado": str(pri),
            "total_dep": total_dep,
            "total_L": total_L,
            "total_P": total_P,
            "cobertura_pct": cobertura_pct,
            "contribucion": contrib,
            "iniciativa": str(inic),
            "pending_equips": pending_equips,
            "rating_po": rating_po,
        })

    return projects

def get_cancelled_detained_summary():
    """
    Texto simple con proyectos Cancelados / Detenidos para mostrar en Accordion.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    name_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    est_idx  = column_index_from_string(COLS["ESTADO_PROYECTO"])

    lines = []
    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        estado = ws.cell(row=row, column=est_idx).value
        if not estado:
            continue
        s = str(estado).strip().lower()
        if ("cancel" in s) or ("detenid" in s):
            nombre = ws.cell(row=row, column=name_idx).value or "(Sin nombre)"
            lines.append(f"- Fila {row}: [{estado}] {nombre}")

    if not lines:
        return "No hay proyectos cancelados o detenidos registrados."
    return "\n".join(lines)

def update_expert_fields(row, priorizado, contribucion, iniciativa):
    """
    Actualiza en la fila:
      - PRIORIZADO (C)
      - CONTRIBUCION (P)
      - INICIATIVA_ESTRATEGICA (Q)
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    pri_idx  = column_index_from_string(COLS["PRIORIZADO"])
    cont_idx = column_index_from_string(COLS["CONTRIBUCION"])
    inic_idx = column_index_from_string(COLS["INICIATIVA_ESTRATEGICA"])

    pri_val = "SI" if str(priorizado).upper().startswith("SI") else "NO"
    ws.cell(row=row, column=pri_idx).value  = pri_val
    ws.cell(row=row, column=cont_idx).value = float(contribucion) if contribucion is not None else 0.0
    ws.cell(row=row, column=inic_idx).value = iniciativa

    wb.save(EXCEL_PATH)

def update_alistamiento_rating(row, rating):
    """
    Actualiza rating de alistamiento (1 a 5) en columna RATING_PO_SYNC.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    rating_idx = column_index_from_string(COLS["RATING_PO_SYNC"])

    r = int(rating) if rating is not None else 0
    if r < 0:
        r = 0
    if r > 5:
        r = 5
    ws.cell(row=row, column=rating_idx).value = r

    wb.save(EXCEL_PATH)


# ======================================================================
# 14. UI – MESA DE EXPERTOS (PÁGINA 4, semáforo grande + pendientes debajo)
# ======================================================================

def build_expert_panel():
    """
    Panel de Mesa de Expertos:
    - Recorre proyectos Nuevo / En curso
    - Muestra semáforo grande de dependencias con la lista de pendientes
      justo debajo
    - Permite marcar priorización, contribución e iniciativa.
    """

    header_box = build_header(
        "Mesa de Expertos",
        "Priorización de iniciativas y análisis de dependencias para líderes y Head of TI"
    )

    expert_state = {
        "projects": [],
        "index": 0,
    }

    w_tren = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Tren:",
        placeholder="Filtrar por Area/Tren/CoE (opcional)",
        ensure_option=False,
        layout=widgets.Layout(width="420px"),
        style={"description_width": "50px"},
    )
    btn_clear_tren = widgets.Button(
        icon="times",
        tooltip="Limpiar Tren",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    def on_clear_tren(b):
        w_tren.value = ""

    btn_clear_tren.on_click(on_clear_tren)
    box_tren = widgets.HBox([w_tren, btn_clear_tren])

    w_counter = widgets.Label(
        value="Proyectos evaluados: 0 / 0",
        layout=widgets.Layout(width="300px"),
    )

    html_proj = widgets.HTML(
        layout=widgets.Layout(width="100%", height="180px")
    )
    html_stats = widgets.HTML(
        layout=widgets.Layout(width="100%", height="120px")
    )

    w_priorizado = widgets.ToggleButtons(
        options=[("No priorizado", "NO"), ("Priorizar (Sí)", "SI")],
        description="Priorizado:",
        style={"description_width": "90px"},
        layout=widgets.Layout(width="360px"),
    )

    w_contrib = widgets.FloatText(
        value=0.0,
        description="Contribución:",
        layout=widgets.Layout(width="260px"),
        style={"description_width": "100px"},
    )

    w_inic_exp = widgets.Combobox(
        options=catalogs.get("iniciativas", []),
        description="Inic. Estrat.:",
        placeholder="Selecciona iniciativa…",
        ensure_option=False,
        layout=widgets.Layout(width="360px"),
        style={"description_width": "100px"},
    )

    btn_save_next = widgets.Button(
        description="Guardar y siguiente",
        button_style="success",
        icon="check",
        layout=widgets.Layout(width="220px", height="35px"),
    )
    btn_skip = widgets.Button(
        description="Saltar",
        button_style="warning",
        icon="forward",
        layout=widgets.Layout(width="120px", height="35px", margin="0 0 0 10px"),
    )
    btn_prev = widgets.Button(
        description="Volver",
        button_style="",
        icon="arrow-left",
        layout=widgets.Layout(width="120px", height="35px", margin="0 0 0 10px"),
    )
    btn_reload = widgets.Button(
        description="Recargar lista",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="160px", height="35px", margin="0 0 0 10px"),
    )

    out_expert = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="80px",
            background_color="white",
        )
    )

    acc_output = widgets.Output(
        layout=widgets.Layout(
            padding="6px",
            background_color="white",
        )
    )
    with acc_output:
        print(get_cancelled_detained_summary())
    accordion = widgets.Accordion(children=[acc_output])
    accordion.set_title(0, "Proyectos cancelados / detenidos (informativo)")

    def render_current():
        out_expert.clear_output()
        projects = expert_state["projects"]
        idx      = expert_state["index"]
        total    = len(projects)

        if total == 0:
            w_counter.value = "Proyectos evaluados: 0 / 0"
            html_proj.value = (
                "<div style='font-family:Segoe UI, Arial; font-size:14px;'>"
                "No hay proyectos en estado Nuevo / En curso para priorizar."
                "</div>"
            )
            html_stats.value = ""
            btn_save_next.disabled = True
            btn_skip.disabled      = True
            btn_prev.disabled      = True
            return

        if idx >= total:
            w_counter.value = f"Proyectos evaluados: {total} / {total}"
            html_proj.value = (
                "<div style='font-family:Segoe UI, Arial; font-size:14px;'>"
                "Has llegado al final de la lista de proyectos.</div>"
            )
            html_stats.value = ""
            btn_save_next.disabled = True
            btn_skip.disabled      = True
            btn_prev.disabled      = False if total > 0 else True
            return

        btn_save_next.disabled = False
        btn_skip.disabled      = False
        btn_prev.disabled      = (idx == 0)

        p = projects[idx]
        w_counter.value = f"Proyectos evaluados: {idx} / {total}"

        sem_color, sem_text = dep_semaforo(p["total_dep"], p["total_L"], p["total_P"])

        pending_list_html = ""
        if p["pending_equips"]:
            items = "".join(f"<li>{eq}</li>" for eq in p["pending_equips"])
            pending_list_html = f"""
            <div style="margin-top:6px;">
              <b>Dependencias pendientes (P):</b>
              <ul style="margin:4px 0 0 18px; padding:0; font-size:12px;">
                {items}
              </ul>
            </div>
            """

        # Semáforo grande + pendientes debajo
        html_proj.value = f"""
        <div style="
            font-family:Segoe UI, Arial;
            font-size:14px;
            color:{DARK_COLOR};
        ">
          <div style="font-size:20px; font-weight:600; margin-bottom:6px;">
            {p['nombre']}
          </div>
          <div style="font-size:14px; margin-bottom:8px; line-height:1.4;">
            {p['descripcion']}
          </div>
          <div style="font-size:13px; color:#333; margin-bottom:4px;">
            Estado actual: <b>{p['estado']}</b> · Priorizado actual: <b>{p['priorizado']}</b>
          </div>
          <div style="font-size:13px; margin-top:4px;">
            <div style="display:flex; align-items:center; gap:8px;">
              <span style="
                  display:inline-block;
                  width:20px;
                  height:20px;
                  border-radius:50%;
                  background-color:{sem_color};
                  border:1px solid #999;
              "></span>
              <span><b>Estado de dependencias:</b> {sem_text}</span>
            </div>
            {pending_list_html}
          </div>
        </div>
        """

        html_stats.value = f"""
        <div style="
            font-family:Segoe UI, Arial;
            font-size:13px;
            color:{DARK_COLOR};
        ">
          <b>Dependencias totales:</b> {p['total_dep']}<br>
          <b>Negociadas (L):</b> {p['total_L']} · <b>Pendientes (P):</b> {p['total_P']}<br>
          <b>% cubrimiento (P / total):</b> {p['cobertura_pct']:.1f}%<br>
          <b>Contribución actual:</b> {p['contribucion']:.2f} ·
          <b>Inic. Estratégica:</b> {p['iniciativa']}
        </div>
        """

        pri_val = str(p["priorizado"]).upper()
        w_priorizado.value = "SI" if pri_val == "SI" else "NO"
        w_contrib.value    = float(p["contribucion"])
        w_inic_exp.value   = p["iniciativa"]

    def reload_projects(b=None):
        expert_state["projects"] = get_expert_project_list(tren_filter=w_tren.value or None)
        expert_state["index"]    = 0
        render_current()
        with out_expert:
            out_expert.clear_output()
            total = len(expert_state["projects"])
            filtro_txt = f" (Tren = {w_tren.value})" if w_tren.value else ""
            print(f"🔄 Lista de proyectos para Mesa de Expertos recargada ({total} proyecto(s)){filtro_txt}.")

    def on_save_next(b):
        out_expert.clear_output()
        projects = expert_state["projects"]
        idx      = expert_state["index"]

        if idx >= len(projects):
            with out_expert:
                print("No hay más proyectos que guardar.")
            return

        p = projects[idx]
        row = p["row"]

        priorizado  = w_priorizado.value
        contrib     = w_contrib.value
        iniciativa  = w_inic_exp.value or ""

        try:
            update_expert_fields(row, priorizado, contrib, iniciativa)
            with out_expert:
                print(
                    f"✅ Proyecto actualizado en fila {row}. "
                    f"Priorizado={priorizado}, Contribución={contrib}, Inic='{iniciativa}'."
                )
        except PermissionError as e:
            with out_expert:
                print("❌ No se pudo guardar (archivo bloqueado). Cierra el Excel si está abierto.")
                print("   Detalle:", e)
            return
        except Exception as e:
            with out_expert:
                print("❌ Error inesperado al guardar:", repr(e))
            return

        expert_state["index"] += 1
        render_current()

    def on_skip(b):
        expert_state["index"] += 1
        render_current()

    def on_prev(b):
        if expert_state["index"] > 0:
            expert_state["index"] -= 1
            render_current()

    btn_save_next.on_click(on_save_next)
    btn_skip.on_click(on_skip)
    btn_prev.on_click(on_prev)
    btn_reload.on_click(reload_projects)

    reload_projects()

    body = widgets.VBox(
        [
            widgets.HBox([box_tren, w_counter]),
            widgets.HTML(
                f"<hr style='margin:4px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            html_proj,
            html_stats,
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Decisión de la Mesa de Expertos</b>"
            ),
            widgets.HBox([w_priorizado, w_contrib]),
            widgets.HBox([w_inic_exp]),
            widgets.HBox([btn_save_next, btn_skip, btn_prev, btn_reload]),
            widgets.HTML(
                f"<hr style='margin:8px 0;border:0;border-top:1px solid {CARD_BORDER};'>"
            ),
            widgets.HTML(
                "<span style='font-size:11px;font-family:Segoe UI, Arial;color:#555;'>"
                "Solo se muestran proyectos en estado <b>Nuevo</b> o <b>En curso</b>. "
                "Los proyectos cancelados/detenidos se consultan abajo como referencia."
                "</span>"
            ),
            accordion,
            out_expert,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_expert = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_expert


# ======================================================================
# 15. UI – MESA DE ALISTAMIENTO (PO Sync) (PÁGINA 5)
#       Semáforo grande + dependencias a la derecha, estrellas como foco
# ======================================================================

def build_alistamiento_panel():
    """
    Mesa de Alistamiento (PO Sync):
    - Filtro por Tren
    - Visual de proyecto en dos columnas:
        izquierda: nombre, descripción, estado
        derecha: semáforo grande + dependencias pendientes (P)
    - Calificación de alistamiento con estrellas (1–5) como elemento central.
    """
    header_box = build_header(
        "Mesa de Alistamiento (PO Sync)",
        "Evaluación de alistamiento en estrellas (1–5) por tren, dependencias y contribución"
    )

    state = {
        "projects": [],
        "index": 0,
    }

    w_tren = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Tren:",
        placeholder="Filtrar por Area/Tren/CoE (opcional)",
        ensure_option=False,
        layout=widgets.Layout(width="420px"),
        style={"description_width": "50px"},
    )
    btn_clear_tren = widgets.Button(
        icon="times",
        tooltip="Limpiar Tren",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    def on_clear_tren(b):
        w_tren.value = ""

    btn_clear_tren.on_click(on_clear_tren)
    box_tren = widgets.HBox([w_tren, btn_clear_tren])

    w_counter = widgets.Label(
        value="Proyectos calificados: 0 / 0",
        layout=widgets.Layout(width="300px"),
    )

    html_proj = widgets.HTML(
        layout=widgets.Layout(width="100%", height="180px")
    )
    html_stats = widgets.HTML(
        layout=widgets.Layout(width="100%", height="110px")
    )

    w_stars = widgets.ToggleButtons(
        options=[
            ("★☆☆☆☆", 1),
            ("★★☆☆☆", 2),
            ("★★★☆☆", 3),
            ("★★★★☆", 4),
            ("★★★★★", 5),
        ],
        description="Alistamiento:",
        style={"description_width": "100px"},
        layout=widgets.Layout(width="420px"),
    )

    btn_save_next = widgets.Button(
        description="Guardar rating y siguiente",
        button_style="success",
        icon="star",
        layout=widgets.Layout(width="260px", height="35px"),
    )
    btn_skip = widgets.Button(
        description="Saltar",
        button_style="warning",
        icon="forward",
        layout=widgets.Layout(width="120px", height="35px", margin="0 0 0 10px"),
    )
    btn_prev = widgets.Button(
        description="Volver",
        button_style="",
        icon="arrow-left",
        layout=widgets.Layout(width="120px", height="35px", margin="0 0 0 10px"),
    )
    btn_reload = widgets.Button(
        description="Recargar lista",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="160px", height="35px", margin="0 0 0 10px"),
    )

    out_alist = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="80px",
            background_color="white",
        )
    )

    def render_current():
        out_alist.clear_output()
        projects = state["projects"]
        idx      = state["index"]
        total    = len(projects)

        if total == 0:
            w_counter.value = "Proyectos calificados: 0 / 0"
            html_proj.value = (
                "<div style='font-family:Segoe UI, Arial; font-size:14px;'>"
                "No hay proyectos en estado Nuevo / En curso para alistamiento."
                "</div>"
            )
            html_stats.value = ""
            btn_save_next.disabled = True
            btn_skip.disabled      = True
            btn_prev.disabled      = True
            return

        if idx >= total:
            w_counter.value = f"Proyectos calificados: {total} / {total}"
            html_proj.value = (
                "<div style='font-family:Segoe UI, Arial; font-size:14px;'>"
                "Has llegado al final de la lista de proyectos.</div>"
            )
            html_stats.value = ""
            btn_save_next.disabled = True
            btn_skip.disabled      = True
            btn_prev.disabled      = False if total > 0 else True
            return

        btn_save_next.disabled = False
        btn_skip.disabled      = False
        btn_prev.disabled      = (idx == 0)

        p = projects[idx]
        w_counter.value = f"Proyectos calificados: {idx} / {total}"

        sem_color, sem_text = dep_semaforo(p["total_dep"], p["total_L"], p["total_P"])

        pending_list_html = ""
        if p["pending_equips"]:
            items = "".join(f"<li>{eq}</li>" for eq in p["pending_equips"])
            pending_list_html = f"""
                <div style="margin-top:6px; font-size:12px;">
                  <b>Pendientes (P):</b>
                  <ul style="margin:4px 0 0 18px; padding:0; font-size:12px;">
                    {items}
                  </ul>
                </div>
            """

        # Layout 2 columnas: izquierda (texto), derecha (semáforo grande + pendientes)
        html_proj.value = f"""
        <div style="
            font-family:Segoe UI, Arial;
            font-size:14px;
            color:{DARK_COLOR};
            display:flex;
            gap:18px;
        ">
          <div style="flex:2;">
            <div style="font-size:20px; font-weight:600; margin-bottom:6px;">
              {p['nombre']}
            </div>
            <div style="font-size:14px; margin-bottom:8px; line-height:1.4;">
              {p['descripcion']}
            </div>
            <div style="font-size:13px; color:#333; margin-bottom:4px;">
              Estado actual: <b>{p['estado']}</b> · Priorizado: <b>{p['priorizado']}</b>
            </div>
          </div>
          <div style="
              flex:1;
              border-left:1px solid {CARD_BORDER};
              padding-left:10px;
              font-size:12px;
          ">
            <div style="font-weight:600; margin-bottom:4px;">Dependencias</div>
            <div style="display:flex; align-items:center; gap:8px; margin-bottom:4px;">
              <span style="
                  display:inline-block;
                  width:22px;
                  height:22px;
                  border-radius:50%;
                  background-color:{sem_color};
                  border:1px solid #999;
              "></span>
              <span>{sem_text}</span>
            </div>
            {pending_list_html}
          </div>
        </div>
        """

        html_stats.value = f"""
        <div style="
            font-family:Segoe UI, Arial;
            font-size:13px;
            color:{DARK_COLOR};
        ">
          <b>Dependencias totales:</b> {p['total_dep']}<br>
          <b>Negociadas (L):</b> {p['total_L']} · <b>Pendientes (P):</b> {p['total_P']}<br>
          <b>% cubrimiento (P / total):</b> {p['cobertura_pct']:.1f}%<br>
          <b>Contribución:</b> {p['contribucion']:.2f} ·
          <b>Inic. Estratégica:</b> {p['iniciativa']}
        </div>
        """

        # rating actual
        rating_po = p.get("rating_po", 0)
        if rating_po < 1 or rating_po > 5:
            rating_po = 3
        w_stars.value = rating_po

    def reload_projects(b=None):
        state["projects"] = get_expert_project_list(tren_filter=w_tren.value or None)
        state["index"]    = 0
        render_current()
        with out_alist:
            out_alist.clear_output()
            total = len(state["projects"])
            filtro_txt = f" (Tren = {w_tren.value})" if w_tren.value else ""
            print(f"🔄 Lista de proyectos para Mesa de Alistamiento recargada ({total} proyecto(s)){filtro_txt}.")

    def on_save_next(b):
        out_alist.clear_output()
        projects = state["projects"]
        idx      = state["index"]

        if idx >= len(projects):
            with out_alist:
                print("No hay más proyectos que guardar.")
            return

        p = projects[idx]
        row = p["row"]
        rating = w_stars.value

        try:
            update_alistamiento_rating(row, rating)
            with out_alist:
                print(
                    f"✅ Rating de alistamiento actualizado en fila {row}. "
                    f"Estrellas = {rating}."
                )
        except PermissionError as e:
            with out_alist:
                print("❌ No se pudo guardar (archivo bloqueado). Cierra el Excel si está abierto.")
                print("   Detalle:", e)
            return
        except Exception as e:
            with out_alist:
                print("❌ Error inesperado al guardar rating:", repr(e))
            return

        state["index"] += 1
        render_current()

    def on_skip(b):
        state["index"] += 1
        render_current()

    def on_prev(b):
        if state["index"] > 0:
            state["index"] -= 1
            render_current()

    btn_save_next.on_click(on_save_next)
    btn_skip.on_click(on_skip)
    btn_prev.on_click(on_prev)
    btn_reload.on_click(reload_projects)

    reload_projects()

    body = widgets.VBox(
        [
            widgets.HBox([box_tren, w_counter]),
            widgets.HTML(
                f"<hr style='margin:4px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            html_proj,
            html_stats,
            widgets.HTML(
                f"<div style='font-family:Segoe UI, Arial; font-size:13px; color:{DARK_COLOR}; margin:4px 0;'>"
                f"<b>Calificación de alistamiento (1–5 estrellas)</b></div>"
            ),
            widgets.HBox([w_stars]),
            widgets.HBox([btn_save_next, btn_skip, btn_prev, btn_reload]),
            out_alist,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_alist = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_alist


# ======================================================================
# 16. FEEDBACK – HOJA SUGERENCIAS (PÁGINA 6)
# ======================================================================

def append_suggestion(usuario, sugerencia):
    """
    Añade una sugerencia a la hoja 'Sugerencias':
    Columnas: Usuario (A), Sugerencia (B)
    """
    wb = load_workbook()
    ws = get_ws_sugerencias(wb)

    next_row = ws.max_row + 1
    ws.cell(row=next_row, column=1).value = usuario
    ws.cell(row=next_row, column=2).value = sugerencia

    wb.save(EXCEL_PATH)

def build_feedback_panel():
    """Página de feedback: login sencillo (nombre) + sugerencia."""

    header_box = build_header(
        "Feedback y sugerencias",
        "Comparte aprendizajes, fricciones y mejoras sobre la herramienta de Gestión de Dependencias TI"
    )

    w_user = widgets.Text(
        description="Nombre:",
        placeholder="Tu nombre / rol",
        layout=widgets.Layout(width="400px"),
        style={"description_width": "70px"},
    )
    w_sugg = widgets.Textarea(
        description="Sugerencia:",
        placeholder="Escribe aquí tu feedback, ideas de mejora, bugs, etc.",
        layout=widgets.Layout(width="650px", height="140px"),
        style={"description_width": "80px"},
    )

    btn_send = widgets.Button(
        description="Enviar feedback",
        button_style="success",
        icon="paper-plane",
        layout=widgets.Layout(width="200px", height="35px", margin="6px 0 0 0"),
    )

    out_feedback = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="90px",
            background_color="white",
        )
    )

    def on_send_clicked(b):
        out_feedback.clear_output()
        with out_feedback:
            usuario = w_user.value.strip()
            suger   = w_sugg.value.strip()
            if not usuario:
                print("⚠️ Por favor indica tu nombre.")
                return
            if not suger:
                print("⚠️ La sugerencia está vacía.")
                return
            try:
                append_suggestion(usuario, suger)
                print(f"✅ Gracias, {usuario}. Tu feedback fue registrado en la hoja 'Sugerencias'.")
                w_sugg.value = ""
            except PermissionError as e:
                print("❌ No se pudo guardar la sugerencia (archivo bloqueado).")
                print("   Cierra el Excel si está abierto.")
                print("   Detalle:", e)
            except Exception as e:
                print("❌ Error inesperado al guardar la sugerencia:", repr(e))

    btn_send.on_click(on_send_clicked)

    body = widgets.VBox(
        [
            widgets.HTML(
                "<div style='font-family:Segoe UI, Arial; font-size:13px; color:#333;'>"
                "Usaremos estas sugerencias para ajustar tanto la UI del notebook "
                "como la estructura de la base GD_v1. "
                "</div>"
            ),
            widgets.HTML("<br>"),
            widgets.HBox([w_user]),
            widgets.HBox([w_sugg]),
            widgets.HBox([btn_send]),
            out_feedback,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_feedback = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_feedback


# ======================================================================
# 17. APP SHELL – TABS (ALTA / CONSULTA / MÉTRICAS / MESA EXP / ALIST / FEEDBACK)
# ======================================================================

form_panel      = build_create_form()
consulta_panel  = build_consult_panel()
metrics_panel   = build_metrics_panel()
expert_panel    = build_expert_panel()
alist_panel     = build_alistamiento_panel()
feedback_panel  = build_feedback_panel()

tabs = widgets.Tab(
    children=[
        form_panel,
        consulta_panel,
        metrics_panel,
        expert_panel,
        alist_panel,
        feedback_panel,
    ],
    layout=widgets.Layout(width="930px"),
)
tabs.set_title(0, "Alta de proyectos")
tabs.set_title(1, "Consulta y edición")
tabs.set_title(2, "Métricas")
tabs.set_title(3, "Mesa de Expertos")
tabs.set_title(4, "Mesa de Alistamiento")
tabs.set_title(5, "Feedback")

app_shell = widgets.VBox(
    [
        widgets.HTML(
            f"""
            <div style="
                background: linear-gradient(90deg, {DARK_COLOR}, {PRIMARY_COLOR});
                color:white;
                padding:8px 16px;
                border-radius:10px 10px 0 0;
                font-family:Segoe UI, Arial, sans-serif;
                font-size:13px;
                margin-bottom:4px;
            ">
              Gestión de Dependencias TI · Cofre GD_v1 (Notebook Python + Excel)
            </div>
            """
        ),
        tabs,
    ],
    layout=widgets.Layout(
        width="950px",
        margin="10px 0 30px 0",
    ),
)

display(app_shell)


In [20]:
# ======================================================================
# GD_v1 · Gestión de Dependencias TI  (Notebook Python + Excel)
# Índice de capítulos
# ======================================================================
# 0.  Instalación (comentado)
# 1.  Imports y configuración general
# 2.  Funciones básicas para Excel + utilidades
# 3.  Carga de catálogos (hoja Datos) y mapping de dependencias / trenes
# 4.  Núcleo dependencias: agregados, semáforo y widgets de rating
# 5.  Alta de proyectos (UI + escritura en Excel)
# 6.  Consultas / resúmenes
# 7.  Actualizar proyecto existente (avance + dependencias)
# 8.  Cálculo de métricas agregadas (para pestaña Métricas)
# 9.  UI – Header corporativo Telefónica
# 10. UI – Alta de proyectos (pestaña 1)
# 11. UI – Consulta + edición (pestaña 2)
# 12. UI – Métricas (pestaña 3)
# 13. Lógica de Mesa de Expertos / Alistamiento (lectura + update Excel)
# 14. UI – Mesa de Expertos (pestaña 4)
# 15. UI – Mesa de Alistamiento (PO Sync) (pestaña 5)
# 16. Feedback – hoja Sugerencias (pestaña 6)
# 17. App Shell – Tabs principales
# ======================================================================

# ======================================================================
# 0. INSTALACIÓN (si hace falta, solo una vez por entorno)
# ======================================================================
# !pip install openpyxl ipywidgets matplotlib

# ======================================================================
# 1. IMPORTS Y CONFIGURACIÓN GENERAL
# ======================================================================
import openpyxl
from openpyxl.utils import column_index_from_string, get_column_letter
from pathlib import Path
from typing import Optional   # 👈 añadido para Optional[str]
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# --- RUTA DEL EXCEL (MAC) ---
EXCEL_PATH = Path("/Users/macuser/Desktop/DIR COMERCIAL/GD/GD_v1.xlsx")

# --- NOMBRES DE HOJAS ---
SHEET_PROYECTOS = "ProyectosTI"
SHEET_DATOS     = "Datos"
SHEET_SUG       = "Sugerencias"

# --- FILAS / CABECERAS ---
START_ROW_PROYECTOS   = 12   # primera fila de datos
HEADER_ROW_PROYECTOS  = 11   # fila de cabecera por defecto

# --- COLUMNAS BASE (hoja ProyectosTI) ---
COLS = {
    "ID": "A",
    "Q_RADICADO": "B",
    "PRIORIZADO": "C",
    "ESTADO_PROYECTO": "D",
    "NOMBRE_PROYECTO": "E",
    "DESCRIPCION_PROYECTO": "F",
    "RESPONSABLE_PROYECTO": "G",
    "AREA_SOLICITANTE": "H",
    "FECHA_INICIO": "I",
    "FECHA_ESTIMADA_CIERRE": "J",
    "LINEA_BASE": "K",
    "LINEA_BASE_Q_GESTION": "L",
    "AVANCE": "M",
    "ESTIMADO_AVANCE": "N",
    "PORC_CUMPLIMIENTO": "O",
    "CONTRIBUCION": "P",              # valor numérico Mesa de Expertos
    "INICIATIVA_ESTRATEGICA": "Q",    # catálogo desde Datos
    # Agregados de dependencias:
    "TOTAL_DEP": "CN",        # L + P
    "TOTAL_L": "CO",          # solo L
    "TOTAL_P": "CP",          # solo P
    "CUBRIMIENTO_DEP": "CQ",  # P / (L+P)  → % de P sobre el total
    # Rating de Mesa de Alistamiento (PO Sync):
    "RATING_PO_SYNC": "CR",   # 1 a 5 estrellas
}

# --- RANGOS DE DEPENDENCIAS (FLAGS y DESCRIPCIONES) ---
FLAG_START_LETTER = "R"
FLAG_END_LETTER   = "BB"
DESC_START_LETTER = "BC"
DESC_END_LETTER   = "CM"

FLAG_START_COL = column_index_from_string(FLAG_START_LETTER)
FLAG_END_COL   = column_index_from_string(FLAG_END_LETTER)
DESC_START_COL = column_index_from_string(DESC_START_LETTER)
DESC_END_COL   = column_index_from_string(DESC_END_LETTER)

# --- VARIABLES GLOBALES ---
DEP_MAPPING       = {}   # {celula/tren/coe -> header descripción}
catalogs          = {}   # catálogos para combos
CELULA_TREN_MAP   = {}   # {celula -> Area/Tren/CoE} desde hoja Datos (E–F)

# --- ESTILO CORPORATIVO TELEFÓNICA ---
PRIMARY_COLOR = "#00a9e0"
DARK_COLOR    = "#001b3c"
LIGHT_BG      = "#f5f9fc"
CARD_BORDER   = "#d0d7de"

# --- LOGO TELEFÓNICA ---
TEF_LOGO = None
try:
    with open("/Users/macuser/Desktop/DIR COMERCIAL/GD/Telefonica logo.png", "rb") as f:
        TEF_LOGO = widgets.Image(
            value=f.read(),
            format="png",
            layout=widgets.Layout(width="80px", height="auto", margin="0 12px 0 0"),
        )
except Exception as e:
    print("⚠️ No se pudo cargar el logo de Telefónica:", e)


# ======================================================================
# 2. FUNCIONES BÁSICAS PARA EXCEL + UTILIDADES
# ======================================================================

def load_workbook():
    """Abre el libro de Excel (.xlsx)."""
    if not EXCEL_PATH.exists():
        raise FileNotFoundError(f"No se encontró el archivo: {EXCEL_PATH}")
    wb = openpyxl.load_workbook(EXCEL_PATH, keep_vba=False)
    if SHEET_PROYECTOS not in wb.sheetnames:
        raise KeyError(f"No existe la hoja '{SHEET_PROYECTOS}'.")
    return wb

def get_ws_proyectos(wb=None):
    wb = wb or load_workbook()
    return wb[SHEET_PROYECTOS]

def get_ws_datos(wb=None):
    wb = wb or load_workbook()
    if SHEET_DATOS not in wb.sheetnames:
        raise KeyError(f"No existe la hoja '{SHEET_DATOS}'.")
    return wb[SHEET_DATOS]

def get_ws_sugerencias(wb=None):
    """
    Devuelve la hoja 'Sugerencias'. Si no existe, la crea con cabecera.
    """
    wb = wb or load_workbook()
    if SHEET_SUG not in wb.sheetnames:
        ws = wb.create_sheet(SHEET_SUG)
        ws["A1"] = "Usuario"
        ws["B1"] = "Sugerencia"
    else:
        ws = wb[SHEET_SUG]
        if ws.max_row == 1 and ws["A1"].value is None:
            ws["A1"] = "Usuario"
            ws["B1"] = "Sugerencia"
    return ws

def get_header_row_proyectos(ws):
    """
    Detecta la fila de cabeceras buscando el texto 'ID'
    en la columna COLS['ID'].
    """
    id_col_idx = column_index_from_string(COLS["ID"])
    for r in range(1, START_ROW_PROYECTOS):
        v = ws.cell(row=r, column=id_col_idx).value
        if isinstance(v, str) and v.strip().lower() == "id":
            return r
    return HEADER_ROW_PROYECTOS

def get_unique_list_from_column(ws, col_letter, start_row=2):
    """Devuelve una lista ordenada sin duplicados de una columna."""
    values = set()
    col_idx = column_index_from_string(col_letter)
    for row in range(start_row, ws.max_row + 1):
        value = ws.cell(row=row, column=col_idx).value
        if value not in (None, ""):
            values.add(str(value))
    return sorted(values)

def get_next_row_and_id(ws, id_col_letter="A", start_row=START_ROW_PROYECTOS):
    """
    Busca la siguiente fila libre y el siguiente ID numérico.
    """
    id_col_idx = column_index_from_string(id_col_letter)
    max_row_used = 0
    max_id_found = 0

    for row in range(start_row, ws.max_row + 1):
        val = ws.cell(row=row, column=id_col_idx).value
        if val not in (None, ""):
            max_row_used = row
            if isinstance(val, (int, float)):
                max_id_found = max(max_id_found, int(val))

    next_row = start_row if max_row_used == 0 else max_row_used + 1
    next_id  = max_id_found + 1 if max_id_found > 0 else 1
    return next_row, next_id

def find_column_by_header(ws, header_name, header_row=1):
    """Busca una columna por el texto exacto del header (case-insensitive)."""
    if not header_name:
        return None
    target = str(header_name).strip().lower()
    for col_idx in range(1, ws.max_column + 1):
        val = ws.cell(row=header_row, column=col_idx).value
        if val is None:
            continue
        if str(val).strip().lower() == target:
            return col_idx
    return None

def find_column_by_header_in_range(ws, header_name, start_col_idx, end_col_idx, header_row):
    """Busca una columna por header, restringida a un rango de columnas."""
    if not header_name:
        return None
    target = str(header_name).strip().lower()
    for col_idx in range(start_col_idx, end_col_idx + 1):
        val = ws.cell(row=header_row, column=col_idx).value
        if val is None:
            continue
        if str(val).strip().lower() == target:
            return col_idx
    return None

def find_area_tren_coe_col(ws):
    """
    Intenta localizar la columna de Area/Tren/CoE en la hoja ProyectosTI,
    buscando palabras clave en la fila de cabeceras.
    """
    header_row = get_header_row_proyectos(ws)
    candidate_idx = None
    for col_idx in range(1, ws.max_column + 1):
        val = ws.cell(row=header_row, column=col_idx).value
        if not val:
            continue
        s = str(val).strip().lower()
        if "tren" in s and "coe" in s:
            return col_idx
        if s in ("area tren coe", "area/tren/coe"):
            candidate_idx = col_idx
    return candidate_idx

def to_num_cell(v):
    """Convierte cualquier valor de celda a float, tolerando texto, %, comas, etc."""
    if v is None or v == "":
        return 0.0
    if isinstance(v, (int, float)):
        return float(v)
    s = str(v).strip()
    if s == "":
        return 0.0
    s = s.replace("%", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return 0.0


# ======================================================================
# 3. CARGA DE CATÁLOGOS Y MAPPING DE DEPENDENCIAS / TRENES (HOJA DATOS)
# ======================================================================

def load_dependency_mapping(wb=None):
    """
    Lee de la hoja 'Datos':
      - 'Celula Dependencia'              → header de flag (ej. 'CÉLULA TASADORES')
      - 'Celula Descripcion Dependencia' → header de descripción (ej. 'DESCRIPCION CÉLULA TASADORES')
    y arma: { 'CÉLULA TASADORES': 'DESCRIPCION CÉLULA TASADORES', ... }
    """
    wb = wb or load_workbook()
    ws_d = get_ws_datos(wb)
    col_cel_dep_idx  = find_column_by_header(ws_d, "Celula Dependencia", header_row=1)
    col_desc_dep_idx = find_column_by_header(ws_d, "Celula Descripcion Dependencia", header_row=1)
    mapping = {}
    if not col_cel_dep_idx or not col_desc_dep_idx:
        return mapping

    for row in range(2, ws_d.max_row + 1):
        cel_name   = ws_d.cell(row=row, column=col_cel_dep_idx).value
        desc_header= ws_d.cell(row=row, column=col_desc_dep_idx).value
        if cel_name and desc_header:
            mapping[str(cel_name).strip()] = str(desc_header).strip()
    return mapping

def load_catalogs():
    """
    Carga catálogos y mappings desde 'Datos'.

    - Estados, Q radicado, responsables, áreas.
    - Relación Area/Tren/CoE (col. E) ↔ Célula (col. F) → CELULA_TREN_MAP.
    - DEP_MAPPING: mapea celula → columna de descripción en ProyectosTI.
    """
    global DEP_MAPPING, catalogs, CELULA_TREN_MAP

    wb   = load_workbook()
    ws_d = get_ws_datos(wb)

    estados_list       = get_unique_list_from_column(ws_d, "A")
    priorizacion_list  = get_unique_list_from_column(ws_d, "B")
    responsables_list  = get_unique_list_from_column(ws_d, "C")
    areas_list         = get_unique_list_from_column(ws_d, "D")

    # --- Mapa Area/Tren/CoE (E) ↔ Célula (F) ---
    CELULA_TREN_MAP = {}
    col_tren_idx = column_index_from_string("E")
    col_cel_idx  = column_index_from_string("F")
    for row in range(2, ws_d.max_row + 1):
        tren_val = ws_d.cell(row=row, column=col_tren_idx).value
        cel_val  = ws_d.cell(row=row, column=col_cel_idx).value
        if tren_val and cel_val:
            CELULA_TREN_MAP[str(cel_val).strip()] = str(tren_val).strip()

    area_tren_coe_list = sorted({v for v in CELULA_TREN_MAP.values()})
    celulas_dep_list   = sorted({k for k in CELULA_TREN_MAP.keys()})

    # Iniciativas Estratégicas
    iniciativas_list = []
    col_ini_idx = find_column_by_header(ws_d, "Iniciativa Estrategica", header_row=1)
    if col_ini_idx:
        col_ini_letter = get_column_letter(col_ini_idx)
        iniciativas_list = get_unique_list_from_column(ws_d, col_ini_letter)
    else:
        iniciativas_list = []

    # Mapping de dependencias (celula → cabecera descripción)
    DEP_MAPPING = load_dependency_mapping(wb)
    if DEP_MAPPING:
        celulas_dep_from_mapping = sorted(DEP_MAPPING.keys())
        celulas_dep_list = sorted(set(celulas_dep_list) | set(celulas_dep_from_mapping))

    catalogs = {
        "estados": estados_list,
        "q_rad": priorizacion_list,
        "responsables": responsables_list,
        "areas": areas_list,
        "area_tren_coe": area_tren_coe_list,
        "celulas_dep": celulas_dep_list,
        "iniciativas": iniciativas_list,
    }

# Cargamos catálogos al arrancar
try:
    load_catalogs()
except Exception as e:
    print("⚠️ No se pudieron cargar catálogos desde 'Datos'. Motivo:", e)
    catalogs = {
        "estados": ["Nuevo", "En curso", "Detenido", "Cancelado", "Finalizado"],
        "q_rad": ["1Q/25", "2Q/25"],
        "responsables": ["Responsable 1"],
        "areas": ["Área 1"],
        "area_tren_coe": ["Tren X"],
        "celulas_dep": ["Célula A"],
        "iniciativas": ["Inic. 1"],
    }
    DEP_MAPPING = {}
    CELULA_TREN_MAP = {}


# ======================================================================
# 4. NÚCLEO DEPENDENCIAS: AGREGADOS, SEMÁFORO, WIDGET ESTRELLAS
# ======================================================================

def compute_dep_aggregates(dep_list):
    """
    A partir de dep_list calcula:
      - total_dep = L+P
      - total_L
      - total_P
      - cubrimiento = P / (L+P)  (0 si no hay dependencias)
    """
    flags = [(d.get("codigo") or "").strip().upper()
             for d in dep_list
             if (d.get("equipo") or "").strip()]
    flags = [f for f in flags if f in ("L", "P")]
    total_dep = len(flags)
    total_L   = sum(1 for f in flags if f == "L")
    total_P   = sum(1 for f in flags if f == "P")
    if total_dep > 0:
        cubrimiento = total_P / total_dep  # % de P sobre el total
    else:
        cubrimiento = 0.0
    return total_dep, total_L, total_P, cubrimiento


def write_dep_aggregates(ws, row, dep_list):
    """
    Escribe en CN–CQ:
      - CN: total L+P
      - CO: solo L
      - CP: solo P
      - CQ: P / (L+P)   (para formatear como % en Excel)
    """
    total_dep, total_L, total_P, cub = compute_dep_aggregates(dep_list)
    cn = column_index_from_string(COLS["TOTAL_DEP"])
    co = column_index_from_string(COLS["TOTAL_L"])
    cp = column_index_from_string(COLS["TOTAL_P"])
    cq = column_index_from_string(COLS["CUBRIMIENTO_DEP"])

    ws.cell(row=row, column=cn).value = total_dep
    ws.cell(row=row, column=co).value = total_L
    ws.cell(row=row, column=cp).value = total_P
    ws.cell(row=row, column=cq).value = cub


def apply_dependencies_to_row(ws, row, dep_list):
    """
    Aplica un conjunto de dependencias dinámicas en la fila `row`:
      dep_list = [{equipo, codigo(P/L), descripcion}, ...]
    Usa DEP_MAPPING + cabeceras detectadas dinámicamente.
    """
    header_row = get_header_row_proyectos(ws)

    # 1) Limpiar flags y descripciones existentes
    for equipo, desc_header in DEP_MAPPING.items():
        flag_col_idx = find_column_by_header_in_range(
            ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
        )
        if flag_col_idx:
            ws.cell(row=row, column=flag_col_idx).value = None

        if desc_header:
            desc_col_idx = find_column_by_header_in_range(
                ws, desc_header, DESC_START_COL, DESC_END_COL, header_row
            )
            if desc_col_idx:
                ws.cell(row=row, column=desc_col_idx).value = None

    # 2) Escribir nuevas dependencias
    for dep in dep_list:
        equipo = (dep.get("equipo") or "").strip()
        codigo = (dep.get("codigo") or "").strip().upper()
        texto  = (dep.get("descripcion") or "").strip()

        if not equipo or codigo not in ("P", "L"):
            continue

        desc_header = DEP_MAPPING.get(equipo)

        flag_col_idx = find_column_by_header_in_range(
            ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
        )
        if flag_col_idx:
            ws.cell(row=row, column=flag_col_idx).value = codigo

        if desc_header:
            desc_col_idx = find_column_by_header_in_range(
                ws, desc_header, DESC_START_COL, DESC_END_COL, header_row
            )
            if desc_col_idx and texto:
                ws.cell(row=row, column=desc_col_idx).value = texto

    # 3) Escribir agregados en CN–CQ
    write_dep_aggregates(ws, row, dep_list)


def dep_semaforo(total_dep, total_L, total_P):
    """
    Devuelve (color_hex, texto) para el semáforo de dependencias.
    """
    if total_dep == 0:
        return "#bdc3c7", "Sin dependencias registradas"
    if total_P == 0 and total_L > 0:
        return "#2ecc71", "Todas las dependencias negociadas (L)"
    if total_L == 0 and total_P > 0:
        return "#e74c3c", "Todas las dependencias pendientes (P)"
    return "#f1c40f", "Mix de dependencias negociadas (L) y pendientes (P)"


def build_semaforo_block(total_dep, total_L, total_P, title="Dependencias"):
    """
    Devuelve HTML con un semáforo completo (3 luces verticales: roja,
    amarilla y verde). Solo una se enciende; las demás quedan en gris.
    Texto ligeramente más grande para mejor lectura.
    """
    _color, text = dep_semaforo(total_dep, total_L, total_P)

    if total_dep == 0:
        state = "apagado"
    elif total_P == 0 and total_L > 0:
        state = "verde"
    elif total_L == 0 and total_P > 0:
        state = "rojo"
    else:
        state = "amarillo"

    def light(color_hex, active):
        fill = color_hex if active else "#e0e0e0"
        return f"""
        <div style="
            width:22px;
            height:22px;
            border-radius:50%;
            background-color:{fill};
            margin:4px auto;
            border:1px solid #999;
        "></div>
        """

    red_light    = light("#e74c3c", state == "rojo")
    yellow_light = light("#f1c40f", state == "amarillo")
    green_light  = light("#2ecc71", state == "verde")

    html = f"""
    <div style="
        font-family:Segoe UI, Arial;
        font-size:13px;
        color:{DARK_COLOR};
        text-align:center;
    ">
      <div style="font-weight:600; margin-bottom:8px;">{title}</div>
      <div style="
          width:60px;
          margin:0 auto 8px auto;
          padding:8px 0;
          border-radius:30px;
          background-color:white;
          border:1px solid {CARD_BORDER};
      ">
        {red_light}
        {yellow_light}
        {green_light}
      </div>
      <div style="font-size:12px; line-height:1.4; max-width:220px; margin:0 auto;">
        {text}
      </div>
    </div>
    """
    return html


def create_star_rating_widget(initial=3):
    """
    Crea un control de rating de 1–5 estrellas:
      - Sin fondo gris del widget.
      - Estrellas encendidas en amarillo según el valor.
    Devuelve: (widget, set_rating(int), get_rating()).
    """
    state = {"value": int(initial)}
    buttons = []

    def update():
        for idx, btn in enumerate(buttons, start=1):
            if idx <= state["value"]:
                btn.style.button_color = "#f1c40f"  # amarillo
            else:
                btn.style.button_color = "#eeeeee"  # gris claro

    def make_button(i):
        btn = widgets.Button(
            description="★",
            layout=widgets.Layout(
                width="32px",
                height="32px",
                padding="0",
                margin="0 2px 0 0",
            ),
        )
        btn.style.button_color = "#eeeeee"

        def on_click(b):
            state["value"] = i
            update()

        btn.on_click(on_click)
        buttons.append(btn)
        return btn

    stars = [make_button(i) for i in range(1, 6)]
    update()

    container = widgets.HBox(
        stars,
        layout=widgets.Layout(align_items="center"),
    )

    def set_rating(value):
        try:
            v = int(value)
        except Exception:
            v = 3
        if v < 1:
            v = 1
        if v > 5:
            v = 5
        state["value"] = v
        update()

    def get_rating():
        return state["value"]

    return container, set_rating, get_rating




# ======================================================================
# 5. ALTA DE PROYECTOS (NUEVO REGISTRO + DEPENDENCIAS)
# ======================================================================

def write_project_with_dependencies(project, dep_list):
    """
    Inserta un nuevo proyecto y aplica sus dependencias + métricas CN–CQ.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    next_row, next_id = get_next_row_and_id(
        ws, id_col_letter=COLS["ID"], start_row=START_ROW_PROYECTOS
    )

    project = project.copy()
    project["ID"] = next_id

    for field, col_letter in COLS.items():
        if field in project:
            col_idx = column_index_from_string(col_letter)
            ws.cell(row=next_row, column=col_idx).value = project.get(field)

    # Dependencias + agregados
    apply_dependencies_to_row(ws, next_row, dep_list)

    wb.save(EXCEL_PATH)
    return next_row, next_id


# ======================================================================
# 6. CONSULTAS / RESÚMENES
# ======================================================================

def get_all_project_names():
    """Devuelve la lista de proyectos para el combo de consulta."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    name_col_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    names = set()
    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        val = ws.cell(row=row, column=name_col_idx).value
        if val not in (None, ""):
            names.add(str(val).strip())
    return sorted(names)

def summarize_by_equipo(equipo_name):
    """Resumen de cobertura de dependencias para un equipo."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    col_flag_idx = find_column_by_header_in_range(
        ws, equipo_name, FLAG_START_COL, FLAG_END_COL, header_row
    )
    if not col_flag_idx:
        return {"found": False, "msg": f"No se encontró la columna '{equipo_name}' en R:BB."}

    name_col_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    q_col_idx    = column_index_from_string(COLS["Q_RADICADO"])

    total = pendientes = negociadas = 0
    rows = []

    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        flag = ws.cell(row=row, column=col_flag_idx).value
        if flag is None or str(flag).strip() == "":
            continue
        flag_up = str(flag).strip().upper()
        if flag_up not in ("P", "L"):
            continue

        total += 1
        if flag_up == "P":
            pendientes += 1
        else:
            negociadas += 1

        nombre = ws.cell(row=row, column=name_col_idx).value
        qrad   = ws.cell(row=row, column=q_col_idx).value
        rows.append({"fila": row, "Q_RADICADO": qrad, "PROYECTO": nombre, "FLAG": flag_up})

    pct_pend = (pendientes/total*100) if total > 0 else 0.0
    return {
        "found": True,
        "equipo": equipo_name,
        "total": total,
        "pendientes": pendientes,
        "negociadas": negociadas,
        "pct_pendientes": pct_pend,
        "rows": rows,
    }

def summarize_by_proyecto(nombre_proyecto):
    """Resumen completo de un proyecto (incluye dependencias y agregados)."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    name_col_idx = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    q_col_idx    = column_index_from_string(COLS["Q_RADICADO"])

    target_row = None
    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        val = ws.cell(row=row, column=name_col_idx).value
        if val and str(val).strip() == nombre_proyecto:
            target_row = row
            break

    if not target_row:
        return {"found": False, "msg": f"No se encontró el proyecto '{nombre_proyecto}'."}

    detalles = []

    for equipo, desc_header in DEP_MAPPING.items():
        flag_col_idx = find_column_by_header_in_range(
            ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
        )
        if not flag_col_idx:
            continue

        flag = ws.cell(row=target_row, column=flag_col_idx).value
        if flag is None or str(flag).strip() == "":
            continue

        flag_up = str(flag).strip().upper()
        if flag_up not in ("P", "L"):
            continue

        desc = ""
        if desc_header:
            desc_col_idx = find_column_by_header_in_range(
                ws, desc_header, DESC_START_COL, DESC_END_COL, header_row
            )
            if desc_col_idx:
                desc = ws.cell(row=target_row, column=desc_col_idx).value or ""

        detalles.append({"equipo": equipo, "FLAG": flag_up, "descripcion": desc})

    total      = len(detalles)
    pendientes = sum(1 for d in detalles if d["FLAG"] == "P")
    negociadas = sum(1 for d in detalles if d["FLAG"] == "L")
    pct_pend   = (pendientes/total*100) if total > 0 else 0.0

    lb_col   = column_index_from_string(COLS["LINEA_BASE"])
    av_col   = column_index_from_string(COLS["AVANCE"])
    est_col  = column_index_from_string(COLS["ESTIMADO_AVANCE"])

    lb  = to_num_cell(ws.cell(row=target_row, column=lb_col).value)
    av  = to_num_cell(ws.cell(row=target_row, column=av_col).value)
    est = to_num_cell(ws.cell(row=target_row, column=est_col).value)
    qrad= ws.cell(row=target_row, column=q_col_idx).value

    cn = column_index_from_string(COLS["TOTAL_DEP"])
    co = column_index_from_string(COLS["TOTAL_L"])
    cp = column_index_from_string(COLS["TOTAL_P"])
    cq = column_index_from_string(COLS["CUBRIMIENTO_DEP"])

    total_dep_xl = to_num_cell(ws.cell(row=target_row, column=cn).value)
    total_L_xl   = to_num_cell(ws.cell(row=target_row, column=co).value)
    total_P_xl   = to_num_cell(ws.cell(row=target_row, column=cp).value)
    cub_xl       = to_num_cell(ws.cell(row=target_row, column=cq).value)

    return {
        "found": True,
        "fila": target_row,
        "proyecto": nombre_proyecto,
        "Q_RADICADO": qrad,
        "total_dep": total,
        "pendientes": pendientes,
        "negociadas": negociadas,
        "pct_pendientes": pct_pend,
        "detalles": detalles,
        "linea_base": float(lb),
        "avance": float(av),
        "estimado": float(est),
        "total_dep_xl": total_dep_xl,
        "total_L_xl": total_L_xl,
        "total_P_xl": total_P_xl,
        "cub_xl": float(cub_xl),
    }


# ======================================================================
# 7. ACTUALIZAR PROYECTO EXISTENTE (AVANCE + DEPENDENCIAS)
# ======================================================================

def update_project_row_and_dependencies(row, avance, estimado, dep_list):
    """
    Actualiza:
      - AVANCE, ESTIMADO_AVANCE, PORC_CUMPLIMIENTO
      - Dependencias P/L + descripciones + CN–CQ
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)

    lb_col  = column_index_from_string(COLS["LINEA_BASE"])
    av_col  = column_index_from_string(COLS["AVANCE"])
    est_col = column_index_from_string(COLS["ESTIMADO_AVANCE"])
    pct_col = column_index_from_string(COLS["PORC_CUMPLIMIENTO"])

    linea_base = to_num_cell(ws.cell(row=row, column=lb_col).value)
    old_av     = to_num_cell(ws.cell(row=row, column=av_col).value)
    old_es     = to_num_cell(ws.cell(row=row, column=est_col).value)

    new_av = float(avance)  if avance  is not None else float(old_av)
    new_es = float(estimado)if estimado is not None else float(old_es)

    ws.cell(row=row, column=av_col).value  = new_av
    ws.cell(row=row, column=est_col).value = new_es

    if new_es > 0:
        pct_cumpl = new_av / new_es
    else:
        pct_cumpl = 0.0
    ws.cell(row=row, column=pct_col).value = pct_cumpl

    apply_dependencies_to_row(ws, row, dep_list)

    wb.save(EXCEL_PATH)

    var_vs_lb = new_av - float(linea_base)
    return {
        "linea_base": float(linea_base),
        "avance": new_av,
        "estimado": new_es,
        "pct_cumpl": pct_cumpl * 100,
        "var_vs_lb_pp": var_vs_lb * 100,
    }


# ======================================================================
# 8. CÁLCULO DE MÉTRICAS AGREGADAS (para la página de Métricas)
# ======================================================================

def compute_metrics(scope="all", filter_value=None):
    """
    scope:
      - "all"    → todos los proyectos
      - "area"   → filtrar por Area/Tren/CoE (usando relación Datos E–F)
      - "celula" → filtrar por una célula (según flag P/L en columna R:BB)
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    name_col_idx   = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    prior_col_idx  = column_index_from_string(COLS["PRIORIZADO"])
    av_col_idx     = column_index_from_string(COLS["AVANCE"])
    cn_idx         = column_index_from_string(COLS["TOTAL_DEP"])
    co_idx         = column_index_from_string(COLS["TOTAL_L"])
    cp_idx         = column_index_from_string(COLS["TOTAL_P"])

    # Filtrado por célula (directo a columna de flags)
    cel_flag_col_idx = None
    if scope == "celula" and filter_value:
        cel_flag_col_idx = find_column_by_header_in_range(
            ws, filter_value, FLAG_START_COL, FLAG_END_COL, header_row
        )

    total_projects       = 0
    total_dep            = 0.0
    total_L              = 0.0
    total_P              = 0.0
    sum_avance           = 0.0
    pri_count            = 0
    pri_avance_sum       = 0.0
    no_pri_count         = 0
    no_pri_avance_sum    = 0.0

    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        name = ws.cell(row=row, column=name_col_idx).value
        if not name:
            continue

        # --- Filtro por Tren usando la relación Area/Tren/CoE ↔ Célula ---
        if scope == "area" and filter_value:
            tren_match = False
            for equipo in DEP_MAPPING.keys():
                flag_col_idx = find_column_by_header_in_range(
                    ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
                )
                if not flag_col_idx:
                    continue
                flag_val = ws.cell(row=row, column=flag_col_idx).value
                if not flag_val:
                    continue
                flag_up = str(flag_val).strip().upper()
                if flag_up not in ("P", "L"):
                    continue
                tren_val = CELULA_TREN_MAP.get(equipo)
                if tren_val and str(tren_val).strip() == str(filter_value).strip():
                    tren_match = True
                    break
            if not tren_match:
                continue

        # --- Filtro por célula concreta ---
        if scope == "celula" and filter_value:
            if not cel_flag_col_idx:
                continue
            flag_val = ws.cell(row=row, column=cel_flag_col_idx).value
            if not flag_val or str(flag_val).strip().upper() not in ("P", "L"):
                continue

        total_projects += 1

        dep_val = to_num_cell(ws.cell(row=row, column=cn_idx).value)
        L_val   = to_num_cell(ws.cell(row=row, column=co_idx).value)
        P_val   = to_num_cell(ws.cell(row=row, column=cp_idx).value)

        total_dep += dep_val
        total_L   += L_val
        total_P   += P_val

        av_val = to_num_cell(ws.cell(row=row, column=av_col_idx).value)
        sum_avance += av_val

        pri_val = ws.cell(row=row, column=prior_col_idx).value
        pri_str = str(pri_val).strip().upper() if pri_val not in (None, "") else ""

        if pri_str == "SI":
            pri_count      += 1
            pri_avance_sum += av_val
        else:
            no_pri_count      += 1
            no_pri_avance_sum += av_val

    if total_projects > 0:
        avg_avance = sum_avance / total_projects
    else:
        avg_avance = 0.0

    if pri_count > 0:
        avg_pri = pri_avance_sum / pri_count
    else:
        avg_pri = 0.0

    if no_pri_count > 0:
        avg_no_pri = no_pri_avance_sum / no_pri_count
    else:
        avg_no_pri = 0.0

    if total_dep > 0:
        cobertura_pct = (total_P / total_dep) * 100.0
    else:
        cobertura_pct = 0.0

    return {
        "total_projects": int(total_projects),
        "total_dep": float(total_dep),
        "total_L": float(total_L),
        "total_P": float(total_P),
        "cobertura_pct": float(cobertura_pct),
        "avg_avance": float(avg_avance),
        "avg_pri": float(avg_pri),
        "avg_no_pri": float(avg_no_pri),
        "num_pri": int(pri_count),
    }


# ======================================================================
# 9. UI – COMPONENTES COMUNES (HEADER TELEFÓNICA)
# ======================================================================

def build_header(title, subtitle=""):
    """Header corporativo Telefónica con logo + título, azul oscuro sobre fondo claro."""
    title_html = f"""
    <div style="
        display:flex;
        flex-direction:column;
        justify-content:center;
    ">
      <div style="
          font-family:Segoe UI, Arial, sans-serif;
          font-size:18px;
          font-weight:600;
          color:{DARK_COLOR};
          margin-bottom:2px;
      ">{title}</div>
      <div style="
          font-family:Segoe UI, Arial, sans-serif;
          font-size:12px;
          color:{DARK_COLOR};
          opacity:0.75;
      ">{subtitle}</div>
    </div>
    """

    box = widgets.HBox(
        [
            TEF_LOGO if TEF_LOGO else widgets.HTML(""),
            widgets.HTML(value=title_html),
        ],
        layout=widgets.Layout(
            background_color="white",
            padding="10px 16px",
            border=f"1px solid {PRIMARY_COLOR}",
            border_radius="10px 10px 0 0",
            align_items="center",
        ),
    )
    return box


# ======================================================================
# 10. UI – ALTA DE PROYECTOS (PÁGINA 1)
# ======================================================================

def build_create_form():
    """Formulario de alta de proyectos + dependencias dinámicas."""

    header_box = build_header(
        "Alta de proyectos",
        "Registro de iniciativas y dependencias entre células / trenes / CoE"
    )

    # ---------- Campos base ----------
    w_nombre = widgets.Text(
        description="Proyecto:",
        placeholder="Nombre del proyecto",
        layout=widgets.Layout(width="600px"),
        style={"description_width": "100px"},
    )
    w_estado = widgets.Dropdown(
        options=catalogs.get("estados", []),
        description="Estado:",
        layout=widgets.Layout(width="290px"),
        style={"description_width": "80px"},
    )
    w_prior_q = widgets.Dropdown(
        options=catalogs.get("q_rad", []),
        description="Q Radicado:",
        layout=widgets.Layout(width="290px"),
        style={"description_width": "100px"},
    )
    w_priorizado = widgets.Dropdown(
        options=[("No", "NO"), ("Sí", "SI")],
        description="Priorizado:",
        layout=widgets.Layout(width="200px"),
        style={"description_width": "90px"},
    )
    w_resp = widgets.Combobox(
        options=catalogs.get("responsables", []),
        description="Responsable:",
        placeholder="Selecciona o escribe…",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "100px"},
    )
    w_area = widgets.Combobox(
        options=catalogs.get("areas", []),
        description="Área solicitante:",
        placeholder="Selecciona o escribe…",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "110px"},
    )

    # Tren detectado automáticamente a partir de las células seleccionadas.
    w_area_tren = widgets.HTML(
        value=(
            f"<div style='font-family:Segoe UI, Arial; font-size:12px; color:{DARK_COLOR};'>"
            "<b>Tren detectado:</b> (sin selección)</div>"
        ),
        layout=widgets.Layout(width="400px"),
    )

    w_iniciativa = widgets.Combobox(
        options=catalogs.get("iniciativas", []),
        description="Inic. Estrat.:",
        placeholder="Selecciona iniciativa…",
        ensure_option=False,
        layout=widgets.Layout(width="400px"),
        style={"description_width": "110px"},
    )
    w_desc = widgets.Textarea(
        description="Descripción:",
        placeholder="Breve descripción del proyecto…",
        layout=widgets.Layout(width="600px", height="110px"),
        style={"description_width": "100px"},
    )
    w_f_ini = widgets.DatePicker(
        description="Fecha inicio:",
        layout=widgets.Layout(width="260px"),
        style={"description_width": "100px"},
    )
    w_f_fin = widgets.DatePicker(
        description="Fecha cierre:",
        layout=widgets.Layout(width="260px"),
        style={"description_width": "100px"},
    )
    w_linea = widgets.FloatSlider(
        value=0.25, min=0.0, max=1.0, step=0.05,
        description="Línea Base:",
        readout_format=".2f",
        layout=widgets.Layout(width="450px"),
        style={"description_width": "100px"},
    )

    # ---------- Dependencias dinámicas (alta) ----------
    dep_rows      = []
    dep_container = widgets.VBox(
        [],
        layout=widgets.Layout(width="100%")
    )
    tren_state = {"trenes": set()}

    def recompute_tren_label():
        """
        Recalcula el/los tren(es) asociados a las células de dependencias,
        usando CELULA_TREN_MAP (hoja Datos, columnas E–F).

        Si una célula no tiene mapping, se usa el nombre de la célula
        como fallback para mostrar algo al usuario.
        """
        trenes = set()
        for r in dep_rows:
            eq_val = r["equipo"].value
            if not eq_val:
                continue
            tren_val = CELULA_TREN_MAP.get(eq_val)
            if tren_val:
                trenes.add(tren_val)
            else:
                # Fallback: mostrar la propia célula/equipo
                trenes.add(eq_val)

        tren_state["trenes"] = trenes

        if not trenes:
            txt = "(sin selección)"
        elif len(trenes) == 1:
            txt = next(iter(trenes))
        else:
            txt = ", ".join(sorted(trenes))

        w_area_tren.value = (
            f"<div style='font-family:Segoe UI, Arial; font-size:12px; color:{DARK_COLOR};'>"
            f"<b>Tren detectado:</b> {txt}</div>"
        )

    def make_dep_row(equipo="", flag="P", descripcion=""):
        cb_equipo = widgets.Combobox(
            options=catalogs.get("celulas_dep", []),
            description="Célula/Tren:",
            placeholder="Célula / Tren / CoE",
            ensure_option=False,
            layout=widgets.Layout(width="360px"),
            style={"description_width": "90px"},
        )
        if equipo:
            cb_equipo.value = equipo

        def on_equipo_change(change):
            if change["name"] == "value":
                recompute_tren_label()

        cb_equipo.observe(on_equipo_change, names="value")

        dd_flag = widgets.Dropdown(
            options=[("Pendiente (P)", "P"), ("Negociada (L)", "L")],
            description="Estado:",
            layout=widgets.Layout(width="200px"),
            style={"description_width": "70px"},
        )
        dd_flag.value = flag or "P"

        ta_desc = widgets.Textarea(
            placeholder="Descripción de la dependencia…",
            layout=widgets.Layout(width="600px", height="60px"),
        )
        ta_desc.value = descripcion or ""

        btn_del = widgets.Button(
            icon="trash",
            tooltip="Eliminar dependencia",
            layout=widgets.Layout(width="40px"),
            button_style="danger",
        )

        top = widgets.HBox(
            [cb_equipo, dd_flag, btn_del],
            layout=widgets.Layout(width="100%", justify_content="flex-start")
        )
        box = widgets.VBox(
            [top, ta_desc],
            layout=widgets.Layout(
                border=f"1px solid {CARD_BORDER}",
                padding="6px",
                margin="4px 0",
                width="100%",
                background_color="white",
            )
        )

        row_dict = {
            "equipo": cb_equipo,
            "flag": dd_flag,
            "desc": ta_desc,
            "box": box,
            "btn": btn_del,
        }

        def on_del_clicked(b):
            if row_dict in dep_rows:
                dep_rows.remove(row_dict)
                dep_container.children = [r["box"] for r in dep_rows]
                recompute_tren_label()

        btn_del.on_click(on_del_clicked)
        return row_dict

    def add_dep_row(equipo="", flag="P", descripcion=""):
        row = make_dep_row(equipo, flag, descripcion)
        dep_rows.append(row        )
        dep_container.children = [r["box"] for r in dep_rows]
        recompute_tren_label()

    btn_add_dep = widgets.Button(
        description="+ Agregar dependencia",
        button_style="info",
        icon="plus",
        layout=widgets.Layout(width="220px"),
    )
    btn_add_dep.on_click(lambda b: add_dep_row())

    # Primer registro vacío
    add_dep_row()

    out = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="90px",
            background_color="white",
            overflow_y="auto",
        )
    )
    btn_save = widgets.Button(
        description="Guardar en Excel",
        button_style="success",
        icon="save",
        layout=widgets.Layout(width="250px", height="40px", margin="10px 0 0 0"),
    )

    def _reset_form():
        """Limpia todos los campos tras un guardado exitoso."""
        w_nombre.value = ""
        w_estado.value = catalogs.get("estados", [None])[0] if catalogs.get("estados") else None
        w_prior_q.value = catalogs.get("q_rad", [None])[0] if catalogs.get("q_rad") else None
        w_priorizado.value = "NO"
        w_resp.value = ""
        w_area.value = ""
        w_iniciativa.value = ""
        w_desc.value = ""
        w_f_ini.value = None
        w_f_fin.value = None
        w_linea.value = 0.25

        # Reset dependencias: dejamos solo una fila vacía
        dep_rows.clear()
        dep_container.children = []
        add_dep_row()

        w_area_tren.value = (
            f"<div style='font-family:Segoe UI, Arial; font-size:12px; color:{DARK_COLOR};'>"
            "<b>Tren detectado:</b> (sin selección)</div>"
        )

    def on_click_guardar(b):
        out.clear_output()
        with out:
            if not w_nombre.value.strip():
                print("⚠️ El nombre del proyecto es obligatorio.")
                return
            if not w_f_ini.value:
                print("⚠️ Debes seleccionar la fecha de inicio.")
                return
            if not w_f_fin.value:
                print("⚠️ Debes seleccionar la fecha de cierre.")
                return
            if w_f_fin.value < w_f_ini.value:
                print("⚠️ La fecha de cierre no puede ser anterior a la de inicio.")
                return

            project = {
                "Q_RADICADO": w_prior_q.value,
                "PRIORIZADO": w_priorizado.value,
                "ESTADO_PROYECTO": w_estado.value,
                "NOMBRE_PROYECTO": w_nombre.value.strip(),
                "DESCRIPCION_PROYECTO": w_desc.value.strip(),
                "RESPONSABLE_PROYECTO": (w_resp.value or "").strip(),
                "AREA_SOLICITANTE": (w_area.value or "").strip(),
                "FECHA_INICIO": w_f_ini.value,
                "FECHA_ESTIMADA_CIERRE": w_f_fin.value,
                "LINEA_BASE": float(w_linea.value),
                "AVANCE": 0.0,
                "ESTIMADO_AVANCE": 1.0,
                "PORC_CUMPLIMIENTO": 0.0,
                "CONTRIBUCION": 0.0,
                "INICIATIVA_ESTRATEGICA": (w_iniciativa.value or "").strip(),
            }

            dep_list = []
            for r in dep_rows:
                eq = r["equipo"].value
                fl = r["flag"].value
                ds = r["desc"].value
                if eq:
                    dep_list.append({
                        "equipo": eq,
                        "codigo": fl,
                        "descripcion": ds,
                    })

            if not dep_list:
                print("⚠️ Debes registrar al menos una dependencia (P o L) para el proyecto.")
                return

            try:
                row, pid = write_project_with_dependencies(project, dep_list)
                print(f"✅ Proyecto guardado en fila {row} con ID {pid}.")
                print(f"   Dependencias registradas: {len(dep_list)}")
                _reset_form()
            except PermissionError as e:
                print("❌ No se pudo guardar (archivo bloqueado).")
                print("   Cierra el Excel si está abierto.")
                print("   Detalle:", e)
            except Exception as e:
                print("❌ Error inesperado al guardar:", repr(e))

    btn_save.on_click(on_click_guardar)

    box_sup = widgets.VBox(
        [
            widgets.HBox([w_nombre]),
            widgets.HBox([w_estado, w_prior_q, w_priorizado]),
            widgets.HBox([w_resp]),
            widgets.HBox([w_area]),
            widgets.HBox([w_area_tren]),
            widgets.HBox([w_iniciativa]),
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="8px 0",
            background_color="white",
        ),
    )
    box_fecha = widgets.VBox(
        [
            widgets.HBox([w_f_ini, w_f_fin]),
            widgets.HBox([w_linea]),
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="0 0 8px 0",
            background_color="white",
        ),
    )
    box_desc = widgets.VBox(
        [w_desc],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="0 0 8px 0",
            background_color="white",
        ),
    )
    box_dep = widgets.VBox(
        [
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Dependencias (P/L + descripción, por equipo)</b>"
            ),
            btn_add_dep,
            dep_container,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="0 0 8px 0",
            background_color="white",
        ),
    )

    body = widgets.VBox(
        [
            box_sup,
            box_fecha,
            box_desc,
            box_dep,
            widgets.HBox([btn_save]),
            widgets.HTML(
                f"<hr style='margin:10px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            out,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="10px 0"),
    )
    return panel






# ======================================================================
# 11. UI – CONSULTA + EDICIÓN (PÁGINA 2)
# ======================================================================

def build_consult_panel():
    """Panel de consulta, cobertura y edición de proyectos."""

    header_box = build_header(
        "Consulta y edición",
        "Cobertura de dependencias y actualización de avance"
    )

    w_tipo = widgets.ToggleButtons(
        options=[
            ("Por Equipo (Tren/Célula/CoE)", "equipo"),
            ("Por Proyecto (edición)", "proyecto"),
        ],
        description="Ver por:",
        style={"description_width": "80px"},
        layout=widgets.Layout(width="840px"),
    )

    w_equipo = widgets.Combobox(
        options=sorted(catalogs.get("celulas_dep", [])),
        description="Equipo:",
        placeholder="Selecciona Tren / Célula / CoE",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "70px"},
    )
    btn_clear_equipo = widgets.Button(
        icon="times",
        tooltip="Limpiar equipo",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    w_proj = widgets.Combobox(
        options=get_all_project_names(),
        description="Proyecto:",
        placeholder="Escribe / selecciona",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "80px"},
    )
    btn_clear_proj = widgets.Button(
        icon="times",
        tooltip="Limpiar proyecto",
        layout=widgets.Layout(width="40px", height="30px", margin="0 0 0 6px"),
    )

    def on_clear_equipo(b):
        w_equipo.value = ""

    def on_clear_proj(b):
        w_proj.value = ""

    btn_clear_equipo.on_click(on_clear_equipo)
    btn_clear_proj.on_click(on_clear_proj)

    box_equipo = widgets.HBox([w_equipo, btn_clear_equipo])
    box_proy   = widgets.HBox([w_proj, btn_clear_proj])

    def toggle_inputs(change):
        if w_tipo.value == "equipo":
            box_equipo.layout.display = "flex"
            box_proy.layout.display   = "none"
        else:
            box_equipo.layout.display = "none"
            box_proy.layout.display   = "flex"

    w_tipo.observe(toggle_inputs, names="value")
    toggle_inputs(None)

    btn_consultar = widgets.Button(
        description="Consultar",
        button_style="info",
        icon="search",
        layout=widgets.Layout(width="200px", height="35px"),
    )
    btn_limpiar = widgets.Button(
        description="Limpiar filtros",
        button_style="",
        icon="eraser",
        layout=widgets.Layout(width="160px", height="35px", margin="0 0 0 10px"),
    )
    btn_refrescar = widgets.Button(
        description="Refrescar catálogos",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="180px", height="35px", margin="0 0 0 10px"),
    )

    out_resumen = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="230px",
            background_color="white",
        )
    )

    edit_state = {"row": None}

    w_edit_avance = widgets.FloatSlider(
        value=0.0, min=0.0, max=1.0, step=0.05,
        description="Avance:",
        readout_format=".2f",
        layout=widgets.Layout(width="360px"),
        style={"description_width": "80px"},
    )
    w_edit_estimado = widgets.FloatSlider(
        value=1.0, min=0.0, max=1.0, step=0.05,
        description="Estimado:",
        readout_format=".2f",
        layout=widgets.Layout(width="360px"),
        style={"description_width": "80px"},
    )

    dep_rows_edit      = []
    dep_container_edit = widgets.VBox(
        [],
        layout=widgets.Layout(width="100%")
    )

    def make_dep_row_edit(equipo="", flag="P", descripcion=""):
        cb_equipo = widgets.Combobox(
            options=catalogs.get("celulas_dep", []),
            description="Célula/Tren:",
            placeholder="Célula / Tren / CoE",
            ensure_option=False,
            layout=widgets.Layout(width="360px"),
            style={"description_width": "90px"},
        )
        if equipo:
            cb_equipo.value = equipo

        dd_flag = widgets.Dropdown(
            options=[("Pendiente (P)", "P"), ("Negociada (L)", "L")],
            description="Estado:",
            layout=widgets.Layout(width="200px"),
            style={"description_width": "70px"},
        )
        dd_flag.value = flag or "P"

        ta_desc = widgets.Textarea(
            placeholder="Descripción de la dependencia…",
            layout=widgets.Layout(width="600px", height="60px"),
        )
        ta_desc.value = descripcion or ""

        btn_del = widgets.Button(
            icon="trash",
            tooltip="Eliminar dependencia",
            layout=widgets.Layout(width="40px"),
            button_style="danger",
        )

        top = widgets.HBox(
            [cb_equipo, dd_flag, btn_del],
            layout=widgets.Layout(width="100%", justify_content="flex-start")
        )
        box = widgets.VBox(
            [top, ta_desc],
            layout=widgets.Layout(
                border=f"1px solid {CARD_BORDER}",
                padding="6px",
                margin="4px 0",
                width="100%",
                background_color="white",
            )
        )

        row_dict = {
            "equipo": cb_equipo,
            "flag": dd_flag,
            "desc": ta_desc,
            "box": box,
            "btn": btn_del,
        }

        def on_del_clicked(b):
            if row_dict in dep_rows_edit:
                dep_rows_edit.remove(row_dict)
                dep_container_edit.children = [r["box"] for r in dep_rows_edit]

        btn_del.on_click(on_del_clicked)
        return row_dict

    def add_dep_row_edit(equipo="", flag="P", descripcion=""):
        row = make_dep_row_edit(equipo, flag, descripcion)
        dep_rows_edit.append(row)
        dep_container_edit.children = [r["box"] for r in dep_rows_edit]

    btn_add_dep_edit = widgets.Button(
        description="+ Agregar dependencia",
        button_style="info",
        icon="plus",
        layout=widgets.Layout(width="220px"),
    )
    btn_add_dep_edit.on_click(lambda b: add_dep_row_edit())

    out_edit_calc = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="90px",
            background_color="white",
        )
    )
    btn_edit = widgets.Button(
        description="Guardar cambios de proyecto",
        button_style="warning",
        icon="edit",
        layout=widgets.Layout(width="260px", height="35px"),
    )

    def refresh_edit_metrics(linea_base, avance, estimado):
        out_edit_calc.clear_output()
        with out_edit_calc:
            if estimado > 0:
                pct_cumpl = avance/estimado*100
            else:
                pct_cumpl = 0.0
            var_pp = (avance - linea_base)*100
            print(f"Línea base: {linea_base:.2f} ({linea_base*100:.1f}%)")
            print(f"Avance actual: {avance:.2f} ({avance*100:.1f}%)")
            print(f"Variación vs línea base: {var_pp:+.1f} pp")
            print(f"% avance vs estimado: {pct_cumpl:.1f}%")

    def on_consultar_clicked(b):
        out_resumen.clear_output()
        with out_resumen:
            if w_tipo.value == "equipo":
                equipo = w_equipo.value
                edit_state["row"] = None
                dep_rows_edit.clear()
                dep_container_edit.children = []
                if not equipo:
                    print("⚠️ Selecciona un equipo.")
                    return
                res = summarize_by_equipo(equipo)
                if not res["found"]:
                    print("⚠️", res["msg"])
                    return
                print(f"📊 Resumen por equipo: {res['equipo']}")
                print(f"- Total dependencias: {res['total']}")
                print(f"- Pendientes (P):     {res['pendientes']}")
                print(f"- Negociadas (L):     {res['negociadas']}")
                print(f"- % sin negociar:     {res['pct_pendientes']:.1f}%")
                print("\nProyectos:")
                for r in res["rows"]:
                    print(f"  · Fila {r['fila']}: [{r['Q_RADICADO']}] {r['PROYECTO']} → {r['FLAG']}")
            else:
                proyecto = w_proj.value
                if not proyecto:
                    print("⚠️ Selecciona un proyecto.")
                    return
                res = summarize_by_proyecto(proyecto)
                if not res["found"]:
                    print("⚠️", res["msg"])
                    return
                edit_state["row"] = res["fila"]
                print(f"📊 Proyecto: [{res['Q_RADICADO']}] {res['proyecto']}")
                print(f"- Fila hoja:          {res['fila']}")
                print(f"- Total dependencias: {res['total_dep_xl']} (L+P)")
                print(f"- Pendientes (P):     {res['total_P_xl']}")
                print(f"- Negociadas (L):     {res['total_L_xl']}")
                print(f"- Cubrimiento (CQ):   {res['cub_xl']:.2f}  (formato % en Excel)")
                print("\nDependencias actuales:")
                if not res["detalles"]:
                    print("  (Sin dependencias registradas)")
                else:
                    for d in res["detalles"]:
                        print(f"  · {d['equipo']} → {d['FLAG']}")

                w_edit_avance.value   = res["avance"]
                w_edit_estimado.value = res["estimado"]
                refresh_edit_metrics(res["linea_base"], res["avance"], res["estimado"])

                dep_rows_edit.clear()
                dep_container_edit.children = []
                for d in res["detalles"]:
                    add_dep_row_edit(d["equipo"], d["FLAG"], d.get("descripcion", ""))
                add_dep_row_edit()

    btn_consultar.on_click(on_consultar_clicked)

    def on_limpiar_clicked(b):
        w_equipo.value = ""
        w_proj.value   = ""
        out_resumen.clear_output()
        out_edit_calc.clear_output()
        edit_state["row"] = None
        dep_rows_edit.clear()
        dep_container_edit.children = []

    btn_limpiar.on_click(on_limpiar_clicked)

    def on_refrescar_clicked(b):
        try:
            load_catalogs()
        except Exception as e:
            out_resumen.clear_output()
            with out_resumen:
                print("⚠️ Error recargando catálogos:", e)
            return

        w_equipo.options = sorted(catalogs.get("celulas_dep", []))
        w_proj.options   = get_all_project_names()
        w_equipo.value   = ""
        w_proj.value     = ""

        out_resumen.clear_output()
        out_edit_calc.clear_output()
        edit_state["row"] = None
        dep_rows_edit.clear()
        dep_container_edit.children = []

        with out_resumen:
            print("🔄 Catálogos y listas recargados desde Excel.")

    btn_refrescar.on_click(on_refrescar_clicked)

    def on_click_edit(b):
        out_edit_calc.clear_output()
        with out_edit_calc:
            row = edit_state.get("row")
            if not row:
                print("⚠️ Primero consulta un proyecto en modo 'Por Proyecto'.")
                return

            dep_list = []
            for r in dep_rows_edit:
                eq = r["equipo"].value
                fl = r["flag"].value
                ds = r["desc"].value
                if eq:
                    dep_list.append({
                        "equipo": eq,
                        "codigo": fl,
                        "descripcion": ds,
                    })

            metrics = update_project_row_and_dependencies(
                row=row,
                avance=w_edit_avance.value,
                estimado=w_edit_estimado.value,
                dep_list=dep_list,
            )
            print("✅ Proyecto actualizado.")
            print(f"Línea base: {metrics['linea_base']:.2f} ({metrics['linea_base']*100:.1f}%)")
            print(f"Avance: {metrics['avance']:.2f} ({metrics['avance']*100:.1f}%)")
            print(f"Variación vs línea base: {metrics['var_vs_lb_pp']:+.1f} pp")
            print(f"% avance vs estimado: {metrics['pct_cumpl']:.1f}%")
            print(f"Dependencias registradas: {len(dep_list)}")

    btn_edit.on_click(on_click_edit)

    edit_box = widgets.VBox(
        [
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Edición de proyecto seleccionado</b>"
            ),
            widgets.HBox([w_edit_avance, w_edit_estimado]),
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Dependencias (P/L + descripción)</b>"
            ),
            btn_add_dep_edit,
            dep_container_edit,
            widgets.HBox([btn_edit]),
            out_edit_calc,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="10px 0 0 0",
            background_color="white",
        ),
    )

    body = widgets.VBox(
        [
            w_tipo,
            box_equipo,
            box_proy,
            widgets.HBox([btn_consultar, btn_limpiar, btn_refrescar]),
            widgets.HTML(
                f"<hr style='margin:8px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            out_resumen,
            edit_box,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel_consulta = widgets.VBox(
        [
            header_box,
            body,
        ],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel_consulta


# ======================================================================
# 12. UI – MÉTRICAS (PÁGINA 3)
# ======================================================================

def _build_tren_equipo_label_html(tren_value: str | None):
    """
    Devuelve HTML con la lista de células/equipos asociados a un Tren,
    según la relación Area/Tren/CoE (E) ↔ Célula (F) en hoja Datos.
    """
    if not tren_value:
        txt = "(sin filtro de Tren aplicado)"
        return (
            f"<div style='font-family:Segoe UI, Arial; font-size:11px; "
            f"color:{DARK_COLOR}; opacity:0.8;'>{txt}</div>"
        )

    equipos = sorted([c for c, t in CELULA_TREN_MAP.items()
                      if str(t).strip() == str(tren_value).strip()])
    if not equipos:
        txt = "No se encontraron células para este Tren en la hoja Datos (cols. E–F)."
    else:
        listado = ", ".join(equipos)
        txt = f"<b>Células del Tren seleccionado:</b> {listado}"

    return (
        f"<div style='font-family:Segoe UI, Arial; font-size:11px; "
        f"color:{DARK_COLOR}; opacity:0.9;'>{txt}</div>"
    )


def build_metrics_panel():
    """Panel de métricas agregadas con gráficos y filtros por Tren / Célula."""

    header_box = build_header(
        "Métricas",
        "Visión agregada de proyectos y dependencias por Tren y Célula"
    )

    # --- Alcance de las métricas ---
    w_scope = widgets.ToggleButtons(
        options=[
            ("Todos", "all"),
            ("Por Tren (Area/Tren/CoE)", "area"),
            ("Por Célula / Equipo", "celula"),
        ],
        description="Ámbito:",
        style={"description_width": "70px"},
        layout=widgets.Layout(width="860px"),
    )

    # --- Filtro por Tren (Area/Tren/CoE) ---
    w_tren = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Tren:",
        placeholder="Filtrar por Area/Tren/CoE (opcional)",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "70px"},
    )
    btn_clear_tren = widgets.Button(
        icon="times",
        tooltip="Limpiar Tren",
        layout=widgets.Layout(width="36px", height="30px", margin="0 0 0 4px"),
    )

    def on_clear_tren(b):
        w_tren.value = ""

    btn_clear_tren.on_click(on_clear_tren)

    tren_label = widgets.HTML(
        value=_build_tren_equipo_label_html(None),
        layout=widgets.Layout(width="860px"),
    )

    box_tren = widgets.VBox(
        [
            widgets.HBox([w_tren, btn_clear_tren]),
            tren_label,
        ],
        layout=widgets.Layout(margin="4px 0 8px 0"),
    )

    # --- Filtro por Célula ---
    w_celula = widgets.Combobox(
        options=catalogs.get("celulas_dep", []),
        description="Célula:",
        placeholder="Selecciona una célula / equipo",
        ensure_option=False,
        layout=widgets.Layout(width="520px"),
        style={"description_width": "70px"},
    )
    btn_clear_cel = widgets.Button(
        icon="times",
        tooltip="Limpiar Célula",
        layout=widgets.Layout(width="36px", height="30px", margin="0 0 0 4px"),
    )

    def on_clear_cel(b):
        w_celula.value = ""

    btn_clear_cel.on_click(on_clear_cel)

    box_celula = widgets.HBox([w_celula, btn_clear_cel])

    # Mostrar / ocultar según scope
    def toggle_filters(*_):
        if w_scope.value == "area":
            box_tren.layout.display = "block"
            box_celula.layout.display = "none"
        elif w_scope.value == "celula":
            box_tren.layout.display = "none"
            box_celula.layout.display = "flex"
        else:
            box_tren.layout.display = "none"
            box_celula.layout.display = "none"

    w_scope.observe(lambda ch: toggle_filters(), names="value")
    toggle_filters()

    # --- Botones de acción ---
    btn_calc = widgets.Button(
        description="Calcular métricas",
        button_style="info",
        icon="bar-chart",
        layout=widgets.Layout(width="220px", height="35px"),
    )
    btn_clear = widgets.Button(
        description="Limpiar filtros",
        button_style="",
        icon="eraser",
        layout=widgets.Layout(width="180px", height="35px", margin="0 0 0 8px"),
    )
    btn_refresh = widgets.Button(
        description="Refrescar catálogos",
        button_style="",
        icon="refresh",
        layout=widgets.Layout(width="200px", height="35px", margin="0 0 0 8px"),
    )

    # --- Salida: texto + gráfico ---
    out_text = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="220px",
            background_color="white",
            overflow_y="auto",
        )
    )
    out_plot = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            width="100%",
            height="260px",
            background_color="white",
            overflow_y="auto",
        )
    )

    # --- Lógica de acciones ---
    def on_calc_clicked(b):
        out_text.clear_output()
        out_plot.clear_output()

        scope = w_scope.value
        filt_val = None

        if scope == "area":
            filt_val = w_tren.value or None
            tren_label.value = _build_tren_equipo_label_html(filt_val)
        elif scope == "celula":
            filt_val = w_celula.value or None

        try:
            metrics = compute_metrics(scope=scope, filter_value=filt_val)
        except Exception as e:
            with out_text:
                print("❌ Error calculando métricas:", e)
            return

        with out_text:
            print("📊 Métricas agregadas")
            if scope == "all":
                print("- Ámbito: todos los proyectos activos.")
            elif scope == "area":
                print(f"- Ámbito: Tren = {filt_val or '(sin seleccionar)'}")
            else:
                print(f"- Ámbito: Célula = {filt_val or '(sin seleccionar)'}")

            print(f"\nProyectos considerados: {metrics['total_projects']}")
            print(f"Total dependencias (L+P): {metrics['total_dep']:.0f}")
            print(f"  · Negociadas (L):       {metrics['total_L']:.0f}")
            print(f"  · Pendientes (P):       {metrics['total_P']:.0f}")
            print(f"Cobertura (P / total):    {metrics['cobertura_pct']:.1f}%")

            print("\nAvance promedio (0–1):")
            print(f"  · Global:            {metrics['avg_avance']:.2f}")
            print(f"  · Priorizados (SI):  {metrics['avg_pri']:.2f}")
            print(f"  · No priorizados:    {metrics['avg_no_pri']:.2f}")
            print(f"Proyectos priorizados (SI): {metrics['num_pri']}")

        with out_plot:
            if metrics["total_dep"] > 0:
                fig, ax = plt.subplots(figsize=(4.5, 3.0))
                ax.bar(["Negociadas (L)", "Pendientes (P)"],
                       [metrics["total_L"], metrics["total_P"]])
                ax.set_title("Distribución de dependencias L / P")
                ax.set_ylabel("Número de dependencias")
                plt.tight_layout()
                plt.show()
            else:
                print("Sin dependencias registradas en el ámbito seleccionado.")

    btn_calc.on_click(on_calc_clicked)

    def on_clear_clicked(b):
        w_scope.value = "all"
        w_tren.value = ""
        w_celula.value = ""
        tren_label.value = _build_tren_equipo_label_html(None)
        out_text.clear_output()
        out_plot.clear_output()
        toggle_filters()

    btn_clear.on_click(on_clear_clicked)

    def on_refresh_clicked(b):
        out_text.clear_output()
        out_plot.clear_output()
        try:
            load_catalogs()
        except Exception as e:
            with out_text:
                print("❌ Error recargando catálogos:", e)
            return

        w_tren.options = catalogs.get("area_tren_coe", [])
        w_celula.options = catalogs.get("celulas_dep", [])
        tren_label.value = _build_tren_equipo_label_html(None)
        w_tren.value = ""
        w_celula.value = ""
        with out_text:
            print("🔄 Catálogos recargados desde hoja Datos.")

    btn_refresh.on_click(on_refresh_clicked)

    body = widgets.VBox(
        [
            w_scope,
            box_tren,
            box_celula,
            widgets.HBox([btn_calc, btn_clear, btn_refresh]),
            widgets.HTML(
                f"<hr style='margin:10px 0;border:0;border-top:2px solid {PRIMARY_COLOR};'>"
            ),
            out_text,
            out_plot,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel = widgets.VBox(
        [header_box, body],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel





# ======================================================================
# 13. LÓGICA DE MESA DE EXPERTOS / ALISTAMIENTO
# ======================================================================

def _collect_board_projects(tren_filter: Optional[str] = None):
    """
    Devuelve la lista de proyectos activos (Nuevo / En curso) para las mesas.

    Cada item:
      {
        'row', 'id', 'q_rad', 'estado', 'priorizado',
        'nombre', 'descripcion_corta',
        'contribucion', 'inic_estrategica',
        'total_dep', 'total_L', 'total_P', 'cub',
        'rating_po',
        'pendientes_list', 'negociadas_list'
      }

    Si tren_filter no es None, solo trae proyectos que tengan al menos una
    dependencia P/L en alguna célula asociada a ese Tren (según CELULA_TREN_MAP).
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    header_row = get_header_row_proyectos(ws)

    id_idx    = column_index_from_string(COLS["ID"])
    q_idx     = column_index_from_string(COLS["Q_RADICADO"])
    pri_idx   = column_index_from_string(COLS["PRIORIZADO"])
    estado_idx= column_index_from_string(COLS["ESTADO_PROYECTO"])
    nom_idx   = column_index_from_string(COLS["NOMBRE_PROYECTO"])
    desc_idx  = column_index_from_string(COLS["DESCRIPCION_PROYECTO"])
    cont_idx  = column_index_from_string(COLS["CONTRIBUCION"])
    inic_idx  = column_index_from_string(COLS["INICIATIVA_ESTRATEGICA"])

    total_dep_idx = column_index_from_string(COLS["TOTAL_DEP"])
    total_L_idx   = column_index_from_string(COLS["TOTAL_L"])
    total_P_idx   = column_index_from_string(COLS["TOTAL_P"])
    cub_idx       = column_index_from_string(COLS["CUBRIMIENTO_DEP"])
    rating_idx    = column_index_from_string(COLS["RATING_PO_SYNC"])

    # Mapa Tren → lista de células, para filtrar
    tren_filter = tren_filter or None
    if tren_filter:
        equipos_tren = [c for c, t in CELULA_TREN_MAP.items()
                        if str(t).strip() == str(tren_filter).strip()]
    else:
        equipos_tren = []

    proyectos = []

    for row in range(START_ROW_PROYECTOS, ws.max_row + 1):
        estado_val = ws.cell(row=row, column=estado_idx).value
        if not estado_val:
            continue
        estado_str = str(estado_val).strip().lower()
        if estado_str not in ("nuevo", "en curso", "en curso "):
            # Solo proyectos activos en las mesas
            continue

        # Filtro por Tren (si aplica)
        if tren_filter and equipos_tren:
            match_tren = False
            for cel in equipos_tren:
                flag_col_idx = find_column_by_header_in_range(
                    ws, cel, FLAG_START_COL, FLAG_END_COL, header_row
                )
                if not flag_col_idx:
                    continue
                flag_val = ws.cell(row=row, column=flag_col_idx).value
                if flag_val and str(flag_val).strip().upper() in ("P", "L"):
                    match_tren = True
                    break
            if not match_tren:
                continue

        id_val  = ws.cell(row=row, column=id_idx).value
        q_val   = ws.cell(row=row, column=q_idx).value
        pri_val = ws.cell(row=row, column=pri_idx).value
        nom_val = ws.cell(row=row, column=nom_idx).value
        desc_val= ws.cell(row=row, column=desc_idx).value
        cont_val= ws.cell(row=row, column=cont_idx).value
        inic_val= ws.cell(row=row, column=inic_idx).value

        total_dep = to_num_cell(ws.cell(row=row, column=total_dep_idx).value)
        total_L   = to_num_cell(ws.cell(row=row, column=total_L_idx).value)
        total_P   = to_num_cell(ws.cell(row=row, column=total_P_idx).value)
        cub       = to_num_cell(ws.cell(row=row, column=cub_idx).value)
        rating_po = to_num_cell(ws.cell(row=row, column=rating_idx).value)

        pendientes = []
        negociadas = []
        for equipo, _desc_header in DEP_MAPPING.items():
            flag_idx = find_column_by_header_in_range(
                ws, equipo, FLAG_START_COL, FLAG_END_COL, header_row
            )
            if not flag_idx:
                continue
            flag_val = ws.cell(row=row, column=flag_idx).value
            if not flag_val:
                continue
            flag_up = str(flag_val).strip().upper()
            if flag_up == "P":
                pendientes.append(equipo)
            elif flag_up == "L":
                negociadas.append(equipo)

        desc_short = ""
        if desc_val:
            s = str(desc_val).strip()
            desc_short = s if len(s) <= 80 else s[:77] + "..."

        proyectos.append(
            {
                "row": row,
                "id": id_val,
                "q_rad": q_val,
                "estado": estado_val,
                "priorizado": (str(pri_val).strip().upper()
                               if pri_val not in (None, "") else ""),
                "nombre": nom_val,
                "descripcion_corta": desc_short,
                "contribucion": to_num_cell(cont_val),
                "inic_estrategica": inic_val or "",
                "total_dep": total_dep,
                "total_L": total_L,
                "total_P": total_P,
                "cub": cub,
                "rating_po": rating_po,
                "pendientes_list": pendientes,
                "negociadas_list": negociadas,
            }
        )

    # Orden por Q + nombre para consistencia
    proyectos.sort(key=lambda x: (str(x["q_rad"]), str(x["nombre"])))
    return proyectos


def _update_po_rating(row: int, rating_value: int):
    """Actualiza el rating de Mesa de Alistamiento (PO Sync) en Excel."""
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    rating_idx = column_index_from_string(COLS["RATING_PO_SYNC"])
    ws.cell(row=row, column=rating_idx).value = int(rating_value)
    wb.save(EXCEL_PATH)


def _update_expert_prioritization(row: int, priorizado: str):
    """
    Actualiza la decisión de priorización de Mesa de Expertos.
    Solo toca la columna PRIORIZADO; contribución / iniciativa
    se dejan como información de referencia.
    """
    wb = load_workbook()
    ws = get_ws_proyectos(wb)
    pri_idx = column_index_from_string(COLS["PRIORIZADO"])
    ws.cell(row=row, column=pri_idx).value = priorizado
    wb.save(EXCEL_PATH)



# ======================================================================
# 14. UI – MESA DE EXPERTOS (PÁGINA 4)
# ======================================================================

def build_expert_board_panel():
    """
    Panel de Mesa de Expertos:
      - Filtra proyectos activos (Nuevo / En curso) por Tren y Q.
      - Muestra detalle de cada proyecto (contribución, inic. estrat., deps P/L).
      - Permite registrar la decisión de priorización (PRIORIZADO: SI/NO/ vacío).
    """
    header_box = build_header(
        "Mesa de Expertos",
        "Priorización de iniciativas por contribución e impacto de dependencias"
    )

    # --- Filtros de tablero ---
    w_tren = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Tren:",
        placeholder="Filtrar por Area/Tren/CoE (opcional)",
        ensure_option=False,
        layout=widgets.Layout(width="420px"),
        style={"description_width": "60px"},
    )
    btn_clear_tren = widgets.Button(
        icon="times",
        tooltip="Limpiar Tren",
        layout=widgets.Layout(width="32px", height="30px", margin="0 0 0 4px"),
    )

    def on_clear_tren(b):
        w_tren.value = ""

    btn_clear_tren.on_click(on_clear_tren)

    w_q = widgets.Combobox(
        options=catalogs.get("q_rad", []),
        description="Q:",
        placeholder="Filtrar por Q (opcional)",
        ensure_option=False,
        layout=widgets.Layout(width="220px"),
        style={"description_width": "30px"},
    )
    btn_clear_q = widgets.Button(
        icon="times",
        tooltip="Limpiar Q",
        layout=widgets.Layout(width="32px", height="30px", margin="0 0 0 4px"),
    )

    def on_clear_q(b):
        w_q.value = ""

    btn_clear_q.on_click(on_clear_q)

    w_only_active = widgets.Checkbox(
        value=True,
        description="Solo proyectos activos (Nuevo/En curso)",
        indent=False,
        layout=widgets.Layout(width="320px", margin="0 0 0 12px"),
    )

    # --- Botones de acción ---
    btn_load = widgets.Button(
        description="Cargar tablero",
        button_style="info",
        icon="refresh",
        layout=widgets.Layout(width="190px", height="34px"),
    )
    btn_clear = widgets.Button(
        description="Limpiar selección",
        button_style="",
        icon="eraser",
        layout=widgets.Layout(width="190px", height="34px", margin="0 0 0 8px"),
    )

    # --- Estado interno ---
    board_state = {
        "projects": [],
    }

    # --- Selección de proyecto ---
    w_project = widgets.Dropdown(
        options=[],
        description="Proyecto:",
        layout=widgets.Layout(width="820px"),
        style={"description_width": "80px"},
    )

    # --- Resumen / lista de proyectos (log superior muy simple) ---
    out_list = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="150px",
            background_color="white",
        )
    )

    # --- Detalle del proyecto seleccionado ---
    w_proj_summary = widgets.HTML(
        value="",
        layout=widgets.Layout(width="640px"),
    )

    w_semaforo = widgets.HTML(
        value="",
        layout=widgets.Layout(width="220px"),
    )

    w_prior_decision = widgets.ToggleButtons(
        options=[
            ("Sin asignar", ""),
            ("Priorizado (SI)", "SI"),
            ("No priorizado (NO)", "NO"),
        ],
        description="Decisión:",
        style={"description_width": "80px"},
        layout=widgets.Layout(width="420px"),
    )

    btn_save_decision = widgets.Button(
        description="Guardar y siguiente",
        button_style="success",
        icon="save",
        layout=widgets.Layout(width="210px", height="34px", margin="0 0 0 10px"),
    )

    out_detail = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="190px",
            background_color="white",
        )
    )

    out_save = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="6px",
            width="100%",
            height="70px",
            background_color="white",
        )
    )

    # --- Handlers de lógica ---

    def _refresh_project_dropdown():
        """Reconstruye las opciones del combo de proyectos a partir de board_state."""
        projects = board_state["projects"]
        if not projects:
            w_project.options = []
            w_project.value = None
            return

        opts = []
        for idx, p in enumerate(projects):
            pri = p.get("priorizado", "") or "-"
            q_val = p.get("q_rad") or "-"
            cid = p.get("id") or "?"
            name = p.get("nombre") or "(sin nombre)"
            label = f"[{q_val}] #{cid} · {name} · PRI={pri}"
            opts.append((label, idx))

        w_project.options = opts
        # seleccionar el primero por defecto
        w_project.value = 0

    def on_load_clicked(b):
        out_list.clear_output()
        out_detail.clear_output()
        out_save.clear_output()
        w_proj_summary.value = ""
        w_semaforo.value = ""
        w_project.options = []
        w_project.value = None

        tren_filter = w_tren.value or None
        q_filter = (w_q.value or "").strip() or None

        try:
            projects = _collect_board_projects(tren_filter=tren_filter)
        except Exception as e:
            with out_list:
                print("❌ Error cargando tablero de proyectos:", e)
            return

        # _collect_board_projects ya filtra por estado activo (Nuevo / En curso),
        # pero respetamos el flag por si en el futuro incluye más estados.
        if w_only_active.value:
            filtered = projects
        else:
            filtered = projects

        if q_filter:
            filtered = [
                p for p in filtered
                if (p.get("q_rad") is not None and str(p["q_rad"]) == q_filter)
            ]

        board_state["projects"] = filtered

        with out_list:
            if not filtered:
                print("⚠️ No se encontraron proyectos para los filtros aplicados.")
            else:
                total = len(filtered)
                con_dec = sum(
                    1 for p in filtered
                    if (p.get("priorizado", "") or "").upper() in ("SI", "NO")
                )
                pend = total - con_dec
                print(
                    f"Proyectos en tablero: {total} · "
                    f"Con decisión (SI/NO): {con_dec} · "
                    f"Pendientes: {pend}"
                )

        _refresh_project_dropdown()
        _on_project_change(None)

    btn_load.on_click(on_load_clicked)

    def on_clear_clicked(b):
        w_tren.value = ""
        w_q.value = ""
        board_state["projects"] = []
        w_project.options = []
        w_project.value = None
        w_proj_summary.value = ""
        w_semaforo.value = ""
        out_list.clear_output()
        out_detail.clear_output()
        out_save.clear_output()

    btn_clear.on_click(on_clear_clicked)

    def _on_project_change(change):
        out_detail.clear_output()
        out_save.clear_output()
        w_proj_summary.value = ""
        w_semaforo.value = ""

        projects = board_state["projects"]
        idx = w_project.value
        if projects is None or projects == [] or idx is None:
            return

        if idx < 0 or idx >= len(projects):
            return

        p = projects[idx]

        # Resumen superior (texto más grande)
        qv = p.get("q_rad") or "-"
        cid = p.get("id") or "?"
        name = p.get("nombre") or "(sin nombre)"
        pri = p.get("priorizado", "") or "-"
        ini = p.get("inic_estrategica") or "-"
        contrib = p.get("contribucion", 0.0) or 0.0

        w_proj_summary.value = f"""
        <div style="font-family:Segoe UI, Arial; font-size:14px; color:{DARK_COLOR}; line-height:1.5;">
          <div style="font-size:18px; font-weight:600; margin-bottom:6px;">
            [{qv}] #{cid} · {name}
          </div>
          <div><b>Priorizado actual:</b> {pri}</div>
          <div><b>Contribución (experto):</b> {contrib:.2f}</div>
          <div><b>Iniciativa estratégica:</b> {ini}</div>
        </div>
        """

        # Semáforo de dependencias (a la derecha)
        total_dep = int(p.get("total_dep", 0) or 0)
        total_L = int(p.get("total_L", 0) or 0)
        total_P = int(p.get("total_P", 0) or 0)
        w_semaforo.value = build_semaforo_block(
            total_dep, total_L, total_P, title="Dependencias P/L"
        )

        # Setear la decisión actual
        current_pri = p.get("priorizado", "") or ""
        if current_pri not in ("SI", "NO", ""):
            current_pri = ""
        w_prior_decision.value = current_pri

        # Detalle de dependencias
        with out_detail:
            print("📌 Dependencias registradas:")
            if total_dep == 0:
                print("  · (Sin dependencias registradas)")
            else:
                pend = p.get("pendientes_list", []) or []
                nego = p.get("negociadas_list", []) or []
                print(f"  · Pendientes (P): {len(pend)}")
                if pend:
                    for e in pend:
                        print(f"      - {e}")
                print(f"  · Negociadas (L): {len(nego)}")
                if nego:
                    for e in nego:
                        print(f"      - {e}")

    w_project.observe(_on_project_change, names="value")

    def on_save_decision(b):
        out_save.clear_output()
        with out_save:
            projects = board_state["projects"]
            idx = w_project.value
            if projects is None or projects == [] or idx is None:
                print("⚠️ No hay proyecto seleccionado.")
                return

            p = projects[idx]
            row = p.get("row")
            if not row:
                print("⚠️ No se encontró la fila del proyecto en Excel.")
                return

            decision = w_prior_decision.value or ""
            try:
                _update_expert_prioritization(row=row, priorizado=decision)
            except PermissionError as e:
                print("❌ No se pudo guardar la decisión (archivo bloqueado).")
                print("   Cierra el Excel si está abierto. Detalle:", e)
                return
            except Exception as e:
                print("❌ Error inesperado guardando decisión:", e)
                return

            # Actualizar en memoria
            p["priorizado"] = decision
            projects[idx] = p
            board_state["projects"] = projects

            # Mover al siguiente proyecto si existe
            next_idx = idx + 1 if idx + 1 < len(projects) else idx
            _refresh_project_dropdown()
            w_project.value = next_idx

            print(f"✅ Decisión guardada para el proyecto #{p.get('id')}: PRIORIZADO = '{decision or '(vacío)'}'.")

    btn_save_decision.on_click(on_save_decision)

    # --- Layout del panel ---
    filtros_box = widgets.VBox(
        [
            widgets.HBox([w_tren, btn_clear_tren, w_q, btn_clear_q]),
            widgets.HBox([w_only_active]),
            widgets.HBox([btn_load, btn_clear]),
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            margin="8px 0",
            background_color="white",
        ),
    )

    decision_box = widgets.HBox(
        [
            w_prior_decision,
            btn_save_decision,
        ],
        layout=widgets.Layout(margin="4px 0 8px 0"),
    )

    # HBox con resumen a la izquierda y semáforo a la derecha
    header_detail_box = widgets.HBox(
        [w_proj_summary, w_semaforo],
        layout=widgets.Layout(
            margin="4px 0",
            justify_content="space-between",
            align_items="flex-start",
        ),
    )

    detail_box = widgets.VBox(
        [
            header_detail_box,
            decision_box,
            out_detail,
            out_save,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            margin="8px 0 0 0",
            background_color="white",
        ),
    )

    body = widgets.VBox(
        [
            filtros_box,
            widgets.HTML(
                f"<b style='color:{DARK_COLOR};'>Resumen de tablero</b>"
            ),
            out_list,
            widgets.HTML(
                f"<b style='color:{DARK_COLOR}; margin-top:8px;'>Detalle de proyecto seleccionado</b>"
            ),
            w_project,
            detail_box,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel = widgets.VBox(
        [header_box, body],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel








# ======================================================================
# 15. UI – MESA DE ALISTAMIENTO (PO SYNC) (PÁGINA 5)
# ======================================================================

def build_po_sync_panel():
    """
    Panel de Mesa de Alistamiento (PO Sync):
      - Filtra proyectos activos por Tren y Q.
      - Opcionalmente, sólo proyectos ya priorizados (PRIORIZADO = SI).
      - Permite registrar rating 1–5 estrellas (RATING_PO_SYNC).
    """
    header_box = build_header(
        "Mesa de Alistamiento (PO Sync)",
        "Evaluación de alistamiento y madurez de proyectos mediante rating 1–5"
    )

    # --- Filtros ---
    w_tren = widgets.Combobox(
        options=catalogs.get("area_tren_coe", []),
        description="Tren:",
        placeholder="Filtrar por Area/Tren/CoE (opcional)",
        ensure_option=False,
        layout=widgets.Layout(width="420px"),
        style={"description_width": "60px"},
    )
    btn_clear_tren = widgets.Button(
        icon="times",
        tooltip="Limpiar Tren",
        layout=widgets.Layout(width="32px", height="30px", margin="0 0 0 4px"),
    )

    def on_clear_tren(b):
        w_tren.value = ""

    btn_clear_tren.on_click(on_clear_tren)

    w_q = widgets.Combobox(
        options=catalogs.get("q_rad", []),
        description="Q:",
        placeholder="Filtrar por Q (opcional)",
        ensure_option=False,
        layout=widgets.Layout(width="220px"),
        style={"description_width": "30px"},
    )
    btn_clear_q = widgets.Button(
        icon="times",
        tooltip="Limpiar Q",
        layout=widgets.Layout(width="32px", height="30px", margin="0 0 0 4px"),
    )

    def on_clear_q(b):
        w_q.value = ""

    btn_clear_q.on_click(on_clear_q)

    w_only_pri = widgets.Checkbox(
        value=True,
        description="Sólo proyectos priorizados (PRI=SI)",
        indent=False,
        layout=widgets.Layout(width="280px", margin="0 0 0 12px"),
    )

    btn_load = widgets.Button(
        description="Cargar tablero PO",
        button_style="info",
        icon="refresh",
        layout=widgets.Layout(width="200px", height="34px"),
    )
    btn_clear = widgets.Button(
        description="Limpiar selección",
        button_style="",
        icon="eraser",
        layout=widgets.Layout(width="190px", height="34px", margin="0 0 0 8px"),
    )

    # --- Estado interno ---
    board_state = {
        "projects": [],
    }

    # --- Selección de proyecto ---
    w_project = widgets.Dropdown(
        options=[],
        description="Proyecto:",
        layout=widgets.Layout(width="820px"),
        style={"description_width": "80px"},
    )

    # Resumen X / Y
    w_stats = widgets.HTML(
        value="",
        layout=widgets.Layout(width="100%"),
    )

    # --- Detalle + rating ---
    w_proj_summary = widgets.HTML(
        value="",
        layout=widgets.Layout(width="100%"),
    )

    rating_widget, set_rating, get_rating = create_star_rating_widget(initial=3)

    btn_save_rating = widgets.Button(
        description="Guardar rating y siguiente",
        button_style="success",
        icon="star",
        layout=widgets.Layout(width="210px", height="34px", margin="0 0 0 10px"),
    )

    out_detail = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            background_color="white",
        )
    )

    # Log de guardado
    out_save = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="6px",
            width="100%",
            background_color="white",
        )
    )

    # --- Helpers ---

    def _update_stats_label():
        projects = board_state.get("projects", []) or []
        total = len(projects)
        rated = sum(
            1 for p in projects
            if int(p.get("rating_po", 0) or 0) in (1, 2, 3, 4, 5)
        )
        pending = max(total - rated, 0)

        if total == 0:
            w_stats.value = (
                f"<div style='font-family:Segoe UI, Arial; font-size:11px; "
                f"color:{DARK_COLOR}; opacity:0.8;'>"
                f"Sin proyectos cargados para los filtros actuales."
                f"</div>"
            )
        else:
            w_stats.value = (
                f"<div style='font-family:Segoe UI, Arial; font-size:11px; "
                f"color:{DARK_COLOR}; opacity:0.9;'>"
                f"<b>Proyectos para evaluación PO Sync:</b> {total} · "
                f"<b>Con rating:</b> {rated} · "
                f"<b>Pendientes:</b> {pending}"
                f"</div>"
            )

    def _refresh_project_dropdown():
        projects = board_state["projects"]
        if not projects:
            w_project.options = []
            w_project.value = None
            return

        opts = []
        for idx, p in enumerate(projects):
            qv = p.get("q_rad") or "-"
            cid = p.get("id") or "?"
            name = p.get("nombre") or "(sin nombre)"
            rating = int(p.get("rating_po", 0) or 0)
            label = f"[{qv}] #{cid} · {name} · Rating={rating if rating > 0 else '-'}"
            opts.append((label, idx))

        w_project.options = opts
        w_project.value = 0

    # --- Handlers ---

    def on_load_clicked(b):
        out_detail.clear_output()
        out_save.clear_output()
        w_proj_summary.value = ""
        w_project.options = []
        w_project.value = None
        set_rating(3)

        tren_filter = w_tren.value or None
        q_filter = (w_q.value or "").strip() or None

        try:
            projects = _collect_board_projects(tren_filter=tren_filter)
        except Exception as e:
            board_state["projects"] = []
            _update_stats_label()
            with out_save:
                print("❌ Error cargando proyectos para PO Sync:", e)
            return

        filtered = projects
        if w_only_pri.value:
            filtered = [
                p for p in filtered
                if str(p.get("priorizado", "")).strip().upper() == "SI"
            ]

        if q_filter:
            filtered = [
                p for p in filtered
                if (p.get("q_rad") is not None and str(p["q_rad"]) == q_filter)
            ]

        board_state["projects"] = filtered
        _update_stats_label()
        _refresh_project_dropdown()
        _on_project_change(None)

    btn_load.on_click(on_load_clicked)

    def on_clear_clicked(b):
        w_tren.value = ""
        w_q.value = ""
        board_state["projects"] = []
        w_project.options = []
        w_project.value = None
        w_proj_summary.value = ""
        out_detail.clear_output()
        out_save.clear_output()
        set_rating(3)
        w_stats.value = ""

    btn_clear.on_click(on_clear_clicked)

    def _on_project_change(change):
        out_detail.clear_output()
        out_save.clear_output()
        w_proj_summary.value = ""

        projects = board_state["projects"]
        idx = w_project.value
        if not projects or idx is None:
            return
        if idx < 0 or idx >= len(projects):
            return

        p = projects[idx]

        qv = p.get("q_rad") or "-"
        cid = p.get("id") or "?"
        name = p.get("nombre") or "(sin nombre)"
        pri = p.get("priorizado", "") or "-"
        rating_po = int(p.get("rating_po", 0) or 0)
        total_dep = int(p.get("total_dep", 0) or 0)
        total_L = int(p.get("total_L", 0) or 0)
        total_P = int(p.get("total_P", 0) or 0)

        # Resumen
        w_proj_summary.value = f"""
        <div style="font-family:Segoe UI, Arial; font-size:12px; color:{DARK_COLOR};">
          <div style="font-size:14px; font-weight:600; margin-bottom:3px;">
            [{qv}] #{cid} · {name}
          </div>
          <div><b>Priorizado (Mesa de Expertos):</b> {pri}</div>
          <div><b>Dependencias (L+P):</b> {total_dep} · L={total_L}, P={total_P}</div>
        </div>
        """

        # Rating actual
        if rating_po in (1, 2, 3, 4, 5):
            set_rating(rating_po)
        else:
            set_rating(3)

        # Detalle de dependencias
        with out_detail:
            print("📌 Dependencias registradas:")
            if total_dep == 0:
                print("  · (Sin dependencias registradas)")
            else:
                pend = p.get("pendientes_list", []) or []
                nego = p.get("negociadas_list", []) or []
                print(f"  · Pendientes (P): {len(pend)}")
                if pend:
                    for e in pend:
                        print(f"      - {e}")
                print(f"  · Negociadas (L): {len(nego)}")
                if nego:
                    for e in nego:
                        print(f"      - {e}")

    w_project.observe(_on_project_change, names="value")

    def on_save_rating(b):
        out_save.clear_output()
        with out_save:
            projects = board_state["projects"]
            idx = w_project.value
            if not projects or idx is None:
                print("⚠️ No hay proyecto seleccionado.")
                return

            p = projects[idx]
            row = p.get("row")
            if not row:
                print("⚠️ No se encontró la fila del proyecto en Excel.")
                return

            val = int(get_rating())
            if val < 1:
                val = 1
            if val > 5:
                val = 5

            try:
                _update_po_rating(row=row, rating_value=val)
            except PermissionError as e:
                print("❌ No se pudo guardar el rating (archivo bloqueado).")
                print("   Cierra el Excel si está abierto. Detalle:", e)
                return
            except Exception as e:
                print("❌ Error inesperado guardando rating:", e)
                return

            # Actualizar en memoria
            p["rating_po"] = val
            projects[idx] = p
            board_state["projects"] = projects
            _update_stats_label()
            _refresh_project_dropdown()

            # Ir al siguiente proyecto si existe
            next_idx = idx + 1 if idx + 1 < len(projects) else idx
            w_project.value = next_idx

            print(f"✅ Rating guardado para el proyecto #{p.get('id')}: {val} ⭐.")

    btn_save_rating.on_click(on_save_rating)

    # --- Layout del panel ---
    filtros_box = widgets.VBox(
        [
            widgets.HBox([w_tren, btn_clear_tren, w_q, btn_clear_q]),
            widgets.HBox([w_only_pri]),
            widgets.HBox([btn_load, btn_clear]),
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            margin="8px 0",
            background_color="white",
        ),
    )

    rating_box = widgets.HBox(
        [
            widgets.HTML(
                f"<div style='font-family:Segoe UI, Arial; font-size:12px; color:{DARK_COLOR};"
                f" margin-right:8px;'><b>Rating de alistamiento:</b></div>"
            ),
            rating_widget,
            btn_save_rating,
        ],
        layout=widgets.Layout(margin="6px 0 8px 0", align_items="center"),
    )

    detail_box = widgets.VBox(
        [
            w_proj_summary,
            rating_box,
            out_detail,
            out_save,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            margin="8px 0 0 0",
            background_color="white",
        ),
    )

    body = widgets.VBox(
        [
            filtros_box,
            w_stats,
            widgets.HTML(
                f"<b style='color:{DARK_COLOR}; margin-top:8px;'>Detalle y rating</b>"
            ),
            w_project,
            detail_box,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel = widgets.VBox(
        [header_box, body],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel






# ======================================================================
# 16. FEEDBACK – HOJA SUGERENCIAS (PÁGINA 6)
# ======================================================================

def _append_suggestion(usuario: str, texto: str):
    """
    Agrega una sugerencia a la hoja 'Sugerencias' (A: Usuario, B: Sugerencia).
    """
    wb = load_workbook()
    ws = get_ws_sugerencias(wb)

    next_row = ws.max_row + 1
    if next_row == 2 and ws["A1"].value is None:
        ws["A1"] = "Usuario"
        ws["B1"] = "Sugerencia"
        next_row = 2

    ws.cell(row=next_row, column=1).value = usuario or ""
    ws.cell(row=next_row, column=2).value = texto or ""
    wb.save(EXCEL_PATH)


def _get_last_suggestions(limit: int = 5):
    """
    Devuelve las últimas `limit` sugerencias de la hoja 'Sugerencias'.
    Cada item: {'usuario':..., 'texto':...}
    """
    wb = load_workbook()
    ws = get_ws_sugerencias(wb)

    rows = []
    for row in range(2, ws.max_row + 1):
        usuario = ws.cell(row=row, column=1).value
        texto = ws.cell(row=row, column=2).value
        if usuario is None and texto is None:
            continue
        rows.append(
            {
                "usuario": str(usuario) if usuario is not None else "",
                "texto": str(texto) if texto is not None else "",
            }
        )

    if not rows:
        return []

    return rows[-limit:]


def build_feedback_panel():
    """
    Panel de feedback:
      - Permite enviar sugerencias / bugs / ideas a la hoja Sugerencias.
      - Muestra un pequeño histórico de las últimas N sugerencias.
    """
    header_box = build_header(
        "Feedback y sugerencias",
        "Espacio para ideas, mejoras y bugs de la herramienta GD_v1"
    )

    w_user = widgets.Text(
        description="Nombre:",
        placeholder="Tu nombre, rol o área…",
        layout=widgets.Layout(width="420px"),
        style={"description_width": "70px"},
    )
    w_sugg = widgets.Textarea(
        description="Sugerencia:",
        placeholder="Describe tu sugerencia, mejora o issue…",
        layout=widgets.Layout(width="650px", height="130px"),
        style={"description_width": "80px"},
    )

    btn_send = widgets.Button(
        description="Enviar sugerencia",
        button_style="success",
        icon="paper-plane",
        layout=widgets.Layout(width="220px", height="36px", margin="6px 0 0 0"),
    )
    btn_clear = widgets.Button(
        description="Limpiar",
        button_style="",
        icon="eraser",
        layout=widgets.Layout(width="140px", height="36px", margin="6px 0 0 8px"),
    )

    out_send = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="6px",
            width="100%",
            height="70px",
            background_color="white",
        )
    )

    # Histórico
    w_show_history = widgets.Checkbox(
        value=True,
        description="Ver últimas sugerencias",
        indent=False,
        layout=widgets.Layout(width="220px", margin="0 0 4px 0"),
    )

    out_history = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="8px",
            width="100%",
            height="200px",
            background_color="white",
        )
    )

    def _refresh_history():
        out_history.clear_output()
        if not w_show_history.value:
            return
        with out_history:
            suggestions = []
            try:
                suggestions = _get_last_suggestions(limit=8)
            except Exception as e:
                print("⚠️ No se pudieron leer las sugerencias:", e)
                return

            if not suggestions:
                print("No hay sugerencias registradas todavía.")
                return

            print("📚 Últimas sugerencias registradas:")
            for idx, s in enumerate(suggestions, start=1):
                u = s["usuario"] or "(sin nombre)"
                t = s["texto"] or ""
                print(f"\n{idx}. {u}")
                print(f"   {t}")

    def on_send_clicked(b):
        out_send.clear_output()
        with out_send:
            usuario = (w_user.value or "").strip()
            texto = (w_sugg.value or "").strip()
            if not texto:
                print("⚠️ El campo de sugerencia no puede estar vacío.")
                return

            try:
                _append_suggestion(usuario, texto)
            except PermissionError as e:
                print("❌ No se pudo guardar la sugerencia (archivo bloqueado).")
                print("   Cierra el Excel si está abierto. Detalle:", e)
                return
            except Exception as e:
                print("❌ Error inesperado guardando sugerencia:", e)
                return

            print("✅ Gracias por tu feedback. Sugerencia registrada en la hoja 'Sugerencias'.")
            # Limpiar sólo el texto para permitir enviar varias con el mismo nombre
            w_sugg.value = ""
            _refresh_history()

    btn_send.on_click(on_send_clicked)

    def on_clear_clicked(b):
        w_user.value = ""
        w_sugg.value = ""
        out_send.clear_output()

    btn_clear.on_click(on_clear_clicked)

    def on_show_history_change(change):
        _refresh_history()

    w_show_history.observe(on_show_history_change, names="value")

    # Inicializar histórico
    _refresh_history()

    form_box = widgets.VBox(
        [
            widgets.HBox([w_user]),
            widgets.HBox([w_sugg]),
            widgets.HBox([btn_send, btn_clear]),
            out_send,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="8px 0",
            background_color="white",
        ),
    )

    history_box = widgets.VBox(
        [
            w_show_history,
            out_history,
        ],
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="10px",
            margin="8px 0 0 0",
            background_color="white",
        ),
    )

    body = widgets.VBox(
        [
            form_box,
            history_box,
        ],
        layout=widgets.Layout(
            padding="12px 16px",
            background_color=LIGHT_BG,
            border=f"1px solid {CARD_BORDER}",
            border_radius="0 0 10px 10px",
        ),
    )

    panel = widgets.VBox(
        [header_box, body],
        layout=widgets.Layout(width="900px", margin="20px 0"),
    )
    return panel


# ======================================================================
# 17. APP SHELL – TABS PRINCIPALES
# ======================================================================

def build_main_app_tabs():
    """
    Construye el contenedor principal de la app GD_v1 con pestañas:
      1) Alta de proyectos
      2) Consulta + edición
      3) Métricas
      4) Mesa de Expertos
      5) Mesa de Alistamiento (PO Sync)
      6) Feedback / Sugerencias
    Devuelve un widgets.Tab listo para hacer display().
    """
    try:
        panel_alta      = build_create_form()
    except Exception as e:
        panel_alta = widgets.HTML(
            value=f"<b>Error construyendo pestaña Alta:</b> {e}"
        )

    try:
        panel_consulta  = build_consult_panel()
    except Exception as e:
        panel_consulta = widgets.HTML(
            value=f"<b>Error construyendo pestaña Consulta:</b> {e}"
        )

    try:
        panel_metricas  = build_metrics_panel()
    except Exception as e:
        panel_metricas = widgets.HTML(
            value=f"<b>Error construyendo pestaña Métricas:</b> {e}"
        )

    try:
        panel_expert    = build_expert_board_panel()
    except Exception as e:
        panel_expert = widgets.HTML(
            value=f"<b>Error construyendo pestaña Mesa de Expertos:</b> {e}"
        )

    try:
        panel_po_sync   = build_po_sync_panel()
    except Exception as e:
        panel_po_sync = widgets.HTML(
            value=f"<b>Error construyendo pestaña Mesa PO Sync:</b> {e}"
        )

    try:
        panel_feedback  = build_feedback_panel()
    except Exception as e:
        panel_feedback = widgets.HTML(
            value=f"<b>Error construyendo pestaña Feedback:</b> {e}"
        )

    tabs = widgets.Tab(
        children=[
            panel_alta,
            panel_consulta,
            panel_metricas,
            panel_expert,
            panel_po_sync,
            panel_feedback,
        ],
        layout=widgets.Layout(width="940px"),
    )
    tabs.set_title(0, "Alta proyectos")
    tabs.set_title(1, "Consulta / edición")
    tabs.set_title(2, "Métricas")
    tabs.set_title(3, "Mesa Expertos")
    tabs.set_title(4, "Mesa PO Sync")
    tabs.set_title(5, "Feedback")

    return tabs


def show_gd_app():
    """
    Helper para mostrar la app principal con pestañas + botón de refresco.
    Uso típico en el notebook:
        app = show_gd_app()
    """
    tabs = build_main_app_tabs()

    btn_refresh = widgets.Button(
        description="Refrescar app (releer Excel)",
        button_style="info",
        icon="refresh",
        layout=widgets.Layout(width="260px", height="36px"),
    )

    out_info = widgets.Output(
        layout=widgets.Layout(
            border=f"1px solid {CARD_BORDER}",
            padding="6px",
            width="100%",
            height="60px",
            background_color="white",
            overflow_y="auto",
        )
    )

    def on_refresh(b):
        with out_info:
            out_info.clear_output()
            print("🔄 Recargando aplicación y catálogos desde Excel...")
        new_tabs = build_main_app_tabs()
        root.children = [controls_box, out_info, new_tabs]

    btn_refresh.on_click(on_refresh)

    controls_box = widgets.HBox(
        [btn_refresh],
        layout=widgets.Layout(
            justify_content="flex-start",
            margin="4px 0 4px 0",
        ),
    )

    root = widgets.VBox(
        [controls_box, out_info, tabs],
        layout=widgets.Layout(width="960px", margin="10px 0"),
    )

    display(root)
    return root



# ======================================================================
# 17.b BOOTSTRAP – ARRANCAR AUTOMÁTICAMENTE EN NOTEBOOK
# ======================================================================
# Si estás en un notebook (que es lo habitual), al ejecutar esta celda
# se construye y se muestra la app directamente.
# Si prefieres control manual, puedes comentar estas dos líneas y luego
# llamar show_gd_app() cuando quieras.

try:
    _gd_app_instance = show_gd_app()
except Exception as _e:
    print("⚠️ No se pudo inicializar la app GD_v1 automáticamente.")
    print("   Detalle:", _e)
    print("   Puedes intentar llamando manualmente: show_gd_app()")
